Links to Scrape

1) List of Jobs in India : https://www.payscale.com/research/IN/Job

In [1]:
import scrapy
from scrapy.crawler import CrawlerProcess

In [2]:
import pandas as pd
import numpy as np
import csv
from scrapy import Selector

In [3]:
class Payscale_Spider(scrapy.Spider):
    
    name = 'payscale_spider'

    def start_requests(self):

        url = 'https://www.payscale.com/research/IN/Job'
        yield scrapy.Request(url = url, callback = self.parse_industries)

    
    def parse_industries(self, response):

        # Extract the industry links
        industry_links = response.xpath('//a[@class = "related-content-card"]/@href').extract()
        
        # Extract the industry titles
        industry_titles = response.xpath('//div[@class = "related-content-card__title"]/text()').extract()

        # Crawling through each of the industries to fetch corresponding jobs
        for industry_link in industry_links[:1]:
            yield response.follow(url = 'https://www.payscale.com' + industry_link, callback = self.parse_jobs)

    
    def parse_jobs(self, response):

        # Extract the job links
        job_links = response.xpath('//a[@class = "subcats__links__item"]/@href').extract() 
        
        # Extract the job titles
        job_titles = response.xpath('//a[@class = "subcats__links__item"]/text()').extract()
        
        #print([job[17:len(job) - 7] for job in job_links])
        
        # Crawling through each of the jobs to fetch corresponding salary and other information
        for link in job_links[:1]:
            yield response.follow(url = 'https://www.payscale.com' + link, callback = self.parse_read_more)
    
    def parse_read_more(self, response):
        
        # Clicking the read more
        required_text = response.xpath()


    def parse_salary(self, response):

        html = """""" + response.text + """"""
        sel = Selector(text = html)
        
        # Extract the job title
        job_title = response.url[41:len(response.url)-7]

        # Extract the average salary
        avg_salary = sel.css('span.paycharts__value::text').extract_first()

        # Extract base salary, bonus, profit sharing, commission and total pay 
        salary_component_title_list = sel.css('div.tablerow__title::text').extract()
        salary_component_value_list = sel.css('div.tablerow__value::text').extract()

        # Extract pay by experience percent level
        pay_percent_component_title_list = sel.css('div.delta-table.pay-by-experience__delta-chart > div.entry > div.name > a::text').extract()
        pay_percent_sign_list = sel.css('div.delta-table.pay-by-experience__delta-chart > div.entry > div.arrow').xpath('./span/@class').extract()
        pay_percent_value_list_without_sign = sel.css('div.delta-table.pay-by-experience__delta-chart > div.entry').xpath('./div[@class="arrow"]/text()').extract()
        pay_percent_component_value_list = list()
        for sign, value in zip(pay_percent_sign_list, pay_percent_value_list_without_sign):
            if sign == 'negative':
                pay_percent_component_value_list.append('-' + value)
            else:
                pay_percent_component_value_list.append(value)

        # Extract pay by experience years level
        pay_by_experience_years_text = sel.xpath('//p[@class="expandable__paragraph"]/text()').extract_first()
        pay_by_experience_years_component_title_list = list()
        pay_by_experience_years_component_value_list = list()
        
        temp_numbers_list = list()
        for text in pay_by_experience_years_text.split():
            if any(char.isdigit() for char in text):
                temp_numbers_list.append(text)
        
        for i in range(0, len(temp_numbers_list) - 2, 3):
            pay_by_experience_years_component_title_list.append(temp_numbers_list[i])
            pay_by_experience_years_component_value_list.append(temp_numbers_list[i + 1])

        # Extract Gender Breakdown Percentage
        gender_component_title_list = sel.css('div.gender__label::text').extract()
        temp_gender_component_value_list = sel.css('div.gender__value::text').extract()
        gender_component_value_list = list()
        for i in range(0, len(temp_gender_component_value_list), 2):
            gender_component_value_list.append(temp_gender_component_value_list[i])
        
        # Extract Health benefits item name
        health_benefits_component_title_list = sel.css('div.healthbenefits__item-name::text').extract()
        temp_health_benefits_component_value_list = sel.css('h2.healthbenefits__item-value::text').extract()
        health_benefits_component_value_list = list()
        for i in range(0, len(temp_health_benefits_component_value_list), 2):
            health_benefits_component_value_list.append(temp_health_benefits_component_value_list[i])

        # Popular Skills
        popular_skills_component_title_list = sel.css('a.popular-skill__name::text').extract()
        popular_skills_component_value_list = sel.css('div.popular-skill__pay::text').extract()

        # Extract Skills that affect salary
        affect_skills_component_title_list = sel.css('div.pay-diff-by-dimension div.name > a::text').extract()
        temp_affect_skills_component_value_list = sel.css('div.pay-diff-by-dimension div.arrow::text').extract()
        temp_sign = response.css('div.pay-diff-by-dimension div.arrow').xpath('./span/@class').extract()
        affect_skills_component_value_list = list()
        for sign, value in zip(temp_sign, temp_affect_skills_component_value_list):
            if sign == 'negative':
                affect_skills_component_value_list.append('-' + value)
            else:
                affect_skills_component_value_list.append(value)

        # Extract Career Paths
        career_path_component_title_list = sel.css('div.tree-chart__leaf_container').xpath('./*[contains(@class,"tree-chart__leaf")]/text()').extract()

        # Extract pay difference by location
        location_component_title_list = sel.css('div.carousel.related-carousel.pay-diff-related-carousel div.related-content-card__title::text').extract()
        temp_location_component_value_list = sel.css('div.carousel.related-carousel.pay-diff-related-carousel div.related-content-card__subtitle::text').extract()
        location_component_value_list = list()
        for value in temp_location_component_value_list:
            if value[0] == '▼':
                location_component_value_list.append('-' + value[2:])
            else:
                location_component_value_list.append(value[2:])
        
        # Extract popular employers
        employers_component_title_list = sel.css('div.carousel.related-carousel.popular-employers-related-carousel div.related-content-card__title::text').extract()
        employers_component_value_list = sel.css('div.carousel.related-carousel.popular-employers-related-carousel div.related-content-card__subtitle::text').extract()
        
        # Extract Related Job Salaries
        related_job_component_title_list = sel.css('span.relatedentity__card-title::text').extract()
        related_job_component_value_list = sel.css('p.relatedentity__card-detail::text').extract()

        # Updating all the values in the dictionary
        salary_dict['job_title'] = job_title
        salary_dict['avg_salary'] = avg_salary
        salary_dict['salary_component'] = salary_component_title_list
        for label, value in zip(salary_component_title_list, salary_component_value_list):
            salary_dict[label] = value
        salary_dict['pay_by_experience_percent'] = pay_percent_component_title_list
        for label, value in zip(pay_percent_component_title_list, pay_percent_component_value_list):
            salary_dict[label] = value
        salary_dict['pay_by_experience_years'] =  pay_by_experience_years_component_title_list
        for label, value in zip( pay_by_experience_years_component_title_list,  pay_by_experience_years_component_value_list):
            salary_dict[label] = value
        salary_dict['gender_component'] =  gender_component_title_list
        for label, value in zip( gender_component_title_list,  gender_component_value_list):
            salary_dict[label] = value
        salary_dict['health_benefits_componentt'] =  health_benefits_component_title_list
        for label, value in zip( health_benefits_component_title_list,  health_benefits_component_value_list):
            salary_dict[label] = value
        salary_dict['popular_skills_component'] =  popular_skills_component_title_list
        for label, value in zip( popular_skills_component_title_list,  popular_skills_component_value_list):
            salary_dict[label] = value
        salary_dict['affect_skills_component'] =  affect_skills_component_title_list
        for label, value in zip( affect_skills_component_title_list,  affect_skills_component_value_list):
            salary_dict[label] = value
        salary_dict['career_path_component'] = career_path_component_title_list
        salary_dict['location_component'] =  location_component_title_list
        for label, value in zip( location_component_title_list, location_component_value_list):
            salary_dict[label] = value
        salary_dict['employers_component'] =  employers_component_title_list
        for label, value in zip(employers_component_title_list,  employers_component_value_list):
            salary_dict[label] = value
        salary_dict['related_job_component'] =  related_job_component_title_list
        for label, value in zip(related_job_component_title_list, related_job_component_value_list):
            salary_dict[label] = value
        
        # # Storing the values in json file
#         df = pd.DataFrame.from_dict(salary_dict, orient='index')
#         df.to_json('' + job_title + '.json')
        print(salary_dict)

salary_dict = dict()

process = CrawlerProcess()
process.crawl(Payscale_Spider)
process.start()

2021-03-02 21:33:02 [scrapy.utils.log] INFO: Scrapy 2.4.1 started (bot: scrapybot)
2021-03-02 21:33:02 [scrapy.utils.log] INFO: Versions: lxml 4.6.1.0, libxml2 2.9.10, cssselect 1.1.0, parsel 1.6.0, w3lib 1.22.0, Twisted 20.3.0, Python 3.8.5 (default, Sep  3 2020, 21:29:08) [MSC v.1916 64 bit (AMD64)], pyOpenSSL 19.1.0 (OpenSSL 1.1.1h  22 Sep 2020), cryptography 3.1.1, Platform Windows-10-10.0.19041-SP0
2021-03-02 21:33:02 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.selectreactor.SelectReactor
2021-03-02 21:33:02 [scrapy.crawler] INFO: Overridden settings:
{}
2021-03-02 21:33:02 [scrapy.extensions.telnet] INFO: Telnet Password: eedbb5c3f09de647
2021-03-02 21:33:02 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.logstats.LogStats']
2021-03-02 21:33:02 [scrapy.middleware] INFO: Enabled downloader middlewares:
['scrapy.downloadermiddlewares.httpauth.HttpAuthMiddleware',
 's

{'job_title': 'Accountant', 'avg_salary': '₹249,659', 'salary_component': ['Base Salary', 'Bonus', 'Profit Sharing', 'Commission', 'Total Pay'], 'Base Salary': '₹123k - ₹506k', 'Bonus': '₹5k - ₹60k', 'Profit Sharing': '₹41 - ₹198k', 'Commission': '₹3k - ₹99k', 'Total Pay': '₹129k - ₹584k', 'pay_by_experience_percent': ['Entry Level', 'Early Career', 'Mid Career', 'Late Career', 'Experienced'], 'Entry Level': '-20%', 'Early Career': '-7%', 'Mid Career': '19%', 'Late Career': '42%', 'Experienced': '58%', 'pay_by_experience_years': ['1', '1-4'], '1': '₹198,631', '1-4': '₹231,272', 'gender_component': ['Male', 'Female', 'Prefer to self-define'], 'Male': '78.5', 'Female': '21.4', 'Prefer to self-define': '0.1', 'health_benefits_componentt': ['Medical', 'Dental', 'Vision', 'None'], 'Medical': '34', 'Dental': '3', 'Vision': '3', 'None': '66', 'popular_skills_component': [], 'affect_skills_component': [], 'career_path_component': [], 'location_component': [], 'employers_component': [], 'relate

In [4]:
# ValueError: arrays must all be same length
# {'job_title': 'Accountant', 'avg_salary': '₹249,369', 'salary_component': ['Base Salary', 'Bonus', 'Profit Sharing', 'Commission', 'Total Pay'], 'Base Salary': '₹123k - ₹507k', 'Bonus': '₹5k - ₹59k', 'Profit Sharing': '₹36 - ₹197k', 'Commission': '₹3k - ₹102k', 'Total Pay': '₹129k - ₹584k', 'pay_by_experience_percent': ['Entry Level', 'Early Career', 'Mid Career', 'Late Career', 'Experienced'], 'Entry Level': '-20%', 'Early Career': '-7%', 'Mid Career': '19%', 'Late Career': '43%', 'Experienced': '62%', 'pay_by_experience_years': ['1', '1-4'], '1': '₹198,831', '1-4': '₹230,933', 'gender_component': ['Male', 'Female', 'Prefer to self-define'], 'Male': '78.7', 'Female': '21.2', 'Prefer to self-define': '0.1', 'health_benefits_componentt': ['Medical', 'Dental', 'Vision', 'None'], 'Medical': '34', 'Dental': '3', 'Vision': '3', 'None': '66', 'popular_skills_component': [], 'affect_skills_component': [], 'career_path_component': [], 'location_component': [], 'employers_component': [], 'related_job_component': []}
# <a class="popular-skill__name" href="/research/IN/Job=Accountant/Salary/ebd17157/General-Ledger-Accounting">General Ledger Accounting</a>

In [3]:
h = """<html><head><script type=\"text/javascript\" async=\"\" src=\"https://cdn.bizible.com/xdc.js?_biz_u=c5d881485d3d485da585260f36966122&amp;_biz_h=802059049&amp;cdn_o=a&amp;jsVer=4.21.02.25\"></script><script type=\"text/javascript\" async=\"\" src=\"//munchkin.marketo.net/159/munchkin.js\"></script><script type=\"text/javascript\" async=\"\" src=\"//j.6sc.co/6si.min.js\"></script><script type=\"text/javascript\" async=\"\" src=\"https://js.driftt.com/include/1615378500000/gmup7bc7xmk3.js\"></script><script src=\"//bat.bing.com/bat.js\" async=\"\"></script><script src=\"//bat.bing.com/bat.js\" async=\"\"></script><script type=\"text/javascript\" async=\"\" src=\"https://snap.licdn.com/li.lms-analytics/insight.min.js\"></script><script type=\"text/javascript\" async=\"\" src=\"https://www.google-analytics.com/plugins/ua/linkid.js\"></script><script type=\"text/javascript\" async=\"\" src=\"https://www.google-analytics.com/analytics.js\"></script><script src=\"https://connect.facebook.net/signals/config/1201912829856435?v=2.9.33&amp;r=stable\" async=\"\"></script><script async=\"\" src=\"https://connect.facebook.net/en_US/fbevents.js\"></script><script async=\"\" src=\"https://www.googletagmanager.com/gtm.js?id=GTM-P3CVWR\"></script><script async=\"\" src=\"https://securepubads.g.doubleclick.net/tag/js/gpt.js\"></script><script>\n          var googletag = googletag || {};\n          googletag.cmd = googletag.cmd || [];</script><link rel=\"preconnect\" href=\"https://cdn-payscale.com\" crossorigin=\"anonymous\"><link rel=\"preconnect\" href=\"https://adservice.google.com\" crossorigin=\"true\"><link rel=\"preconnect\" href=\"https://fonts.gstatic.com\" crossorigin=\"anonymous\"><link rel=\"preconnect\" href=\"https://fonts.googleapis.com\" crossorigin=\"anonymous\"><meta name=\"viewport\" content=\"width=device-width\"><meta charset=\"utf-8\"><title>Accountant Salary in India | PayScale</title><meta name=\"description\" content=\"The average salary for an Accountant in India is ₹249,659. Visit PayScale to research accountant salaries by city, experience, skill, employer and more.\"><link rel=\"canonical\" href=\"https://www.payscale.com/research/IN/Job=Accountant/Salary\"><script>\n              if (typeof window !== 'undefined' && window !== null) {\n                window.dataLayer = window.dataLayer || [];\n                window.dataLayer.push({\n                  rcSection: 'Job',\n                  cacheControl: 'public',\n                  origin: 'payscale'\n                });\n              }</script><link rel=\"icon\" href=\"https://cdn-payscale.com/content/favicon/favicon-32x32.png\" sizes=\"32x32\" type=\"image/png\"><script>!function(t){var o={};function r(e){if(o[e])return o[e].exports;var n=o[e]={i:e,l:!1,exports:{}};return t[e].call(n.exports,n,n.exports,r),n.l=!0,n.exports}r.m=t,r.c=o,r.d=function(e,n,t){r.o(e,n)||Object.defineProperty(e,n,{enumerable:!0,get:t})},r.r=function(e){\"undefined\"!=typeof Symbol&&Symbol.toStringTag&&Object.defineProperty(e,Symbol.toStringTag,{value:\"Module\"}),Object.defineProperty(e,\"__esModule\",{value:!0})},r.t=function(n,e){if(1&e&&(n=r(n)),8&e)return n;if(4&e&&\"object\"==typeof n&&n&&n.__esModule)return n;var t=Object.create(null);if(r.r(t),Object.defineProperty(t,\"default\",{enumerable:!0,value:n}),2&e&&\"string\"!=typeof n)for(var o in n)r.d(t,o,function(e){return n[e]}.bind(null,o));return t},r.n=function(e){var n=e&&e.__esModule?function(){return e.default}:function(){return e};return r.d(n,\"a\",n),n},r.o=function(e,n){return Object.prototype.hasOwnProperty.call(e,n)},r.p=\"\",r(r.s=17)}({0:function(e,n,t){\"use strict\";t.r(n),t.d(n,\"hasAcceptedCookiePolicy\",function(){return r}),t.d(n,\"createNecessaryCookie\",function(){return c}),t.d(n,\"createCookie\",function(){return i}),t.d(n,\"readCookie\",function(){return o}),t.d(n,\"eraseCookie\",function(){return u});var r=function(){var e=o(\"accept-cookie\"),n=o(\"client-origin\");return\"yes\"===e||\"na\"===e||\"non-eu\"===n&&(c(\"accept-cookie\",\"na\",\".payscale.com\",365),!0)},c=function(e,n,t,o){var r=\"\";if(o){var c=new Date;c.setTime(c.getTime()+24*o*60*60*1e3),r=\"; expires=\".concat(c.toGMTString())}var i=t?\"\".concat(e,\"=\").concat(n).concat(r,\"; domain=\").concat(t,\"; path=/\"):\"\".concat(e,\"=\").concat(n).concat(r,\"; path=/\");document.cookie=i},i=function(e,n,t,o){(r()||-1===o)&&c(e,n,t,o)},o=function(e){for(var n=\"\".concat(e,\"=\"),t=document.cookie.split(\";\"),o=0;o<t.length;o++){for(var r=t[o];\" \"===r.charAt(0);)r=r.substring(1,r.length);if(0===r.indexOf(n))return r.substring(n.length,r.length)}return null},u=function(e,n){if(n)i(e,\"\",n,-1);else for(var t=document.domain.split(\".\"),o=0;o<t.length;o++){var r=\"\".concat(t.slice(o,t.length).join(\".\"));i(e,\"\",r,-1),i(e,\"\",\".\".concat(r),-1)}}},17:function(e,n,t){\"use strict\";t.r(n);var o=t(0);\"undefined\"!=typeof window&&Object.keys(o).forEach(function(e){window[e]=o[e]})}});</script><script>(function(w,d,s,l,i){w[l]=w[l]||[];w[l].push({'gtm.start':\n  new Date().getTime(),event:'gtm.js'});var f=d.getElementsByTagName(s)[0],\n  j=d.createElement(s),dl=l!='dataLayer'?'&l='+l:'';j.async=true;j.src=\n  'https://www.googletagmanager.com/gtm.js?id='+i+dl;f.parentNode.insertBefore(j,f);\n  })(window,document,'script','dataLayer','GTM-P3CVWR');</script><script crossorigin=\"anonymous\">\n          (function(e){var t=new XMLHttpRequest;t.open(\"GET\",e,!0),t.onreadystatechange=(function(){if(4==t.readyState&&200==t.status){let e=t.responseText;e=e.replace(/}/g,\"font-display: swap; }\");const a=document.getElementsByTagName(\"head\")[0],n=document.createElement(\"style\");n.appendChild(document.createTextNode(e)),a.appendChild(n)}}),t.send()})(\"https://fonts.googleapis.com/css?family=Roboto:300,400,700,900\");\n          </script><script crossorigin=\"anonymous\" type=\"application/ld+json\">{\"@context\":\"http://schema.googleapis.com/\",\"@id\":\"/research/IN/Job=Accountant/Salary\",\"occupationalCategory\":\"13-2010.00 - Accountants and Auditors\",\"skills\":\"General Ledger Accounting, Accounts Payable, Microsoft Excel, Bookkeeping, Financial Reporting, Accounts Receivable, Account Management, Tax Preparation, Sales Tax Reports, Microsoft Office, Accounting, Enterprise Resource Planning (ERP), Financial Analysis, Tax Compliance, SAP Financial Accounting and Controlling (SAP FICO), Auditing, Reconciliation, Payroll Administration, Data Analysis, Microsoft Word, Banking, Financial Applications, Tax Consulting, Data Entry, Cash Handling, Account Sales, Billing, Budget Management, Database Management & Reporting, Internal Audit, Typing\",\"educationRequirements\":null,\"experienceRequirements\":null,\"qualifications\":null,\"responsibilities\":null,\"mainEntityOfPage\":{\"lastReviewed\":\"2021-02-23T23:33:13.90966Z\",\"description\":null,\"name\":null,\"url\":null,\"@type\":\"WebPage\"},\"occupationLocation\":[{\"description\":null,\"name\":\"India\",\"url\":null,\"@type\":\"Country\"}],\"estimatedSalary\":[{\"currency\":\"INR\",\"unitText\":\"YEAR\",\"percentile10\":\"128947.05\",\"percentile25\":\"188425.71\",\"median\":\"265158.91\",\"percentile75\":\"384549.98\",\"percentile90\":\"584411.15\",\"description\":null,\"name\":\"total\",\"url\":null,\"@type\":\"MonetaryAmountDistribution\"},{\"currency\":\"INR\",\"unitText\":\"YEAR\",\"percentile10\":\"123469.27\",\"percentile25\":\"182393.41\",\"median\":\"249659.22\",\"percentile75\":\"354995.71\",\"percentile90\":\"506338.08\",\"description\":null,\"name\":\"base\",\"url\":null,\"@type\":\"MonetaryAmountDistribution\"},{\"currency\":\"INR\",\"unitText\":\"YEAR\",\"percentile10\":\"4917.77\",\"percentile25\":\"9951.54\",\"median\":\"18657.9\",\"percentile75\":\"31037.74\",\"percentile90\":\"59567.62\",\"description\":null,\"name\":\"bonus\",\"url\":null,\"@type\":\"MonetaryAmountDistribution\"}],\"sampleSize\":6805,\"yearsExperienceMin\":null,\"yearsExperienceMax\":null,\"hiringOrganization\":null,\"description\":\"Accountants perform financial calculations for companies in a wide variety of fields. Some common duties include creating sales and cash flow reports, administering payroll, keeping balance sheets, carrying out billing activities, managing budgets and keeping inventory. The accountant may also be responsible for filing taxes for the company, as well as reviewing past reports to generate income forecasts.\\r\\nOccasionally, internal audits must be carried out to make sure that the various areas of the company are performing as expected; the accountant must also make sure that staff members are adhering to company policies and relevant laws. The accountant should be able to create accurate, detailed reports to illustrate data; sometimes, these reports have to be presented to management. The accountant may oversee the financial transactions of one department or multiple departments within their organization. \\r\\nA bachelor's degree in accounting is required for this position, as is status as a certified public accountant (CPA). Previous accounting experience is generally required or preferred as well. Knowledge of accounting software such as Quickbooks and Microsoft Excel is needed. Additionally, many of the accountant's tasks are performed independently so it is essential to be self-motivated; however, collaboration is necessary, and the accountant must be able to work as part of a team. \\r\\n\",\"name\":\"Accountant\",\"url\":\"/research/IN/Job=Accountant/Salary\",\"@type\":\"OccupationAggregation\"}</script><meta name=\"next-head-count\" content=\"11\"><link rel=\"preload\" href=\"https://cdn-payscale.com/content/research-center-web/prod/2021.0208.2046.252-master/_next/static/css/styles.b227bf72.chunk.css\" as=\"style\"><link rel=\"stylesheet\" href=\"https://cdn-payscale.com/content/research-center-web/prod/2021.0208.2046.252-master/_next/static/css/styles.b227bf72.chunk.css\" data-n-p=\"\"><link rel=\"preload\" href=\"https://cdn-payscale.com/content/research-center-web/prod/2021.0208.2046.252-master/_next/static/css/pages/job/pay.53f55404.chunk.css\" as=\"style\"><link rel=\"stylesheet\" href=\"https://cdn-payscale.com/content/research-center-web/prod/2021.0208.2046.252-master/_next/static/css/pages/job/pay.53f55404.chunk.css\" data-n-p=\"\"><noscript data-n-css=\"\"></noscript><link rel=\"preload\" href=\"https://cdn-payscale.com/content/research-center-web/prod/2021.0208.2046.252-master/_next/static/chunks/main-6cec33473fbdb69509e8.js\" as=\"script\"><link rel=\"preload\" href=\"https://cdn-payscale.com/content/research-center-web/prod/2021.0208.2046.252-master/_next/static/chunks/webpack-c060148b42bba71f8a7a.js\" as=\"script\"><link rel=\"preload\" href=\"https://cdn-payscale.com/content/research-center-web/prod/2021.0208.2046.252-master/_next/static/chunks/commons.0c513591aac5aea1df4c.js\" as=\"script\"><link rel=\"preload\" href=\"https://cdn-payscale.com/content/research-center-web/prod/2021.0208.2046.252-master/_next/static/chunks/f6078781a05fe1bcb0902d23dbbb2662c8d200b3.233d4f01bf6ec4c37fc3.js\" as=\"script\"><link rel=\"preload\" href=\"https://cdn-payscale.com/content/research-center-web/prod/2021.0208.2046.252-master/_next/static/chunks/pages/_app-89f1c4721ae4f2fe3fde.js\" as=\"script\"><link rel=\"preload\" href=\"https://cdn-payscale.com/content/research-center-web/prod/2021.0208.2046.252-master/_next/static/chunks/framework.75101f9487a7a475ce88.js\" as=\"script\"><link rel=\"preload\" href=\"https://cdn-payscale.com/content/research-center-web/prod/2021.0208.2046.252-master/_next/static/chunks/cc69ab74a9ba72eea94736558729e81c2bd655f3.26c13cf5aedb6ea95a1c.js\" as=\"script\"><link rel=\"preload\" href=\"https://cdn-payscale.com/content/research-center-web/prod/2021.0208.2046.252-master/_next/static/chunks/4d4feda2456de235db09b29c583479af129999e0.1fb7365a3f7fbeba8abc.js\" as=\"script\"><link rel=\"preload\" href=\"https://cdn-payscale.com/content/research-center-web/prod/2021.0208.2046.252-master/_next/static/chunks/fa26498fafef60f501c64b06cec839d007da4e70.343e0494ce0488088446.js\" as=\"script\"><link rel=\"preload\" href=\"https://cdn-payscale.com/content/research-center-web/prod/2021.0208.2046.252-master/_next/static/chunks/4d1d62af250902ce0f83bc284e189364548c5c21.39a093079767798521c5.js\" as=\"script\"><link rel=\"preload\" href=\"https://cdn-payscale.com/content/research-center-web/prod/2021.0208.2046.252-master/_next/static/chunks/4699d7e6a61b952e51311f390f955e9a91c7a24d.ea92ec11cbdb296acd78.js\" as=\"script\"><link rel=\"preload\" href=\"https://cdn-payscale.com/content/research-center-web/prod/2021.0208.2046.252-master/_next/static/chunks/1d702dc2a46913e334a0d518d1f316ce60acc9ce.3808c6f06c8377373706.js\" as=\"script\"><link rel=\"preload\" href=\"https://cdn-payscale.com/content/research-center-web/prod/2021.0208.2046.252-master/_next/static/chunks/f651bd44fd1cbb875495552cc10ebe204e4809e8.36ed96af6144c8de99e8.js\" as=\"script\"><link rel=\"preload\" href=\"https://cdn-payscale.com/content/research-center-web/prod/2021.0208.2046.252-master/_next/static/chunks/f457c96651799a12cf52d5e467d410eb70c755ff.db47a5eb374ee61b5d4a.js\" as=\"script\"><link rel=\"preload\" href=\"https://cdn-payscale.com/content/research-center-web/prod/2021.0208.2046.252-master/_next/static/chunks/a129799a303bc049890e96af24a6d6ea5a880873.3bc74ab583f25c8441ec.js\" as=\"script\"><link rel=\"preload\" href=\"https://cdn-payscale.com/content/research-center-web/prod/2021.0208.2046.252-master/_next/static/chunks/styles.c2666581feb32a659f31.js\" as=\"script\"><link rel=\"preload\" href=\"https://cdn-payscale.com/content/research-center-web/prod/2021.0208.2046.252-master/_next/static/chunks/pages/job/pay-04b93fe8d5a6d6a2d6a9.js\" as=\"script\"><style>/* cyrillic-ext */\n@font-face {\n  font-family: 'Roboto';\n  font-style: normal;\n  font-weight: 300;\n  src: url(https://fonts.gstatic.com/s/roboto/v20/KFOlCnqEu92Fr1MmSU5fCRc4EsA.woff2) format('woff2');\n  unicode-range: U+0460-052F, U+1C80-1C88, U+20B4, U+2DE0-2DFF, U+A640-A69F, U+FE2E-FE2F;\nfont-display: swap; }\n/* cyrillic */\n@font-face {\n  font-family: 'Roboto';\n  font-style: normal;\n  font-weight: 300;\n  src: url(https://fonts.gstatic.com/s/roboto/v20/KFOlCnqEu92Fr1MmSU5fABc4EsA.woff2) format('woff2');\n  unicode-range: U+0400-045F, U+0490-0491, U+04B0-04B1, U+2116;\nfont-display: swap; }\n/* greek-ext */\n@font-face {\n  font-family: 'Roboto';\n  font-style: normal;\n  font-weight: 300;\n  src: url(https://fonts.gstatic.com/s/roboto/v20/KFOlCnqEu92Fr1MmSU5fCBc4EsA.woff2) format('woff2');\n  unicode-range: U+1F00-1FFF;\nfont-display: swap; }\n/* greek */\n@font-face {\n  font-family: 'Roboto';\n  font-style: normal;\n  font-weight: 300;\n  src: url(https://fonts.gstatic.com/s/roboto/v20/KFOlCnqEu92Fr1MmSU5fBxc4EsA.woff2) format('woff2');\n  unicode-range: U+0370-03FF;\nfont-display: swap; }\n/* vietnamese */\n@font-face {\n  font-family: 'Roboto';\n  font-style: normal;\n  font-weight: 300;\n  src: url(https://fonts.gstatic.com/s/roboto/v20/KFOlCnqEu92Fr1MmSU5fCxc4EsA.woff2) format('woff2');\n  unicode-range: U+0102-0103, U+0110-0111, U+0128-0129, U+0168-0169, U+01A0-01A1, U+01AF-01B0, U+1EA0-1EF9, U+20AB;\nfont-display: swap; }\n/* latin-ext */\n@font-face {\n  font-family: 'Roboto';\n  font-style: normal;\n  font-weight: 300;\n  src: url(https://fonts.gstatic.com/s/roboto/v20/KFOlCnqEu92Fr1MmSU5fChc4EsA.woff2) format('woff2');\n  unicode-range: U+0100-024F, U+0259, U+1E00-1EFF, U+2020, U+20A0-20AB, U+20AD-20CF, U+2113, U+2C60-2C7F, U+A720-A7FF;\nfont-display: swap; }\n/* latin */\n@font-face {\n  font-family: 'Roboto';\n  font-style: normal;\n  font-weight: 300;\n  src: url(https://fonts.gstatic.com/s/roboto/v20/KFOlCnqEu92Fr1MmSU5fBBc4.woff2) format('woff2');\n  unicode-range: U+0000-00FF, U+0131, U+0152-0153, U+02BB-02BC, U+02C6, U+02DA, U+02DC, U+2000-206F, U+2074, U+20AC, U+2122, U+2191, U+2193, U+2212, U+2215, U+FEFF, U+FFFD;\nfont-display: swap; }\n/* cyrillic-ext */\n@font-face {\n  font-family: 'Roboto';\n  font-style: normal;\n  font-weight: 400;\n  src: url(https://fonts.gstatic.com/s/roboto/v20/KFOmCnqEu92Fr1Mu72xKOzY.woff2) format('woff2');\n  unicode-range: U+0460-052F, U+1C80-1C88, U+20B4, U+2DE0-2DFF, U+A640-A69F, U+FE2E-FE2F;\nfont-display: swap; }\n/* cyrillic */\n@font-face {\n  font-family: 'Roboto';\n  font-style: normal;\n  font-weight: 400;\n  src: url(https://fonts.gstatic.com/s/roboto/v20/KFOmCnqEu92Fr1Mu5mxKOzY.woff2) format('woff2');\n  unicode-range: U+0400-045F, U+0490-0491, U+04B0-04B1, U+2116;\nfont-display: swap; }\n/* greek-ext */\n@font-face {\n  font-family: 'Roboto';\n  font-style: normal;\n  font-weight: 400;\n  src: url(https://fonts.gstatic.com/s/roboto/v20/KFOmCnqEu92Fr1Mu7mxKOzY.woff2) format('woff2');\n  unicode-range: U+1F00-1FFF;\nfont-display: swap; }\n/* greek */\n@font-face {\n  font-family: 'Roboto';\n  font-style: normal;\n  font-weight: 400;\n  src: url(https://fonts.gstatic.com/s/roboto/v20/KFOmCnqEu92Fr1Mu4WxKOzY.woff2) format('woff2');\n  unicode-range: U+0370-03FF;\nfont-display: swap; }\n/* vietnamese */\n@font-face {\n  font-family: 'Roboto';\n  font-style: normal;\n  font-weight: 400;\n  src: url(https://fonts.gstatic.com/s/roboto/v20/KFOmCnqEu92Fr1Mu7WxKOzY.woff2) format('woff2');\n  unicode-range: U+0102-0103, U+0110-0111, U+0128-0129, U+0168-0169, U+01A0-01A1, U+01AF-01B0, U+1EA0-1EF9, U+20AB;\nfont-display: swap; }\n/* latin-ext */\n@font-face {\n  font-family: 'Roboto';\n  font-style: normal;\n  font-weight: 400;\n  src: url(https://fonts.gstatic.com/s/roboto/v20/KFOmCnqEu92Fr1Mu7GxKOzY.woff2) format('woff2');\n  unicode-range: U+0100-024F, U+0259, U+1E00-1EFF, U+2020, U+20A0-20AB, U+20AD-20CF, U+2113, U+2C60-2C7F, U+A720-A7FF;\nfont-display: swap; }\n/* latin */\n@font-face {\n  font-family: 'Roboto';\n  font-style: normal;\n  font-weight: 400;\n  src: url(https://fonts.gstatic.com/s/roboto/v20/KFOmCnqEu92Fr1Mu4mxK.woff2) format('woff2');\n  unicode-range: U+0000-00FF, U+0131, U+0152-0153, U+02BB-02BC, U+02C6, U+02DA, U+02DC, U+2000-206F, U+2074, U+20AC, U+2122, U+2191, U+2193, U+2212, U+2215, U+FEFF, U+FFFD;\nfont-display: swap; }\n/* cyrillic-ext */\n@font-face {\n  font-family: 'Roboto';\n  font-style: normal;\n  font-weight: 700;\n  src: url(https://fonts.gstatic.com/s/roboto/v20/KFOlCnqEu92Fr1MmWUlfCRc4EsA.woff2) format('woff2');\n  unicode-range: U+0460-052F, U+1C80-1C88, U+20B4, U+2DE0-2DFF, U+A640-A69F, U+FE2E-FE2F;\nfont-display: swap; }\n/* cyrillic */\n@font-face {\n  font-family: 'Roboto';\n  font-style: normal;\n  font-weight: 700;\n  src: url(https://fonts.gstatic.com/s/roboto/v20/KFOlCnqEu92Fr1MmWUlfABc4EsA.woff2) format('woff2');\n  unicode-range: U+0400-045F, U+0490-0491, U+04B0-04B1, U+2116;\nfont-display: swap; }\n/* greek-ext */\n@font-face {\n  font-family: 'Roboto';\n  font-style: normal;\n  font-weight: 700;\n  src: url(https://fonts.gstatic.com/s/roboto/v20/KFOlCnqEu92Fr1MmWUlfCBc4EsA.woff2) format('woff2');\n  unicode-range: U+1F00-1FFF;\nfont-display: swap; }\n/* greek */\n@font-face {\n  font-family: 'Roboto';\n  font-style: normal;\n  font-weight: 700;\n  src: url(https://fonts.gstatic.com/s/roboto/v20/KFOlCnqEu92Fr1MmWUlfBxc4EsA.woff2) format('woff2');\n  unicode-range: U+0370-03FF;\nfont-display: swap; }\n/* vietnamese */\n@font-face {\n  font-family: 'Roboto';\n  font-style: normal;\n  font-weight: 700;\n  src: url(https://fonts.gstatic.com/s/roboto/v20/KFOlCnqEu92Fr1MmWUlfCxc4EsA.woff2) format('woff2');\n  unicode-range: U+0102-0103, U+0110-0111, U+0128-0129, U+0168-0169, U+01A0-01A1, U+01AF-01B0, U+1EA0-1EF9, U+20AB;\nfont-display: swap; }\n/* latin-ext */\n@font-face {\n  font-family: 'Roboto';\n  font-style: normal;\n  font-weight: 700;\n  src: url(https://fonts.gstatic.com/s/roboto/v20/KFOlCnqEu92Fr1MmWUlfChc4EsA.woff2) format('woff2');\n  unicode-range: U+0100-024F, U+0259, U+1E00-1EFF, U+2020, U+20A0-20AB, U+20AD-20CF, U+2113, U+2C60-2C7F, U+A720-A7FF;\nfont-display: swap; }\n/* latin */\n@font-face {\n  font-family: 'Roboto';\n  font-style: normal;\n  font-weight: 700;\n  src: url(https://fonts.gstatic.com/s/roboto/v20/KFOlCnqEu92Fr1MmWUlfBBc4.woff2) format('woff2');\n  unicode-range: U+0000-00FF, U+0131, U+0152-0153, U+02BB-02BC, U+02C6, U+02DA, U+02DC, U+2000-206F, U+2074, U+20AC, U+2122, U+2191, U+2193, U+2212, U+2215, U+FEFF, U+FFFD;\nfont-display: swap; }\n/* cyrillic-ext */\n@font-face {\n  font-family: 'Roboto';\n  font-style: normal;\n  font-weight: 900;\n  src: url(https://fonts.gstatic.com/s/roboto/v20/KFOlCnqEu92Fr1MmYUtfCRc4EsA.woff2) format('woff2');\n  unicode-range: U+0460-052F, U+1C80-1C88, U+20B4, U+2DE0-2DFF, U+A640-A69F, U+FE2E-FE2F;\nfont-display: swap; }\n/* cyrillic */\n@font-face {\n  font-family: 'Roboto';\n  font-style: normal;\n  font-weight: 900;\n  src: url(https://fonts.gstatic.com/s/roboto/v20/KFOlCnqEu92Fr1MmYUtfABc4EsA.woff2) format('woff2');\n  unicode-range: U+0400-045F, U+0490-0491, U+04B0-04B1, U+2116;\nfont-display: swap; }\n/* greek-ext */\n@font-face {\n  font-family: 'Roboto';\n  font-style: normal;\n  font-weight: 900;\n  src: url(https://fonts.gstatic.com/s/roboto/v20/KFOlCnqEu92Fr1MmYUtfCBc4EsA.woff2) format('woff2');\n  unicode-range: U+1F00-1FFF;\nfont-display: swap; }\n/* greek */\n@font-face {\n  font-family: 'Roboto';\n  font-style: normal;\n  font-weight: 900;\n  src: url(https://fonts.gstatic.com/s/roboto/v20/KFOlCnqEu92Fr1MmYUtfBxc4EsA.woff2) format('woff2');\n  unicode-range: U+0370-03FF;\nfont-display: swap; }\n/* vietnamese */\n@font-face {\n  font-family: 'Roboto';\n  font-style: normal;\n  font-weight: 900;\n  src: url(https://fonts.gstatic.com/s/roboto/v20/KFOlCnqEu92Fr1MmYUtfCxc4EsA.woff2) format('woff2');\n  unicode-range: U+0102-0103, U+0110-0111, U+0128-0129, U+0168-0169, U+01A0-01A1, U+01AF-01B0, U+1EA0-1EF9, U+20AB;\nfont-display: swap; }\n/* latin-ext */\n@font-face {\n  font-family: 'Roboto';\n  font-style: normal;\n  font-weight: 900;\n  src: url(https://fonts.gstatic.com/s/roboto/v20/KFOlCnqEu92Fr1MmYUtfChc4EsA.woff2) format('woff2');\n  unicode-range: U+0100-024F, U+0259, U+1E00-1EFF, U+2020, U+20A0-20AB, U+20AD-20CF, U+2113, U+2C60-2C7F, U+A720-A7FF;\nfont-display: swap; }\n/* latin */\n@font-face {\n  font-family: 'Roboto';\n  font-style: normal;\n  font-weight: 900;\n  src: url(https://fonts.gstatic.com/s/roboto/v20/KFOlCnqEu92Fr1MmYUtfBBc4.woff2) format('woff2');\n  unicode-range: U+0000-00FF, U+0131, U+0152-0153, U+02BB-02BC, U+02C6, U+02DA, U+02DC, U+2000-206F, U+2074, U+20AC, U+2122, U+2191, U+2193, U+2212, U+2215, U+FEFF, U+FFFD;\nfont-display: swap; }\n</style><meta http-equiv=\"origin-trial\" content=\"A+b/H0b8RPXNaJgaNFpO0YOFuGK6myDQXlwnJB3SwzvNMfcndat4DZYMrP4ClJIzYWo3/yP2S+8FTZ/lpqbPAAEAAABueyJvcmlnaW4iOiJodHRwczovL2ltYXNkay5nb29nbGVhcGlzLmNvbTo0NDMiLCJmZWF0dXJlIjoiVHJ1c3RUb2tlbnMiLCJleHBpcnkiOjE2MjYyMjA3OTksImlzVGhpcmRQYXJ0eSI6dHJ1ZX0=\"><meta http-equiv=\"origin-trial\" content=\"A9ZgbRtm4pU3oZiuNzOsKcC8ppFSZdcjP2qYcdQrFKVzkmiWH1kdYY1Mi9x7G8+PS8HV9Ha9Cz0gaMdKsiVZIgMAAAB7eyJvcmlnaW4iOiJodHRwczovL2RvdWJsZWNsaWNrLm5ldDo0NDMiLCJmZWF0dXJlIjoiVHJ1c3RUb2tlbnMiLCJleHBpcnkiOjE2MjYyMjA3OTksImlzU3ViZG9tYWluIjp0cnVlLCJpc1RoaXJkUGFydHkiOnRydWV9\"><meta http-equiv=\"origin-trial\" content=\"AxL6oBxcpn5rQDPKSAs+d0oxNyJYq2/4esBUh3Yx5z8QfcLu+AU8iFCXYRcr/CEEfDnkxxLTsvXPJFQBxHfvkgMAAACBeyJvcmlnaW4iOiJodHRwczovL2dvb2dsZXRhZ3NlcnZpY2VzLmNvbTo0NDMiLCJmZWF0dXJlIjoiVHJ1c3RUb2tlbnMiLCJleHBpcnkiOjE2MjYyMjA3OTksImlzU3ViZG9tYWluIjp0cnVlLCJpc1RoaXJkUGFydHkiOnRydWV9\"><meta http-equiv=\"origin-trial\" content=\"A9KPtG5kl3oLTk21xqynDPGQ5t18bSOpwt0w6kGa6dEWbuwjpffmdUpR3W+faZDubGT+KIk2do0BX2ca16x8qAcAAACBeyJvcmlnaW4iOiJodHRwczovL2dvb2dsZXN5bmRpY2F0aW9uLmNvbTo0NDMiLCJmZWF0dXJlIjoiVHJ1c3RUb2tlbnMiLCJleHBpcnkiOjE2MjYyMjA3OTksImlzU3ViZG9tYWluIjp0cnVlLCJpc1RoaXJkUGFydHkiOnRydWV9\"><meta http-equiv=\"origin-trial\" content=\"A3HucHUo1oW9s+9kIKz8mLkbcmdaj5lxt3eiIMp1Nh49dkkBlg1Fhg4Fd/r0vL69mRRA36YutI9P/lJUfL8csQoAAACFeyJvcmlnaW4iOiJodHRwczovL2RvdWJsZWNsaWNrLm5ldDo0NDMiLCJmZWF0dXJlIjoiQ29udmVyc2lvbk1lYXN1cmVtZW50IiwiZXhwaXJ5IjoxNjI2MjIwNzk5LCJpc1N1YmRvbWFpbiI6dHJ1ZSwiaXNUaGlyZFBhcnR5Ijp0cnVlfQ==\"><script src=\"https://securepubads.g.doubleclick.net/gpt/pubads_impl_2021030401.js\" async=\"\"></script><script src=\"https://geolocation.onetrust.com/cookieconsentpub/v1/geo/location\" async=\"\" type=\"text/javascript\"></script><style type=\"text/css\">.__react_component_tooltip {\n  border-radius: 3px;\n  display: inline-block;\n  font-size: 13px;\n  left: -999em;\n  opacity: 0;\n  padding: 8px 21px;\n  position: fixed;\n  pointer-events: none;\n  transition: opacity 0.3s ease-out;\n  top: -999em;\n  visibility: hidden;\n  z-index: 999;\n}\n.__react_component_tooltip.allow_hover, .__react_component_tooltip.allow_click {\n  pointer-events: auto;\n}\n.__react_component_tooltip::before, .__react_component_tooltip::after {\n  content: \"\";\n  width: 0;\n  height: 0;\n  position: absolute;\n}\n.__react_component_tooltip.show {\n  opacity: 0.9;\n  margin-top: 0;\n  margin-left: 0;\n  visibility: visible;\n}\n.__react_component_tooltip.place-top::before {\n  border-left: 10px solid transparent;\n  border-right: 10px solid transparent;\n  bottom: -8px;\n  left: 50%;\n  margin-left: -10px;\n}\n.__react_component_tooltip.place-bottom::before {\n  border-left: 10px solid transparent;\n  border-right: 10px solid transparent;\n  top: -8px;\n  left: 50%;\n  margin-left: -10px;\n}\n.__react_component_tooltip.place-left::before {\n  border-top: 6px solid transparent;\n  border-bottom: 6px solid transparent;\n  right: -8px;\n  top: 50%;\n  margin-top: -5px;\n}\n.__react_component_tooltip.place-right::before {\n  border-top: 6px solid transparent;\n  border-bottom: 6px solid transparent;\n  left: -8px;\n  top: 50%;\n  margin-top: -5px;\n}\n.__react_component_tooltip .multi-line {\n  display: block;\n  padding: 2px 0;\n  text-align: center;\n}</style><link rel=\"preload\" href=\"https://adservice.google.co.in/adsid/integrator.js?domain=www.payscale.com\" as=\"script\"><script type=\"text/javascript\" src=\"https://adservice.google.co.in/adsid/integrator.js?domain=www.payscale.com\"></script><link rel=\"preload\" href=\"https://adservice.google.com/adsid/integrator.js?domain=www.payscale.com\" as=\"script\"><script type=\"text/javascript\" src=\"https://adservice.google.com/adsid/integrator.js?domain=www.payscale.com\"></script><script src=\"https://cdn.cookielaw.org/scripttemplates/6.13.0/otBannerSdk.js\" async=\"\" type=\"text/javascript\"></script><style id=\"onetrust-style\">#onetrust-banner-sdk{-ms-text-size-adjust:100%;-webkit-text-size-adjust:100%}#onetrust-banner-sdk .onetrust-vendors-list-handler{cursor:pointer;color:#1f96db;font-size:inherit;font-weight:bold;text-decoration:none;margin-left:5px}#onetrust-banner-sdk .onetrust-vendors-list-handler:hover{color:#1f96db}#onetrust-banner-sdk .ot-close-icon,#onetrust-pc-sdk .ot-close-icon,#ot-sync-ntfy .ot-close-icon{background-image:url(\"data:image/svg+xml;base64,PHN2ZyB2ZXJzaW9uPSIxLjEiIHhtbG5zPSJodHRwOi8vd3d3LnczLm9yZy8yMDAwL3N2ZyIgeG1sbnM6eGxpbms9Imh0dHA6Ly93d3cudzMub3JnLzE5OTkveGxpbmsiIHg9IjBweCIgeT0iMHB4IiB3aWR0aD0iMzQ4LjMzM3B4IiBoZWlnaHQ9IjM0OC4zMzNweCIgdmlld0JveD0iMCAwIDM0OC4zMzMgMzQ4LjMzNCIgc3R5bGU9ImVuYWJsZS1iYWNrZ3JvdW5kOm5ldyAwIDAgMzQ4LjMzMyAzNDguMzM0OyIgeG1sOnNwYWNlPSJwcmVzZXJ2ZSI+PGc+PHBhdGggZmlsbD0iIzU2NTY1NiIgZD0iTTMzNi41NTksNjguNjExTDIzMS4wMTYsMTc0LjE2NWwxMDUuNTQzLDEwNS41NDljMTUuNjk5LDE1LjcwNSwxNS42OTksNDEuMTQ1LDAsNTYuODVjLTcuODQ0LDcuODQ0LTE4LjEyOCwxMS43NjktMjguNDA3LDExLjc2OWMtMTAuMjk2LDAtMjAuNTgxLTMuOTE5LTI4LjQxOS0xMS43NjlMMTc0LjE2NywyMzEuMDAzTDY4LjYwOSwzMzYuNTYzYy03Ljg0Myw3Ljg0NC0xOC4xMjgsMTEuNzY5LTI4LjQxNiwxMS43NjljLTEwLjI4NSwwLTIwLjU2My0zLjkxOS0yOC40MTMtMTEuNzY5Yy0xNS42OTktMTUuNjk4LTE1LjY5OS00MS4xMzksMC01Ni44NWwxMDUuNTQtMTA1LjU0OUwxMS43NzQsNjguNjExYy0xNS42OTktMTUuNjk5LTE1LjY5OS00MS4xNDUsMC01Ni44NDRjMTUuNjk2LTE1LjY4Nyw0MS4xMjctMTUuNjg3LDU2LjgyOSwwbDEwNS41NjMsMTA1LjU1NEwyNzkuNzIxLDExLjc2N2MxNS43MDUtMTUuNjg3LDQxLjEzOS0xNS42ODcsNTYuODMyLDBDMzUyLjI1OCwyNy40NjYsMzUyLjI1OCw1Mi45MTIsMzM2LjU1OSw2OC42MTF6Ii8+PC9nPjwvc3ZnPg==\");background-size:contain;background-repeat:no-repeat;background-position:center;height:12px;width:12px}#onetrust-banner-sdk .powered-by-logo,#onetrust-banner-sdk .ot-pc-footer-logo a,#onetrust-pc-sdk .powered-by-logo,#onetrust-pc-sdk .ot-pc-footer-logo a,#ot-sync-ntfy .powered-by-logo,#ot-sync-ntfy .ot-pc-footer-logo a{background-size:contain;background-repeat:no-repeat;background-position:center;height:25px;width:152px;display:block}#onetrust-banner-sdk h3 *,#onetrust-banner-sdk h4 *,#onetrust-banner-sdk h6 *,#onetrust-banner-sdk button *,#onetrust-banner-sdk a[data-parent-id] *,#onetrust-pc-sdk h3 *,#onetrust-pc-sdk h4 *,#onetrust-pc-sdk h6 *,#onetrust-pc-sdk button *,#onetrust-pc-sdk a[data-parent-id] *,#ot-sync-ntfy h3 *,#ot-sync-ntfy h4 *,#ot-sync-ntfy h6 *,#ot-sync-ntfy button *,#ot-sync-ntfy a[data-parent-id] *{font-size:inherit;font-weight:inherit;color:inherit}#onetrust-banner-sdk .ot-hide,#onetrust-pc-sdk .ot-hide,#ot-sync-ntfy .ot-hide{display:none !important}#onetrust-pc-sdk .ot-sdk-row .ot-sdk-column{padding:0}#onetrust-pc-sdk .ot-sdk-container{padding-right:0}#onetrust-pc-sdk .ot-sdk-row{flex-direction:initial;width:100%}#onetrust-pc-sdk [type=\"checkbox\"]:checked,#onetrust-pc-sdk [type=\"checkbox\"]:not(:checked){pointer-events:initial}#onetrust-pc-sdk [type=\"checkbox\"]:disabled+label::before,#onetrust-pc-sdk [type=\"checkbox\"]:disabled+label:after,#onetrust-pc-sdk [type=\"checkbox\"]:disabled+label{pointer-events:none;opacity:0.7}#onetrust-pc-sdk #vendor-list-content{transform:translate3d(0, 0, 0)}#onetrust-pc-sdk li input[type=\"checkbox\"]{z-index:1}#onetrust-pc-sdk li .ot-checkbox label{z-index:2}#onetrust-pc-sdk li .ot-checkbox input[type=\"checkbox\"]{height:auto;width:auto}#onetrust-pc-sdk li .host-title a,#onetrust-pc-sdk li .ot-host-name a,#onetrust-pc-sdk li .accordion-text,#onetrust-pc-sdk li .ot-acc-txt{z-index:2;position:relative}#onetrust-pc-sdk input{margin:3px 0.1ex}#onetrust-pc-sdk .toggle-always-active{opacity:0.6;cursor:default}#onetrust-pc-sdk .pc-logo,#onetrust-pc-sdk .ot-pc-logo{height:60px;width:180px;background-position:center;background-size:contain;background-repeat:no-repeat}#onetrust-pc-sdk .ot-tooltip .ot-tooltiptext{visibility:hidden;width:120px;background-color:#555;color:#fff;text-align:center;padding:5px 0;border-radius:6px;position:absolute;z-index:1;bottom:125%;left:50%;margin-left:-60px;opacity:0;transition:opacity 0.3s}#onetrust-pc-sdk .ot-tooltip .ot-tooltiptext::after{content:\"\";position:absolute;top:100%;left:50%;margin-left:-5px;border-width:5px;border-style:solid;border-color:#555 transparent transparent transparent}#onetrust-pc-sdk .ot-tooltip:hover .ot-tooltiptext{visibility:visible;opacity:1}#onetrust-pc-sdk .ot-tooltip{position:relative;display:inline-block;z-index:3}#onetrust-pc-sdk .ot-tooltip svg{color:grey;height:20px;width:20px}#onetrust-pc-sdk .screen-reader-only,#onetrust-pc-sdk .ot-scrn-rdr,.ot-sdk-cookie-policy .screen-reader-only,.ot-sdk-cookie-policy .ot-scrn-rdr{border:0;clip:rect(0 0 0 0);height:1px;margin:-1px;overflow:hidden;padding:0;position:absolute;width:1px}#onetrust-pc-sdk.ot-fade-in,.onetrust-pc-dark-filter.ot-fade-in{animation-name:onetrust-fade-in;animation-duration:400ms;animation-timing-function:ease-in-out}#onetrust-pc-sdk.ot-hide{display:none !important}.onetrust-pc-dark-filter.ot-hide{display:none !important}#ot-sdk-btn.ot-sdk-show-settings,#ot-sdk-btn.optanon-show-settings{color:#68b631;border:1px solid #68b631;height:auto;white-space:normal;word-wrap:break-word;padding:0.8em 2em;font-size:0.8em;line-height:1.2;cursor:pointer;-moz-transition:0.1s ease;-o-transition:0.1s ease;-webkit-transition:1s ease;transition:0.1s ease}#ot-sdk-btn.ot-sdk-show-settings:hover,#ot-sdk-btn.optanon-show-settings:hover{color:#fff;background-color:#68b631}.onetrust-pc-dark-filter{background:rgba(0,0,0,0.5);z-index:2147483646;width:100%;height:100%;overflow:hidden;position:fixed;top:0;bottom:0;left:0}@keyframes onetrust-fade-in{0%{opacity:0}100%{opacity:1}}@media only screen and (min-width: 426px) and (max-width: 896px) and (orientation: landscape){#onetrust-pc-sdk p{font-size:0.75em}}#onetrust-banner-sdk .banner-option-input:focus+label{outline:1px solid #000;outline-style:auto}\n#onetrust-banner-sdk,#onetrust-pc-sdk,#ot-sdk-cookie-policy,#ot-sync-ntfy{font-size:16px}#onetrust-banner-sdk *,#onetrust-banner-sdk ::after,#onetrust-banner-sdk ::before,#onetrust-pc-sdk *,#onetrust-pc-sdk ::after,#onetrust-pc-sdk ::before,#ot-sdk-cookie-policy *,#ot-sdk-cookie-policy ::after,#ot-sdk-cookie-policy ::before,#ot-sync-ntfy *,#ot-sync-ntfy ::after,#ot-sync-ntfy ::before{-webkit-box-sizing:content-box;-moz-box-sizing:content-box;box-sizing:content-box}#onetrust-banner-sdk div,#onetrust-banner-sdk span,#onetrust-banner-sdk h1,#onetrust-banner-sdk h2,#onetrust-banner-sdk h3,#onetrust-banner-sdk h4,#onetrust-banner-sdk h5,#onetrust-banner-sdk h6,#onetrust-banner-sdk p,#onetrust-banner-sdk img,#onetrust-banner-sdk svg,#onetrust-banner-sdk button,#onetrust-banner-sdk section,#onetrust-banner-sdk a,#onetrust-banner-sdk label,#onetrust-banner-sdk input,#onetrust-banner-sdk ul,#onetrust-banner-sdk li,#onetrust-banner-sdk nav,#onetrust-banner-sdk table,#onetrust-banner-sdk thead,#onetrust-banner-sdk tr,#onetrust-banner-sdk td,#onetrust-banner-sdk tbody,#onetrust-banner-sdk .ot-main-content,#onetrust-banner-sdk .ot-toggle,#onetrust-banner-sdk #ot-content,#onetrust-banner-sdk #ot-pc-content,#onetrust-banner-sdk .checkbox,#onetrust-pc-sdk div,#onetrust-pc-sdk span,#onetrust-pc-sdk h1,#onetrust-pc-sdk h2,#onetrust-pc-sdk h3,#onetrust-pc-sdk h4,#onetrust-pc-sdk h5,#onetrust-pc-sdk h6,#onetrust-pc-sdk p,#onetrust-pc-sdk img,#onetrust-pc-sdk svg,#onetrust-pc-sdk button,#onetrust-pc-sdk section,#onetrust-pc-sdk a,#onetrust-pc-sdk label,#onetrust-pc-sdk input,#onetrust-pc-sdk ul,#onetrust-pc-sdk li,#onetrust-pc-sdk nav,#onetrust-pc-sdk table,#onetrust-pc-sdk thead,#onetrust-pc-sdk tr,#onetrust-pc-sdk td,#onetrust-pc-sdk tbody,#onetrust-pc-sdk .ot-main-content,#onetrust-pc-sdk .ot-toggle,#onetrust-pc-sdk #ot-content,#onetrust-pc-sdk #ot-pc-content,#onetrust-pc-sdk .checkbox,#ot-sdk-cookie-policy div,#ot-sdk-cookie-policy span,#ot-sdk-cookie-policy h1,#ot-sdk-cookie-policy h2,#ot-sdk-cookie-policy h3,#ot-sdk-cookie-policy h4,#ot-sdk-cookie-policy h5,#ot-sdk-cookie-policy h6,#ot-sdk-cookie-policy p,#ot-sdk-cookie-policy img,#ot-sdk-cookie-policy svg,#ot-sdk-cookie-policy button,#ot-sdk-cookie-policy section,#ot-sdk-cookie-policy a,#ot-sdk-cookie-policy label,#ot-sdk-cookie-policy input,#ot-sdk-cookie-policy ul,#ot-sdk-cookie-policy li,#ot-sdk-cookie-policy nav,#ot-sdk-cookie-policy table,#ot-sdk-cookie-policy thead,#ot-sdk-cookie-policy tr,#ot-sdk-cookie-policy td,#ot-sdk-cookie-policy tbody,#ot-sdk-cookie-policy .ot-main-content,#ot-sdk-cookie-policy .ot-toggle,#ot-sdk-cookie-policy #ot-content,#ot-sdk-cookie-policy #ot-pc-content,#ot-sdk-cookie-policy .checkbox,#ot-sync-ntfy div,#ot-sync-ntfy span,#ot-sync-ntfy h1,#ot-sync-ntfy h2,#ot-sync-ntfy h3,#ot-sync-ntfy h4,#ot-sync-ntfy h5,#ot-sync-ntfy h6,#ot-sync-ntfy p,#ot-sync-ntfy img,#ot-sync-ntfy svg,#ot-sync-ntfy button,#ot-sync-ntfy section,#ot-sync-ntfy a,#ot-sync-ntfy label,#ot-sync-ntfy input,#ot-sync-ntfy ul,#ot-sync-ntfy li,#ot-sync-ntfy nav,#ot-sync-ntfy table,#ot-sync-ntfy thead,#ot-sync-ntfy tr,#ot-sync-ntfy td,#ot-sync-ntfy tbody,#ot-sync-ntfy .ot-main-content,#ot-sync-ntfy .ot-toggle,#ot-sync-ntfy #ot-content,#ot-sync-ntfy #ot-pc-content,#ot-sync-ntfy .checkbox{font-family:inherit;font-weight:normal;-webkit-font-smoothing:auto;letter-spacing:normal;line-height:normal;padding:0;margin:0;height:auto;min-height:0;max-height:none;width:auto;min-width:0;max-width:none;border-radius:0;border:none;clear:none;float:none;position:static;bottom:auto;left:auto;right:auto;top:auto;text-align:left;text-decoration:none;text-indent:0;text-shadow:none;text-transform:none;white-space:normal;background:none;overflow:visible;vertical-align:baseline;visibility:visible;z-index:auto;box-shadow:none}#onetrust-banner-sdk label:before,#onetrust-banner-sdk label:after,#onetrust-banner-sdk .checkbox:after,#onetrust-banner-sdk .checkbox:before,#onetrust-pc-sdk label:before,#onetrust-pc-sdk label:after,#onetrust-pc-sdk .checkbox:after,#onetrust-pc-sdk .checkbox:before,#ot-sdk-cookie-policy label:before,#ot-sdk-cookie-policy label:after,#ot-sdk-cookie-policy .checkbox:after,#ot-sdk-cookie-policy .checkbox:before,#ot-sync-ntfy label:before,#ot-sync-ntfy label:after,#ot-sync-ntfy .checkbox:after,#ot-sync-ntfy .checkbox:before{content:\"\";content:none}\n#onetrust-banner-sdk .ot-sdk-container,#onetrust-pc-sdk .ot-sdk-container,#ot-sdk-cookie-policy .ot-sdk-container{position:relative;width:100%;max-width:100%;margin:0 auto;padding:0 20px;box-sizing:border-box}#onetrust-banner-sdk .ot-sdk-column,#onetrust-banner-sdk .ot-sdk-columns,#onetrust-pc-sdk .ot-sdk-column,#onetrust-pc-sdk .ot-sdk-columns,#ot-sdk-cookie-policy .ot-sdk-column,#ot-sdk-cookie-policy .ot-sdk-columns{width:100%;float:left;box-sizing:border-box;padding:0;display:initial}@media (min-width: 400px){#onetrust-banner-sdk .ot-sdk-container,#onetrust-pc-sdk .ot-sdk-container,#ot-sdk-cookie-policy .ot-sdk-container{width:90%;padding:0}}@media (min-width: 550px){#onetrust-banner-sdk .ot-sdk-container,#onetrust-pc-sdk .ot-sdk-container,#ot-sdk-cookie-policy .ot-sdk-container{width:100%}#onetrust-banner-sdk .ot-sdk-column,#onetrust-banner-sdk .ot-sdk-columns,#onetrust-pc-sdk .ot-sdk-column,#onetrust-pc-sdk .ot-sdk-columns,#ot-sdk-cookie-policy .ot-sdk-column,#ot-sdk-cookie-policy .ot-sdk-columns{margin-left:4%}#onetrust-banner-sdk .ot-sdk-column:first-child,#onetrust-banner-sdk .ot-sdk-columns:first-child,#onetrust-pc-sdk .ot-sdk-column:first-child,#onetrust-pc-sdk .ot-sdk-columns:first-child,#ot-sdk-cookie-policy .ot-sdk-column:first-child,#ot-sdk-cookie-policy .ot-sdk-columns:first-child{margin-left:0}#onetrust-banner-sdk .ot-sdk-one.ot-sdk-column,#onetrust-banner-sdk .ot-sdk-one.ot-sdk-columns,#onetrust-pc-sdk .ot-sdk-one.ot-sdk-column,#onetrust-pc-sdk .ot-sdk-one.ot-sdk-columns,#ot-sdk-cookie-policy .ot-sdk-one.ot-sdk-column,#ot-sdk-cookie-policy .ot-sdk-one.ot-sdk-columns{width:4.66666666667%}#onetrust-banner-sdk .ot-sdk-two.ot-sdk-columns,#onetrust-pc-sdk .ot-sdk-two.ot-sdk-columns,#ot-sdk-cookie-policy .ot-sdk-two.ot-sdk-columns{width:13.3333333333%}#onetrust-banner-sdk .ot-sdk-three.ot-sdk-columns,#onetrust-pc-sdk .ot-sdk-three.ot-sdk-columns,#ot-sdk-cookie-policy .ot-sdk-three.ot-sdk-columns{width:22%}#onetrust-banner-sdk .ot-sdk-four.ot-sdk-columns,#onetrust-pc-sdk .ot-sdk-four.ot-sdk-columns,#ot-sdk-cookie-policy .ot-sdk-four.ot-sdk-columns{width:30.6666666667%}#onetrust-banner-sdk .ot-sdk-five.ot-sdk-columns,#onetrust-pc-sdk .ot-sdk-five.ot-sdk-columns,#ot-sdk-cookie-policy .ot-sdk-five.ot-sdk-columns{width:39.3333333333%}#onetrust-banner-sdk .ot-sdk-six.ot-sdk-columns,#onetrust-pc-sdk .ot-sdk-six.ot-sdk-columns,#ot-sdk-cookie-policy .ot-sdk-six.ot-sdk-columns{width:48%}#onetrust-banner-sdk .ot-sdk-seven.ot-sdk-columns,#onetrust-pc-sdk .ot-sdk-seven.ot-sdk-columns,#ot-sdk-cookie-policy .ot-sdk-seven.ot-sdk-columns{width:56.6666666667%}#onetrust-banner-sdk .ot-sdk-eight.ot-sdk-columns,#onetrust-pc-sdk .ot-sdk-eight.ot-sdk-columns,#ot-sdk-cookie-policy .ot-sdk-eight.ot-sdk-columns{width:65.3333333333%}#onetrust-banner-sdk .ot-sdk-nine.ot-sdk-columns,#onetrust-pc-sdk .ot-sdk-nine.ot-sdk-columns,#ot-sdk-cookie-policy .ot-sdk-nine.ot-sdk-columns{width:74%}#onetrust-banner-sdk .ot-sdk-ten.ot-sdk-columns,#onetrust-pc-sdk .ot-sdk-ten.ot-sdk-columns,#ot-sdk-cookie-policy .ot-sdk-ten.ot-sdk-columns{width:82.6666666667%}#onetrust-banner-sdk .ot-sdk-eleven.ot-sdk-columns,#onetrust-pc-sdk .ot-sdk-eleven.ot-sdk-columns,#ot-sdk-cookie-policy .ot-sdk-eleven.ot-sdk-columns{width:91.3333333333%}#onetrust-banner-sdk .ot-sdk-twelve.ot-sdk-columns,#onetrust-pc-sdk .ot-sdk-twelve.ot-sdk-columns,#ot-sdk-cookie-policy .ot-sdk-twelve.ot-sdk-columns{width:100%;margin-left:0}#onetrust-banner-sdk .ot-sdk-one-third.ot-sdk-column,#onetrust-pc-sdk .ot-sdk-one-third.ot-sdk-column,#ot-sdk-cookie-policy .ot-sdk-one-third.ot-sdk-column{width:30.6666666667%}#onetrust-banner-sdk .ot-sdk-two-thirds.ot-sdk-column,#onetrust-pc-sdk .ot-sdk-two-thirds.ot-sdk-column,#ot-sdk-cookie-policy .ot-sdk-two-thirds.ot-sdk-column{width:65.3333333333%}#onetrust-banner-sdk .ot-sdk-one-half.ot-sdk-column,#onetrust-pc-sdk .ot-sdk-one-half.ot-sdk-column,#ot-sdk-cookie-policy .ot-sdk-one-half.ot-sdk-column{width:48%}#onetrust-banner-sdk .ot-sdk-offset-by-one.ot-sdk-column,#onetrust-banner-sdk .ot-sdk-offset-by-one.ot-sdk-columns,#onetrust-pc-sdk .ot-sdk-offset-by-one.ot-sdk-column,#onetrust-pc-sdk .ot-sdk-offset-by-one.ot-sdk-columns,#ot-sdk-cookie-policy .ot-sdk-offset-by-one.ot-sdk-column,#ot-sdk-cookie-policy .ot-sdk-offset-by-one.ot-sdk-columns{margin-left:8.66666666667%}#onetrust-banner-sdk .ot-sdk-offset-by-two.ot-sdk-column,#onetrust-banner-sdk .ot-sdk-offset-by-two.ot-sdk-columns,#onetrust-pc-sdk .ot-sdk-offset-by-two.ot-sdk-column,#onetrust-pc-sdk .ot-sdk-offset-by-two.ot-sdk-columns,#ot-sdk-cookie-policy .ot-sdk-offset-by-two.ot-sdk-column,#ot-sdk-cookie-policy .ot-sdk-offset-by-two.ot-sdk-columns{margin-left:17.3333333333%}#onetrust-banner-sdk .ot-sdk-offset-by-three.ot-sdk-column,#onetrust-banner-sdk .ot-sdk-offset-by-three.ot-sdk-columns,#onetrust-pc-sdk .ot-sdk-offset-by-three.ot-sdk-column,#onetrust-pc-sdk .ot-sdk-offset-by-three.ot-sdk-columns,#ot-sdk-cookie-policy .ot-sdk-offset-by-three.ot-sdk-column,#ot-sdk-cookie-policy .ot-sdk-offset-by-three.ot-sdk-columns{margin-left:26%}#onetrust-banner-sdk .ot-sdk-offset-by-four.ot-sdk-column,#onetrust-banner-sdk .ot-sdk-offset-by-four.ot-sdk-columns,#onetrust-pc-sdk .ot-sdk-offset-by-four.ot-sdk-column,#onetrust-pc-sdk .ot-sdk-offset-by-four.ot-sdk-columns,#ot-sdk-cookie-policy .ot-sdk-offset-by-four.ot-sdk-column,#ot-sdk-cookie-policy .ot-sdk-offset-by-four.ot-sdk-columns{margin-left:34.6666666667%}#onetrust-banner-sdk .ot-sdk-offset-by-five.ot-sdk-column,#onetrust-banner-sdk .ot-sdk-offset-by-five.ot-sdk-columns,#onetrust-pc-sdk .ot-sdk-offset-by-five.ot-sdk-column,#onetrust-pc-sdk .ot-sdk-offset-by-five.ot-sdk-columns,#ot-sdk-cookie-policy .ot-sdk-offset-by-five.ot-sdk-column,#ot-sdk-cookie-policy .ot-sdk-offset-by-five.ot-sdk-columns{margin-left:43.3333333333%}#onetrust-banner-sdk .ot-sdk-offset-by-six.ot-sdk-column,#onetrust-banner-sdk .ot-sdk-offset-by-six.ot-sdk-columns,#onetrust-pc-sdk .ot-sdk-offset-by-six.ot-sdk-column,#onetrust-pc-sdk .ot-sdk-offset-by-six.ot-sdk-columns,#ot-sdk-cookie-policy .ot-sdk-offset-by-six.ot-sdk-column,#ot-sdk-cookie-policy .ot-sdk-offset-by-six.ot-sdk-columns{margin-left:52%}#onetrust-banner-sdk .ot-sdk-offset-by-seven.ot-sdk-column,#onetrust-banner-sdk .ot-sdk-offset-by-seven.ot-sdk-columns,#onetrust-pc-sdk .ot-sdk-offset-by-seven.ot-sdk-column,#onetrust-pc-sdk .ot-sdk-offset-by-seven.ot-sdk-columns,#ot-sdk-cookie-policy .ot-sdk-offset-by-seven.ot-sdk-column,#ot-sdk-cookie-policy .ot-sdk-offset-by-seven.ot-sdk-columns{margin-left:60.6666666667%}#onetrust-banner-sdk .ot-sdk-offset-by-eight.ot-sdk-column,#onetrust-banner-sdk .ot-sdk-offset-by-eight.ot-sdk-columns,#onetrust-pc-sdk .ot-sdk-offset-by-eight.ot-sdk-column,#onetrust-pc-sdk .ot-sdk-offset-by-eight.ot-sdk-columns,#ot-sdk-cookie-policy .ot-sdk-offset-by-eight.ot-sdk-column,#ot-sdk-cookie-policy .ot-sdk-offset-by-eight.ot-sdk-columns{margin-left:69.3333333333%}#onetrust-banner-sdk .ot-sdk-offset-by-nine.ot-sdk-column,#onetrust-banner-sdk .ot-sdk-offset-by-nine.ot-sdk-columns,#onetrust-pc-sdk .ot-sdk-offset-by-nine.ot-sdk-column,#onetrust-pc-sdk .ot-sdk-offset-by-nine.ot-sdk-columns,#ot-sdk-cookie-policy .ot-sdk-offset-by-nine.ot-sdk-column,#ot-sdk-cookie-policy .ot-sdk-offset-by-nine.ot-sdk-columns{margin-left:78%}#onetrust-banner-sdk .ot-sdk-offset-by-ten.ot-sdk-column,#onetrust-banner-sdk .ot-sdk-offset-by-ten.ot-sdk-columns,#onetrust-pc-sdk .ot-sdk-offset-by-ten.ot-sdk-column,#onetrust-pc-sdk .ot-sdk-offset-by-ten.ot-sdk-columns,#ot-sdk-cookie-policy .ot-sdk-offset-by-ten.ot-sdk-column,#ot-sdk-cookie-policy .ot-sdk-offset-by-ten.ot-sdk-columns{margin-left:86.6666666667%}#onetrust-banner-sdk .ot-sdk-offset-by-eleven.ot-sdk-column,#onetrust-banner-sdk .ot-sdk-offset-by-eleven.ot-sdk-columns,#onetrust-pc-sdk .ot-sdk-offset-by-eleven.ot-sdk-column,#onetrust-pc-sdk .ot-sdk-offset-by-eleven.ot-sdk-columns,#ot-sdk-cookie-policy .ot-sdk-offset-by-eleven.ot-sdk-column,#ot-sdk-cookie-policy .ot-sdk-offset-by-eleven.ot-sdk-columns{margin-left:95.3333333333%}#onetrust-banner-sdk .ot-sdk-offset-by-one-third.ot-sdk-column,#onetrust-banner-sdk .ot-sdk-offset-by-one-third.ot-sdk-columns,#onetrust-pc-sdk .ot-sdk-offset-by-one-third.ot-sdk-column,#onetrust-pc-sdk .ot-sdk-offset-by-one-third.ot-sdk-columns,#ot-sdk-cookie-policy .ot-sdk-offset-by-one-third.ot-sdk-column,#ot-sdk-cookie-policy .ot-sdk-offset-by-one-third.ot-sdk-columns{margin-left:34.6666666667%}#onetrust-banner-sdk .ot-sdk-offset-by-two-thirds.ot-sdk-column,#onetrust-banner-sdk .ot-sdk-offset-by-two-thirds.ot-sdk-columns,#onetrust-pc-sdk .ot-sdk-offset-by-two-thirds.ot-sdk-column,#onetrust-pc-sdk .ot-sdk-offset-by-two-thirds.ot-sdk-columns,#ot-sdk-cookie-policy .ot-sdk-offset-by-two-thirds.ot-sdk-column,#ot-sdk-cookie-policy .ot-sdk-offset-by-two-thirds.ot-sdk-columns{margin-left:69.3333333333%}#onetrust-banner-sdk .ot-sdk-offset-by-one-half.ot-sdk-column,#onetrust-banner-sdk .ot-sdk-offset-by-one-half.ot-sdk-columns,#onetrust-pc-sdk .ot-sdk-offset-by-one-half.ot-sdk-column,#onetrust-pc-sdk .ot-sdk-offset-by-one-half.ot-sdk-columns,#ot-sdk-cookie-policy .ot-sdk-offset-by-one-half.ot-sdk-column,#ot-sdk-cookie-policy .ot-sdk-offset-by-one-half.ot-sdk-columns{margin-left:52%}}#onetrust-banner-sdk h1,#onetrust-banner-sdk h2,#onetrust-banner-sdk h3,#onetrust-banner-sdk h4,#onetrust-banner-sdk h5,#onetrust-banner-sdk h6,#onetrust-pc-sdk h1,#onetrust-pc-sdk h2,#onetrust-pc-sdk h3,#onetrust-pc-sdk h4,#onetrust-pc-sdk h5,#onetrust-pc-sdk h6,#ot-sdk-cookie-policy h1,#ot-sdk-cookie-policy h2,#ot-sdk-cookie-policy h3,#ot-sdk-cookie-policy h4,#ot-sdk-cookie-policy h5,#ot-sdk-cookie-policy h6{margin-top:0;font-weight:600;font-family:inherit}#onetrust-banner-sdk h1,#onetrust-pc-sdk h1,#ot-sdk-cookie-policy h1{font-size:1.5rem;line-height:1.2}#onetrust-banner-sdk h2,#onetrust-pc-sdk h2,#ot-sdk-cookie-policy h2{font-size:1.5rem;line-height:1.25}#onetrust-banner-sdk h3,#onetrust-pc-sdk h3,#ot-sdk-cookie-policy h3{font-size:1.5rem;line-height:1.3}#onetrust-banner-sdk h4,#onetrust-pc-sdk h4,#ot-sdk-cookie-policy h4{font-size:1.5rem;line-height:1.35}#onetrust-banner-sdk h5,#onetrust-pc-sdk h5,#ot-sdk-cookie-policy h5{font-size:1.5rem;line-height:1.5}#onetrust-banner-sdk h6,#onetrust-pc-sdk h6,#ot-sdk-cookie-policy h6{font-size:1.5rem;line-height:1.6}@media (min-width: 550px){#onetrust-banner-sdk h1,#onetrust-pc-sdk h1,#ot-sdk-cookie-policy h1{font-size:1.5rem}#onetrust-banner-sdk h2,#onetrust-pc-sdk h2,#ot-sdk-cookie-policy h2{font-size:1.5rem}#onetrust-banner-sdk h3,#onetrust-pc-sdk h3,#ot-sdk-cookie-policy h3{font-size:1.5rem}#onetrust-banner-sdk h4,#onetrust-pc-sdk h4,#ot-sdk-cookie-policy h4{font-size:1.5rem}#onetrust-banner-sdk h5,#onetrust-pc-sdk h5,#ot-sdk-cookie-policy h5{font-size:1.5rem}#onetrust-banner-sdk h6,#onetrust-pc-sdk h6,#ot-sdk-cookie-policy h6{font-size:1.5rem}}#onetrust-banner-sdk p,#onetrust-pc-sdk p,#ot-sdk-cookie-policy p{margin:0 0 1em 0;font-family:inherit;line-height:normal}#onetrust-banner-sdk a,#onetrust-pc-sdk a,#ot-sdk-cookie-policy a{color:#565656;text-decoration:underline}#onetrust-banner-sdk a:hover,#onetrust-pc-sdk a:hover,#ot-sdk-cookie-policy a:hover{color:#565656;text-decoration:none}#onetrust-banner-sdk .ot-sdk-button,#onetrust-banner-sdk button,#onetrust-pc-sdk .ot-sdk-button,#onetrust-pc-sdk button,#ot-sdk-cookie-policy .ot-sdk-button,#ot-sdk-cookie-policy button{margin-bottom:1rem;font-family:inherit}#onetrust-banner-sdk .ot-sdk-button,#onetrust-banner-sdk button,#onetrust-banner-sdk input[type=\"submit\"],#onetrust-banner-sdk input[type=\"reset\"],#onetrust-banner-sdk input[type=\"button\"],#onetrust-pc-sdk .ot-sdk-button,#onetrust-pc-sdk button,#onetrust-pc-sdk input[type=\"submit\"],#onetrust-pc-sdk input[type=\"reset\"],#onetrust-pc-sdk input[type=\"button\"],#ot-sdk-cookie-policy .ot-sdk-button,#ot-sdk-cookie-policy button,#ot-sdk-cookie-policy input[type=\"submit\"],#ot-sdk-cookie-policy input[type=\"reset\"],#ot-sdk-cookie-policy input[type=\"button\"]{display:inline-block;height:38px;padding:0 30px;color:#555;text-align:center;font-size:0.9em;font-weight:400;line-height:38px;letter-spacing:0.01em;text-decoration:none;white-space:nowrap;background-color:transparent;border-radius:2px;border:1px solid #bbb;cursor:pointer;box-sizing:border-box}#onetrust-banner-sdk .ot-sdk-button:hover,#onetrust-banner-sdk :not(.ot-leg-btn-container)>button:hover,#onetrust-banner-sdk input[type=\"submit\"]:hover,#onetrust-banner-sdk input[type=\"reset\"]:hover,#onetrust-banner-sdk input[type=\"button\"]:hover,#onetrust-banner-sdk .ot-sdk-button:focus,#onetrust-banner-sdk :not(.ot-leg-btn-container)>button:focus,#onetrust-banner-sdk input[type=\"submit\"]:focus,#onetrust-banner-sdk input[type=\"reset\"]:focus,#onetrust-banner-sdk input[type=\"button\"]:focus,#onetrust-pc-sdk .ot-sdk-button:hover,#onetrust-pc-sdk :not(.ot-leg-btn-container)>button:hover,#onetrust-pc-sdk input[type=\"submit\"]:hover,#onetrust-pc-sdk input[type=\"reset\"]:hover,#onetrust-pc-sdk input[type=\"button\"]:hover,#onetrust-pc-sdk .ot-sdk-button:focus,#onetrust-pc-sdk :not(.ot-leg-btn-container)>button:focus,#onetrust-pc-sdk input[type=\"submit\"]:focus,#onetrust-pc-sdk input[type=\"reset\"]:focus,#onetrust-pc-sdk input[type=\"button\"]:focus,#ot-sdk-cookie-policy .ot-sdk-button:hover,#ot-sdk-cookie-policy :not(.ot-leg-btn-container)>button:hover,#ot-sdk-cookie-policy input[type=\"submit\"]:hover,#ot-sdk-cookie-policy input[type=\"reset\"]:hover,#ot-sdk-cookie-policy input[type=\"button\"]:hover,#ot-sdk-cookie-policy .ot-sdk-button:focus,#ot-sdk-cookie-policy :not(.ot-leg-btn-container)>button:focus,#ot-sdk-cookie-policy input[type=\"submit\"]:focus,#ot-sdk-cookie-policy input[type=\"reset\"]:focus,#ot-sdk-cookie-policy input[type=\"button\"]:focus{color:#333;border-color:#888;opacity:0.7}#onetrust-banner-sdk .ot-sdk-button:focus,#onetrust-banner-sdk :not(.ot-leg-btn-container)>button:focus,#onetrust-banner-sdk input[type=\"submit\"]:focus,#onetrust-banner-sdk input[type=\"reset\"]:focus,#onetrust-banner-sdk input[type=\"button\"]:focus,#onetrust-pc-sdk .ot-sdk-button:focus,#onetrust-pc-sdk :not(.ot-leg-btn-container)>button:focus,#onetrust-pc-sdk input[type=\"submit\"]:focus,#onetrust-pc-sdk input[type=\"reset\"]:focus,#onetrust-pc-sdk input[type=\"button\"]:focus,#ot-sdk-cookie-policy .ot-sdk-button:focus,#ot-sdk-cookie-policy :not(.ot-leg-btn-container)>button:focus,#ot-sdk-cookie-policy input[type=\"submit\"]:focus,#ot-sdk-cookie-policy input[type=\"reset\"]:focus,#ot-sdk-cookie-policy input[type=\"button\"]:focus{outline:2px solid #000}#onetrust-banner-sdk .ot-sdk-button.ot-sdk-button-primary,#onetrust-banner-sdk button.ot-sdk-button-primary,#onetrust-banner-sdk input[type=\"submit\"].ot-sdk-button-primary,#onetrust-banner-sdk input[type=\"reset\"].ot-sdk-button-primary,#onetrust-banner-sdk input[type=\"button\"].ot-sdk-button-primary,#onetrust-pc-sdk .ot-sdk-button.ot-sdk-button-primary,#onetrust-pc-sdk button.ot-sdk-button-primary,#onetrust-pc-sdk input[type=\"submit\"].ot-sdk-button-primary,#onetrust-pc-sdk input[type=\"reset\"].ot-sdk-button-primary,#onetrust-pc-sdk input[type=\"button\"].ot-sdk-button-primary,#ot-sdk-cookie-policy .ot-sdk-button.ot-sdk-button-primary,#ot-sdk-cookie-policy button.ot-sdk-button-primary,#ot-sdk-cookie-policy input[type=\"submit\"].ot-sdk-button-primary,#ot-sdk-cookie-policy input[type=\"reset\"].ot-sdk-button-primary,#ot-sdk-cookie-policy input[type=\"button\"].ot-sdk-button-primary{color:#fff;background-color:#33c3f0;border-color:#33c3f0}#onetrust-banner-sdk .ot-sdk-button.ot-sdk-button-primary:hover,#onetrust-banner-sdk button.ot-sdk-button-primary:hover,#onetrust-banner-sdk input[type=\"submit\"].ot-sdk-button-primary:hover,#onetrust-banner-sdk input[type=\"reset\"].ot-sdk-button-primary:hover,#onetrust-banner-sdk input[type=\"button\"].ot-sdk-button-primary:hover,#onetrust-banner-sdk .ot-sdk-button.ot-sdk-button-primary:focus,#onetrust-banner-sdk button.ot-sdk-button-primary:focus,#onetrust-banner-sdk input[type=\"submit\"].ot-sdk-button-primary:focus,#onetrust-banner-sdk input[type=\"reset\"].ot-sdk-button-primary:focus,#onetrust-banner-sdk input[type=\"button\"].ot-sdk-button-primary:focus,#onetrust-pc-sdk .ot-sdk-button.ot-sdk-button-primary:hover,#onetrust-pc-sdk button.ot-sdk-button-primary:hover,#onetrust-pc-sdk input[type=\"submit\"].ot-sdk-button-primary:hover,#onetrust-pc-sdk input[type=\"reset\"].ot-sdk-button-primary:hover,#onetrust-pc-sdk input[type=\"button\"].ot-sdk-button-primary:hover,#onetrust-pc-sdk .ot-sdk-button.ot-sdk-button-primary:focus,#onetrust-pc-sdk button.ot-sdk-button-primary:focus,#onetrust-pc-sdk input[type=\"submit\"].ot-sdk-button-primary:focus,#onetrust-pc-sdk input[type=\"reset\"].ot-sdk-button-primary:focus,#onetrust-pc-sdk input[type=\"button\"].ot-sdk-button-primary:focus,#ot-sdk-cookie-policy .ot-sdk-button.ot-sdk-button-primary:hover,#ot-sdk-cookie-policy button.ot-sdk-button-primary:hover,#ot-sdk-cookie-policy input[type=\"submit\"].ot-sdk-button-primary:hover,#ot-sdk-cookie-policy input[type=\"reset\"].ot-sdk-button-primary:hover,#ot-sdk-cookie-policy input[type=\"button\"].ot-sdk-button-primary:hover,#ot-sdk-cookie-policy .ot-sdk-button.ot-sdk-button-primary:focus,#ot-sdk-cookie-policy button.ot-sdk-button-primary:focus,#ot-sdk-cookie-policy input[type=\"submit\"].ot-sdk-button-primary:focus,#ot-sdk-cookie-policy input[type=\"reset\"].ot-sdk-button-primary:focus,#ot-sdk-cookie-policy input[type=\"button\"].ot-sdk-button-primary:focus{color:#fff;background-color:#1eaedb;border-color:#1eaedb}#onetrust-banner-sdk input[type=\"email\"],#onetrust-banner-sdk input[type=\"number\"],#onetrust-banner-sdk input[type=\"search\"],#onetrust-banner-sdk input[type=\"text\"],#onetrust-banner-sdk input[type=\"tel\"],#onetrust-banner-sdk input[type=\"url\"],#onetrust-banner-sdk input[type=\"password\"],#onetrust-banner-sdk textarea,#onetrust-banner-sdk select,#onetrust-pc-sdk input[type=\"email\"],#onetrust-pc-sdk input[type=\"number\"],#onetrust-pc-sdk input[type=\"search\"],#onetrust-pc-sdk input[type=\"text\"],#onetrust-pc-sdk input[type=\"tel\"],#onetrust-pc-sdk input[type=\"url\"],#onetrust-pc-sdk input[type=\"password\"],#onetrust-pc-sdk textarea,#onetrust-pc-sdk select,#ot-sdk-cookie-policy input[type=\"email\"],#ot-sdk-cookie-policy input[type=\"number\"],#ot-sdk-cookie-policy input[type=\"search\"],#ot-sdk-cookie-policy input[type=\"text\"],#ot-sdk-cookie-policy input[type=\"tel\"],#ot-sdk-cookie-policy input[type=\"url\"],#ot-sdk-cookie-policy input[type=\"password\"],#ot-sdk-cookie-policy textarea,#ot-sdk-cookie-policy select{height:38px;padding:6px 10px;background-color:#fff;border:1px solid #d1d1d1;border-radius:4px;box-shadow:none;box-sizing:border-box}#onetrust-banner-sdk input[type=\"email\"],#onetrust-banner-sdk input[type=\"number\"],#onetrust-banner-sdk input[type=\"search\"],#onetrust-banner-sdk input[type=\"text\"],#onetrust-banner-sdk input[type=\"tel\"],#onetrust-banner-sdk input[type=\"url\"],#onetrust-banner-sdk input[type=\"password\"],#onetrust-banner-sdk textarea,#onetrust-pc-sdk input[type=\"email\"],#onetrust-pc-sdk input[type=\"number\"],#onetrust-pc-sdk input[type=\"search\"],#onetrust-pc-sdk input[type=\"text\"],#onetrust-pc-sdk input[type=\"tel\"],#onetrust-pc-sdk input[type=\"url\"],#onetrust-pc-sdk input[type=\"password\"],#onetrust-pc-sdk textarea,#ot-sdk-cookie-policy input[type=\"email\"],#ot-sdk-cookie-policy input[type=\"number\"],#ot-sdk-cookie-policy input[type=\"search\"],#ot-sdk-cookie-policy input[type=\"text\"],#ot-sdk-cookie-policy input[type=\"tel\"],#ot-sdk-cookie-policy input[type=\"url\"],#ot-sdk-cookie-policy input[type=\"password\"],#ot-sdk-cookie-policy textarea{-webkit-appearance:none;-moz-appearance:none;appearance:none}#onetrust-banner-sdk textarea,#onetrust-pc-sdk textarea,#ot-sdk-cookie-policy textarea{min-height:65px;padding-top:6px;padding-bottom:6px}#onetrust-banner-sdk input[type=\"email\"]:focus,#onetrust-banner-sdk input[type=\"number\"]:focus,#onetrust-banner-sdk input[type=\"search\"]:focus,#onetrust-banner-sdk input[type=\"text\"]:focus,#onetrust-banner-sdk input[type=\"tel\"]:focus,#onetrust-banner-sdk input[type=\"url\"]:focus,#onetrust-banner-sdk input[type=\"password\"]:focus,#onetrust-banner-sdk textarea:focus,#onetrust-banner-sdk select:focus,#onetrust-pc-sdk input[type=\"email\"]:focus,#onetrust-pc-sdk input[type=\"number\"]:focus,#onetrust-pc-sdk input[type=\"search\"]:focus,#onetrust-pc-sdk input[type=\"text\"]:focus,#onetrust-pc-sdk input[type=\"tel\"]:focus,#onetrust-pc-sdk input[type=\"url\"]:focus,#onetrust-pc-sdk input[type=\"password\"]:focus,#onetrust-pc-sdk textarea:focus,#onetrust-pc-sdk select:focus,#ot-sdk-cookie-policy input[type=\"email\"]:focus,#ot-sdk-cookie-policy input[type=\"number\"]:focus,#ot-sdk-cookie-policy input[type=\"search\"]:focus,#ot-sdk-cookie-policy input[type=\"text\"]:focus,#ot-sdk-cookie-policy input[type=\"tel\"]:focus,#ot-sdk-cookie-policy input[type=\"url\"]:focus,#ot-sdk-cookie-policy input[type=\"password\"]:focus,#ot-sdk-cookie-policy textarea:focus,#ot-sdk-cookie-policy select:focus{border:1px solid #000;outline:0}#onetrust-banner-sdk label,#onetrust-banner-sdk legend,#onetrust-pc-sdk label,#onetrust-pc-sdk legend,#ot-sdk-cookie-policy label,#ot-sdk-cookie-policy legend{display:block;margin-bottom:0.5rem;font-weight:600}#onetrust-banner-sdk fieldset,#onetrust-pc-sdk fieldset,#ot-sdk-cookie-policy fieldset{padding:0;border-width:0}#onetrust-banner-sdk input[type=\"checkbox\"],#onetrust-banner-sdk input[type=\"radio\"],#onetrust-pc-sdk input[type=\"checkbox\"],#onetrust-pc-sdk input[type=\"radio\"],#ot-sdk-cookie-policy input[type=\"checkbox\"],#ot-sdk-cookie-policy input[type=\"radio\"]{display:inline}#onetrust-banner-sdk label>.label-body,#onetrust-pc-sdk label>.label-body,#ot-sdk-cookie-policy label>.label-body{display:inline-block;margin-left:0.5rem;font-weight:normal}#onetrust-banner-sdk ul,#onetrust-pc-sdk ul,#ot-sdk-cookie-policy ul{list-style:circle inside}#onetrust-banner-sdk ol,#onetrust-pc-sdk ol,#ot-sdk-cookie-policy ol{list-style:decimal inside}#onetrust-banner-sdk ol,#onetrust-banner-sdk ul,#onetrust-pc-sdk ol,#onetrust-pc-sdk ul,#ot-sdk-cookie-policy ol,#ot-sdk-cookie-policy ul{padding-left:0;margin-top:0}#onetrust-banner-sdk ul ul,#onetrust-banner-sdk ul ol,#onetrust-banner-sdk ol ol,#onetrust-banner-sdk ol ul,#onetrust-pc-sdk ul ul,#onetrust-pc-sdk ul ol,#onetrust-pc-sdk ol ol,#onetrust-pc-sdk ol ul,#ot-sdk-cookie-policy ul ul,#ot-sdk-cookie-policy ul ol,#ot-sdk-cookie-policy ol ol,#ot-sdk-cookie-policy ol ul{margin:1.5rem 0 1.5rem 3rem;font-size:90%}#onetrust-banner-sdk li,#onetrust-pc-sdk li,#ot-sdk-cookie-policy li{margin-bottom:1rem}#onetrust-banner-sdk code,#onetrust-pc-sdk code,#ot-sdk-cookie-policy code{padding:0.2rem 0.5rem;margin:0 0.2rem;font-size:90%;white-space:nowrap;background:#f1f1f1;border:1px solid #e1e1e1;border-radius:4px}#onetrust-banner-sdk pre>code,#onetrust-pc-sdk pre>code,#ot-sdk-cookie-policy pre>code{display:block;padding:1rem 1.5rem;white-space:pre}#onetrust-banner-sdk th,#onetrust-banner-sdk td,#onetrust-pc-sdk th,#onetrust-pc-sdk td,#ot-sdk-cookie-policy th,#ot-sdk-cookie-policy td{padding:12px 15px;text-align:left;border-bottom:1px solid #e1e1e1}#onetrust-banner-sdk .ot-sdk-u-full-width,#onetrust-pc-sdk .ot-sdk-u-full-width,#ot-sdk-cookie-policy .ot-sdk-u-full-width{width:100%;box-sizing:border-box}#onetrust-banner-sdk .ot-sdk-u-max-full-width,#onetrust-pc-sdk .ot-sdk-u-max-full-width,#ot-sdk-cookie-policy .ot-sdk-u-max-full-width{max-width:100%;box-sizing:border-box}#onetrust-banner-sdk .ot-sdk-u-pull-right,#onetrust-pc-sdk .ot-sdk-u-pull-right,#ot-sdk-cookie-policy .ot-sdk-u-pull-right{float:right}#onetrust-banner-sdk .ot-sdk-u-pull-left,#onetrust-pc-sdk .ot-sdk-u-pull-left,#ot-sdk-cookie-policy .ot-sdk-u-pull-left{float:left}#onetrust-banner-sdk hr,#onetrust-pc-sdk hr,#ot-sdk-cookie-policy hr{margin-top:3rem;margin-bottom:3.5rem;border-width:0;border-top:1px solid #e1e1e1}#onetrust-banner-sdk .ot-sdk-container:after,#onetrust-banner-sdk .ot-sdk-row:after,#onetrust-banner-sdk .ot-sdk-u-cf,#onetrust-pc-sdk .ot-sdk-container:after,#onetrust-pc-sdk .ot-sdk-row:after,#onetrust-pc-sdk .ot-sdk-u-cf,#ot-sdk-cookie-policy .ot-sdk-container:after,#ot-sdk-cookie-policy .ot-sdk-row:after,#ot-sdk-cookie-policy .ot-sdk-u-cf{content:\"\";display:table;clear:both}#onetrust-banner-sdk .ot-sdk-row,#onetrust-pc-sdk .ot-sdk-row,#ot-sdk-cookie-policy .ot-sdk-row{margin:0;max-width:none;display:block}\n#onetrust-banner-sdk{box-shadow:0 0 18px rgba(0,0,0,.2)}#onetrust-banner-sdk.otFlat{position:fixed;z-index:2147483645;bottom:0;right:0;left:0;background-color:#fff;max-height:90%;overflow-x:hidden;overflow-y:auto}#onetrust-banner-sdk>.ot-sdk-container{overflow:hidden}#onetrust-banner-sdk::-webkit-scrollbar{width:11px}#onetrust-banner-sdk::-webkit-scrollbar-thumb{border-radius:10px;background:#c1c1c1}#onetrust-banner-sdk{scrollbar-arrow-color:#c1c1c1;scrollbar-darkshadow-color:#c1c1c1;scrollbar-face-color:#c1c1c1;scrollbar-shadow-color:#c1c1c1}#onetrust-banner-sdk #onetrust-policy{margin:1.25em 0 .625em 2em;overflow:hidden}#onetrust-banner-sdk #onetrust-policy .ot-gv-list-handler{float:left;font-size:.82em;padding:0;margin-bottom:0;border:0;line-height:normal;height:auto;width:auto}#onetrust-banner-sdk #onetrust-policy-title{font-size:1.2em;line-height:1.3;margin-bottom:10px}#onetrust-banner-sdk #onetrust-policy-text{clear:both;text-align:left;font-size:.88em;line-height:1.4}#onetrust-banner-sdk #onetrust-policy-text *{font-size:inherit;line-height:inherit}#onetrust-banner-sdk #onetrust-policy-title,#onetrust-banner-sdk #onetrust-policy-text{color:dimgray;float:left}#onetrust-banner-sdk #onetrust-button-group-parent{min-height:1px;text-align:center}#onetrust-banner-sdk #onetrust-button-group{display:inline-block}#onetrust-banner-sdk #onetrust-accept-btn-handler,#onetrust-banner-sdk #onetrust-reject-all-handler,#onetrust-banner-sdk #onetrust-pc-btn-handler{background-color:#68b631;color:#fff;border-color:#68b631;margin-right:1em;min-width:125px;height:auto;white-space:normal;word-break:break-word;word-wrap:break-word;padding:12px 10px;line-height:1.2;font-size:.813em;font-weight:600}#onetrust-banner-sdk #onetrust-pc-btn-handler.cookie-setting-link{background-color:#fff;border:none;color:#68b631;text-decoration:underline;padding-left:0;padding-right:0}#onetrust-banner-sdk .onetrust-close-btn-ui{width:44px;height:44px;background-size:12px;border:none;position:relative;margin:auto;padding:0}#onetrust-banner-sdk .banner_logo{display:none}#onetrust-banner-sdk .ot-b-addl-desc{clear:both;float:left;display:block}#onetrust-banner-sdk #banner-options{float:left;display:table;margin-right:0;margin-left:1em;width:calc(100% - 1em)}#onetrust-banner-sdk .banner-option-input{cursor:pointer;width:auto;height:auto;border:none;padding:0;padding-right:3px;margin:0 0 10px;font-size:.82em;line-height:1.4}#onetrust-banner-sdk .banner-option-input *{pointer-events:none;font-size:inherit;line-height:inherit}#onetrust-banner-sdk .banner-option-input[aria-expanded=true]~.banner-option-details{display:block;height:auto}#onetrust-banner-sdk .banner-option-input[aria-expanded=true] .ot-arrow-container{transform:rotate(90deg)}#onetrust-banner-sdk .banner-option{margin-bottom:12px;margin-left:0;border:none;float:left;padding:0}#onetrust-banner-sdk .banner-option:first-child{padding-left:2px}#onetrust-banner-sdk .banner-option:not(:first-child){padding:0;border:none}#onetrust-banner-sdk .banner-option-header{cursor:pointer;display:inline-block}#onetrust-banner-sdk .banner-option-header :first-child{color:dimgray;font-weight:bold;float:left}#onetrust-banner-sdk .banner-option-header .ot-arrow-container{display:inline-block;border-top:6px solid transparent;border-bottom:6px solid transparent;border-left:6px solid dimgray;margin-left:10px;vertical-align:middle}#onetrust-banner-sdk .banner-option-details{display:none;font-size:.83em;line-height:1.5;padding:10px 0px 5px 10px;margin-right:10px;height:0px}#onetrust-banner-sdk .banner-option-details *{font-size:inherit;line-height:inherit;color:dimgray}#onetrust-banner-sdk .ot-arrow-container,#onetrust-banner-sdk .banner-option-details{transition:all 300ms ease-in 0s;-webkit-transition:all 300ms ease-in 0s;-moz-transition:all 300ms ease-in 0s;-o-transition:all 300ms ease-in 0s}#onetrust-banner-sdk .ot-dpd-container{float:left}#onetrust-banner-sdk .ot-dpd-title{margin-bottom:10px}#onetrust-banner-sdk .ot-dpd-title,#onetrust-banner-sdk .ot-dpd-desc{font-size:.88em;line-height:1.4;color:dimgray}#onetrust-banner-sdk .ot-dpd-title *,#onetrust-banner-sdk .ot-dpd-desc *{font-size:inherit;line-height:inherit}#onetrust-banner-sdk.ot-iab-2 #onetrust-policy-text *{margin-bottom:0}#onetrust-banner-sdk.ot-iab-2 .onetrust-vendors-list-handler{display:block;margin-left:0;margin-top:5px;clear:both;margin-bottom:0;padding:0;border:0;height:auto;width:auto}#onetrust-banner-sdk.ot-iab-2 #onetrust-button-group button{display:block}#onetrust-banner-sdk #onetrust-policy-text,#onetrust-banner-sdk .ot-dpd-desc,#onetrust-banner-sdk .ot-b-addl-desc{font-size:.813em;line-height:1.5}#onetrust-banner-sdk .ot-dpd-desc{margin-bottom:10px}#onetrust-banner-sdk .ot-dpd-desc>.ot-b-addl-desc{margin-top:10px;margin-bottom:10px;font-size:1em}@media only screen and (max-width: 425px){#onetrust-banner-sdk #onetrust-close-btn-container{position:absolute;top:10px;right:10px}#onetrust-banner-sdk #onetrust-policy{margin-left:0}#onetrust-banner-sdk #onetrust-button-group{display:block}#onetrust-banner-sdk #onetrust-accept-btn-handler,#onetrust-banner-sdk #onetrust-reject-all-handler,#onetrust-banner-sdk #onetrust-pc-btn-handler{width:100%}#onetrust-banner-sdk .onetrust-close-btn-ui{top:auto;transform:none}#onetrust-banner-sdk #onetrust-policy-title{display:inline;float:none}#onetrust-banner-sdk #banner-options{margin:0;padding:0;width:100%}}@media only screen and (min-width: 426px)and (max-width: 896px){#onetrust-banner-sdk #onetrust-close-btn-container{position:absolute;top:0;right:0}#onetrust-banner-sdk #onetrust-policy{margin-left:1em;margin-right:1em}#onetrust-banner-sdk .onetrust-close-btn-ui{top:10px;right:10px}#onetrust-banner-sdk:not(.ot-iab-2) #onetrust-group-container{width:95%}#onetrust-banner-sdk.ot-iab-2 #onetrust-group-container{width:100%}#onetrust-banner-sdk #onetrust-button-group-parent{width:100%;position:relative;margin-left:0}#onetrust-banner-sdk #onetrust-button-group button{display:inline-block}#onetrust-banner-sdk #onetrust-button-group{margin-right:0;text-align:center}#onetrust-banner-sdk .has-reject-all-button #onetrust-pc-btn-handler{float:left}#onetrust-banner-sdk .has-reject-all-button #onetrust-reject-all-handler,#onetrust-banner-sdk .has-reject-all-button #onetrust-accept-btn-handler{float:right}#onetrust-banner-sdk .has-reject-all-button #onetrust-button-group{width:calc(100% - 2em);margin-right:0}#onetrust-banner-sdk .has-reject-all-button #onetrust-pc-btn-handler.cookie-setting-link{padding-left:0px;text-align:left}#onetrust-banner-sdk.ot-buttons-fw .ot-sdk-three button{width:100%;text-align:center}#onetrust-banner-sdk.ot-buttons-fw #onetrust-button-group-parent button{float:none}#onetrust-banner-sdk.ot-buttons-fw #onetrust-pc-btn-handler.cookie-setting-link{text-align:center}}@media only screen and (min-width: 550px){#onetrust-banner-sdk .banner-option:not(:first-child){border-left:1px solid #d8d8d8;padding-left:25px}}@media only screen and (min-width: 425px)and (max-width: 550px){#onetrust-banner-sdk.ot-iab-2 #onetrust-button-group,#onetrust-banner-sdk.ot-iab-2 #onetrust-policy,#onetrust-banner-sdk.ot-iab-2 .banner-option{width:100%}}@media only screen and (min-width: 769px){#onetrust-banner-sdk #onetrust-button-group{margin-right:30%}#onetrust-banner-sdk #banner-options{margin-left:2em;margin-right:5em;margin-bottom:1.25em;width:calc(100% - 7em)}}@media only screen and (min-width: 897px)and (max-width: 1023px){#onetrust-banner-sdk.vertical-align-content #onetrust-button-group-parent{position:absolute;top:50%;left:75%;transform:translateY(-50%)}#onetrust-banner-sdk #onetrust-close-btn-container{top:50%;margin:auto;transform:translate(-50%, -50%);position:absolute;padding:0;right:0}#onetrust-banner-sdk #onetrust-close-btn-container button{position:relative;margin:0;right:-22px;top:2px}}@media only screen and (min-width: 1024px){#onetrust-banner-sdk #onetrust-close-btn-container{top:50%;margin:auto;transform:translate(-50%, -50%);position:absolute;right:0}#onetrust-banner-sdk #onetrust-close-btn-container button{right:-12px}#onetrust-banner-sdk #onetrust-policy{margin-left:2em}#onetrust-banner-sdk.vertical-align-content #onetrust-button-group-parent{position:absolute;top:50%;left:60%;transform:translateY(-50%)}#onetrust-banner-sdk.ot-iab-2 #onetrust-policy-title{width:50%}#onetrust-banner-sdk.ot-iab-2 #onetrust-policy-text,#onetrust-banner-sdk.ot-iab-2 :not(.ot-dpd-desc)>.ot-b-addl-desc{margin-bottom:1em;width:50%;border-right:1px solid #d8d8d8;padding-right:1rem}#onetrust-banner-sdk.ot-iab-2 #onetrust-policy-text{margin-bottom:0;padding-bottom:1em}#onetrust-banner-sdk.ot-iab-2 :not(.ot-dpd-desc)>.ot-b-addl-desc{margin-bottom:0;padding-bottom:1em}#onetrust-banner-sdk.ot-iab-2 .ot-dpd-container{width:45%;padding-left:1rem;display:inline-block;float:none}#onetrust-banner-sdk.ot-iab-2 .ot-dpd-title{line-height:1.7}#onetrust-banner-sdk.ot-iab-2 #onetrust-button-group-parent{left:auto;right:4%;margin-left:0}#onetrust-banner-sdk.ot-iab-2 #onetrust-button-group button{display:block}#onetrust-banner-sdk:not(.ot-iab-2) #onetrust-button-group-parent{margin:auto;width:30%}#onetrust-banner-sdk:not(.ot-iab-2) #onetrust-group-container{width:60%}#onetrust-banner-sdk #onetrust-button-group{margin-right:auto}#onetrust-banner-sdk #onetrust-accept-btn-handler,#onetrust-banner-sdk #onetrust-reject-all-handler,#onetrust-banner-sdk #onetrust-pc-btn-handler{margin-top:1em}}@media only screen and (min-width: 890px){#onetrust-banner-sdk.ot-buttons-fw:not(.ot-iab-2) #onetrust-button-group-parent{padding-left:3%;padding-right:4%;margin-left:0}#onetrust-banner-sdk.ot-buttons-fw:not(.ot-iab-2) #onetrust-button-group{margin-right:0;margin-top:1.25em;width:100%}#onetrust-banner-sdk.ot-buttons-fw:not(.ot-iab-2) #onetrust-button-group button{width:100%;margin-bottom:5px;margin-top:5px}#onetrust-banner-sdk.ot-buttons-fw:not(.ot-iab-2) #onetrust-button-group button:last-of-type{margin-bottom:20px}}@media only screen and (min-width: 1280px){#onetrust-banner-sdk:not(.ot-iab-2) #onetrust-group-container{width:55%}#onetrust-banner-sdk:not(.ot-iab-2) #onetrust-button-group-parent{width:44%;padding-left:2%;padding-right:2%}#onetrust-banner-sdk:not(.ot-iab-2).vertical-align-content #onetrust-button-group-parent{position:absolute;left:55%}}\n        #onetrust-consent-sdk #onetrust-banner-sdk {background-color: #FFFFFF;}\n            #onetrust-consent-sdk #onetrust-policy-title,\n                    #onetrust-consent-sdk #onetrust-policy-text,\n                    #onetrust-consent-sdk .ot-b-addl-desc,\n                    #onetrust-consent-sdk .ot-dpd-desc,\n                    #onetrust-consent-sdk .ot-dpd-title,\n                    #onetrust-consent-sdk #onetrust-policy-text *:not(.onetrust-vendors-list-handler),\n                    #onetrust-consent-sdk .ot-dpd-desc *:not(.onetrust-vendors-list-handler),\n                    #onetrust-consent-sdk #onetrust-banner-sdk #banner-options *,\n                    #onetrust-banner-sdk .ot-cat-header {\n                        color: #302c2c;\n                    }\n            #onetrust-consent-sdk #onetrust-banner-sdk .banner-option-details {\n                    background-color: #c3c3ca;}\n            #onetrust-consent-sdk #onetrust-accept-btn-handler,\n                         #onetrust-banner-sdk #onetrust-reject-all-handler {\n                            background-color: #027bc5;border-color: #027bc5;\n                            color: #FFFFFF;\n                        }\n            #onetrust-consent-sdk #onetrust-pc-btn-handler,\n            #onetrust-consent-sdk #onetrust-pc-btn-handler.cookie-setting-link {\n                color: #027bc5; border-color: #027bc5;\n                background-color: \n                #FFFFFF;\n            }#onetrust-pc-sdk{position:fixed;width:730px;max-width:730px;height:610px;left:0;right:0;top:0;bottom:0;margin:auto;font-size:16px;z-index:2147483647;border-radius:2px;background-color:#fff;box-shadow:0 2px 4px 0 rgba(0,0,0,0),0 7px 14px 0 rgba(50,50,93,.1)}#onetrust-pc-sdk *,#onetrust-pc-sdk ::after,#onetrust-pc-sdk ::before{box-sizing:content-box}#onetrust-pc-sdk #ot-addtl-venlst .ot-arw-cntr,#onetrust-pc-sdk .ot-hide-tgl{visibility:hidden}#onetrust-pc-sdk #ot-addtl-venlst .ot-arw-cntr *,#onetrust-pc-sdk .ot-hide-tgl *{visibility:hidden}#onetrust-pc-sdk #ot-pc-content,#onetrust-pc-sdk #ot-pc-lst{height:calc(100% - 185px)}#onetrust-pc-sdk li{list-style:none}#onetrust-pc-sdk ul,#onetrust-pc-sdk li{margin:0}#onetrust-pc-sdk a{text-decoration:none}#onetrust-pc-sdk .ot-link-btn{padding:0;margin-bottom:0;border:0;font-weight:normal;line-height:normal;width:auto;height:auto}#onetrust-pc-sdk .ot-grps-cntr *::-webkit-scrollbar,#onetrust-pc-sdk .ot-pc-scrollbar::-webkit-scrollbar{width:11px}#onetrust-pc-sdk .ot-grps-cntr *::-webkit-scrollbar-thumb,#onetrust-pc-sdk .ot-pc-scrollbar::-webkit-scrollbar-thumb{border-radius:10px;background:#c1c1c1}#onetrust-pc-sdk .ot-grps-cntr *,#onetrust-pc-sdk .ot-pc-scrollbar{scrollbar-arrow-color:#c1c1c1;scrollbar-darkshadow-color:#c1c1c1;scrollbar-face-color:#c1c1c1;scrollbar-shadow-color:#c1c1c1}#onetrust-pc-sdk .ot-pc-header{height:auto;padding:10px;display:table;vertical-align:middle;width:calc(100% - 20px);min-height:52px;border-bottom:1px solid #d8d8d8;position:relative}#onetrust-pc-sdk .ot-pc-logo{display:table-cell;vertical-align:middle;width:180px;height:40px}#onetrust-pc-sdk .ot-title-cntr{position:relative;display:table-cell;vertical-align:middle;width:calc(100% - 190px);padding-left:10px}#onetrust-pc-sdk .ot-always-active{font-size:.813em;line-height:1.5;font-weight:700;color:#3860be}#onetrust-pc-sdk .ot-close-cntr{float:right;position:absolute;right:-9px;top:50%;transform:translateY(-50%)}#onetrust-pc-sdk #ot-pc-content{position:relative;overflow-y:auto;overflow-x:hidden}#onetrust-pc-sdk .ot-grps-cntr,#onetrust-pc-sdk .ot-grps-cntr>*{height:100%;overflow-y:auto}#onetrust-pc-sdk .category-menu-switch-handler{cursor:pointer;border-left:10px solid transparent;background-color:#f4f4f4;border-bottom:1px solid #d7d7d7;padding-top:12px;padding-right:5px;padding-bottom:12px;padding-left:12px;overflow:hidden}#onetrust-pc-sdk .category-menu-switch-handler h3{float:left;text-align:left;margin:0;color:dimgray;line-height:1.4;font-size:.875em;word-break:break-word;word-wrap:break-word}#onetrust-pc-sdk .ot-active-menu{border-left:10px solid #68b631;background-color:#fff;border-bottom:none;position:relative}#onetrust-pc-sdk .ot-active-menu h3{color:#263238;font-weight:bold}#onetrust-pc-sdk .ot-desc-cntr{word-break:break-word;word-wrap:break-word;padding-top:20px;padding-right:16px;padding-bottom:15px}#onetrust-pc-sdk .ot-grp-desc{word-break:break-word;word-wrap:break-word;text-align:left;font-size:.813em;line-height:1.5;margin:0}#onetrust-pc-sdk .ot-grp-desc *{font-size:inherit;line-height:inherit}#onetrust-pc-sdk #ot-pc-desc a{color:#3860be;cursor:pointer;font-size:1em}#onetrust-pc-sdk #ot-pc-desc a:hover{color:#1883fd}#onetrust-pc-sdk #ot-pc-desc *{font-size:inherit}#onetrust-pc-sdk #ot-pc-desc ul li{padding:10px 0px;border-bottom:1px solid #e2e2e2}#onetrust-pc-sdk .ot-btn-subcntr{float:right}#onetrust-pc-sdk .ot-close-icon{background-image:url(\"data:image/svg+xml;base64,PHN2ZyB4bWxucz0iaHR0cDovL3d3dy53My5vcmcvMjAwMC9zdmciIHhtbG5zOnhsaW5rPSJodHRwOi8vd3d3LnczLm9yZy8xOTk5L3hsaW5rIiB4PSIwcHgiIHk9IjBweCIgd2lkdGg9IjQ3Ljk3MSIgaGVpZ2h0PSI0Ny45NzEiIHZpZXdCb3g9IjAgMCA0Ny45NzEgNDcuOTcxIiBzdHlsZT0iZW5hYmxlLWJhY2tncm91bmQ6bmV3IDAgMCA0Ny45NzEgNDcuOTcxOyIgeG1sOnNwYWNlPSJwcmVzZXJ2ZSI+PGc+PHBhdGggZD0iTTI4LjIyOCwyMy45ODZMNDcuMDkyLDUuMTIyYzEuMTcyLTEuMTcxLDEuMTcyLTMuMDcxLDAtNC4yNDJjLTEuMTcyLTEuMTcyLTMuMDctMS4xNzItNC4yNDIsMEwyMy45ODYsMTkuNzQ0TDUuMTIxLDAuODhjLTEuMTcyLTEuMTcyLTMuMDctMS4xNzItNC4yNDIsMGMtMS4xNzIsMS4xNzEtMS4xNzIsMy4wNzEsMCw0LjI0MmwxOC44NjUsMTguODY0TDAuODc5LDQyLjg1Yy0xLjE3MiwxLjE3MS0xLjE3MiwzLjA3MSwwLDQuMjQyQzEuNDY1LDQ3LjY3NywyLjIzMyw0Ny45NywzLDQ3Ljk3czEuNTM1LTAuMjkzLDIuMTIxLTAuODc5bDE4Ljg2NS0xOC44NjRMNDIuODUsNDcuMDkxYzAuNTg2LDAuNTg2LDEuMzU0LDAuODc5LDIuMTIxLDAuODc5czEuNTM1LTAuMjkzLDIuMTIxLTAuODc5YzEuMTcyLTEuMTcxLDEuMTcyLTMuMDcxLDAtNC4yNDJMMjguMjI4LDIzLjk4NnoiLz48L2c+PC9zdmc+\");background-size:12px;background-repeat:no-repeat;background-position:center;height:44px;width:44px;display:inline-block}#onetrust-pc-sdk .ot-tgl{float:right;position:relative;z-index:1}#onetrust-pc-sdk .ot-tgl input:checked+.ot-switch .ot-switch-nob{background-color:#cddcf2}#onetrust-pc-sdk .ot-tgl input:checked+.ot-switch .ot-switch-nob:before{-webkit-transform:translateX(16px);-ms-transform:translateX(16px);transform:translateX(16px);background-color:#4285f4}#onetrust-pc-sdk .ot-tgl input:focus+.ot-switch .ot-switch-nob:before{box-shadow:0 0 1px #2196f3;outline:#000 auto 5px}#onetrust-pc-sdk .ot-switch{position:relative;display:inline-block;width:35px;height:10px;margin-bottom:0}#onetrust-pc-sdk .ot-switch-nob{position:absolute;cursor:pointer;top:0;left:0;right:0;bottom:0;background-color:#f2f1f1;border:none;transition:all .2s ease-in 0s;-moz-transition:all .2s ease-in 0s;-o-transition:all .2s ease-in 0s;-webkit-transition:all .2s ease-in 0s;border-radius:46px}#onetrust-pc-sdk .ot-switch-nob:before{position:absolute;content:\"\";height:20px;width:20px;bottom:1px;background-color:#7d7d7d;-webkit-transition:.4s;border-radius:100%;top:-5px;transition:.4s}#onetrust-pc-sdk .ot-chkbox{z-index:1;position:relative}#onetrust-pc-sdk .ot-chkbox input:checked~label::before{background-color:#3860be}#onetrust-pc-sdk .ot-chkbox input+label::after{content:none;color:#fff}#onetrust-pc-sdk .ot-chkbox input:checked+label::after{content:\"\"}#onetrust-pc-sdk .ot-chkbox input:focus+label::before{outline-style:solid;outline-width:2px;outline-style:auto}#onetrust-pc-sdk .ot-chkbox label{position:relative;height:20px;padding-left:30px;display:inline-block;cursor:pointer}#onetrust-pc-sdk .ot-chkbox label::before,#onetrust-pc-sdk .ot-chkbox label::after{position:absolute;content:\"\";display:inline-block;border-radius:3px}#onetrust-pc-sdk .ot-chkbox label::before{height:18px;width:18px;border:1px solid #3860be;left:0px}#onetrust-pc-sdk .ot-chkbox label::after{height:5px;width:9px;border-left:3px solid;border-bottom:3px solid;transform:rotate(-45deg);-o-transform:rotate(-45deg);-ms-transform:rotate(-45deg);-webkit-transform:rotate(-45deg);left:4px;top:5px}#onetrust-pc-sdk .ot-label-txt{display:none}#onetrust-pc-sdk .ot-fltr-opt .ot-label-txt{display:block}#onetrust-pc-sdk .ot-chkbox input,#onetrust-pc-sdk .ot-tgl input{position:absolute;opacity:0;width:0;height:0}#onetrust-pc-sdk .ot-arw-cntr{float:right;position:relative}#onetrust-pc-sdk .ot-arw{width:16px;height:16px;margin-left:5px;color:dimgray;display:inline-block;vertical-align:middle;-webkit-transition:all 300ms ease-in 0s;-moz-transition:all 300ms ease-in 0s;-o-transition:all 300ms ease-in 0s;transition:all 300ms ease-in 0s}#onetrust-pc-sdk input:checked~.ot-acc-hdr .ot-arw{transform:rotate(90deg);-o-transform:rotate(90deg);-ms-transform:rotate(90deg);-webkit-transform:rotate(90deg)}#onetrust-pc-sdk .ot-label-status{font-size:.75em;position:relative;top:2px;display:none;padding-right:5px;float:left}#onetrust-pc-sdk #ot-lst-cnt .ot-label-status{top:-6px}#onetrust-pc-sdk .ot-fltr-btns{margin-left:15px;overflow:hidden;margin-right:15px}#onetrust-pc-sdk .ot-fltr-btns button{padding:12px 30px}#onetrust-pc-sdk .ot-pc-footer{position:absolute;bottom:0px;width:100%;max-height:160px;border-top:1px solid #d8d8d8}#onetrust-pc-sdk .ot-pc-footer button{margin-top:20px;margin-bottom:20px;font-weight:600;font-size:.813em;min-height:40px;height:auto;line-height:normal;padding:10px 30px}#onetrust-pc-sdk .ot-tab-desc{margin-left:3%}#onetrust-pc-sdk .ot-grp-hdr1{display:inline-block;width:100%;margin-bottom:10px}#onetrust-pc-sdk .ot-desc-cntr h3{color:#263238;display:inline-block;vertical-align:middle;margin:0;font-weight:bold;font-size:.875em;line-height:1.3;max-width:70%}#onetrust-pc-sdk #ot-pvcy-hdr{margin-bottom:10px}#onetrust-pc-sdk .ot-vlst-cntr{overflow:hidden}#onetrust-pc-sdk .category-vendors-list-handler,#onetrust-pc-sdk .category-host-list-handler,#onetrust-pc-sdk .category-vendors-list-handler+a{display:block;float:left;color:#3860be;font-size:.813em;font-weight:400;line-height:1.1;cursor:pointer}#onetrust-pc-sdk .category-vendors-list-handler:hover,#onetrust-pc-sdk .category-host-list-handler:hover,#onetrust-pc-sdk .category-vendors-list-handler+a:hover{color:#1883fd}#onetrust-pc-sdk .category-vendors-list-handler+a::after{content:\"\";height:15px;width:15px;background-repeat:no-repeat;margin-left:5px;float:right;background-image:url(\"data:image/svg+xml,%3Csvg xmlns='http://www.w3.org/2000/svg' viewBox='0 0 511.626 511.627'%3E%3Cg fill='%231276CE'%3E%3Cpath d='M392.857 292.354h-18.274c-2.669 0-4.859.855-6.563 2.573-1.718 1.708-2.573 3.897-2.573 6.563v91.361c0 12.563-4.47 23.315-13.415 32.262-8.945 8.945-19.701 13.414-32.264 13.414H82.224c-12.562 0-23.317-4.469-32.264-13.414-8.945-8.946-13.417-19.698-13.417-32.262V155.31c0-12.562 4.471-23.313 13.417-32.259 8.947-8.947 19.702-13.418 32.264-13.418h200.994c2.669 0 4.859-.859 6.57-2.57 1.711-1.713 2.566-3.9 2.566-6.567V82.221c0-2.662-.855-4.853-2.566-6.563-1.711-1.713-3.901-2.568-6.57-2.568H82.224c-22.648 0-42.016 8.042-58.102 24.125C8.042 113.297 0 132.665 0 155.313v237.542c0 22.647 8.042 42.018 24.123 58.095 16.086 16.084 35.454 24.13 58.102 24.13h237.543c22.647 0 42.017-8.046 58.101-24.13 16.085-16.077 24.127-35.447 24.127-58.095v-91.358c0-2.669-.856-4.859-2.574-6.57-1.713-1.718-3.903-2.573-6.565-2.573z'/%3E%3Cpath d='M506.199 41.971c-3.617-3.617-7.905-5.424-12.85-5.424H347.171c-4.948 0-9.233 1.807-12.847 5.424-3.617 3.615-5.428 7.898-5.428 12.847s1.811 9.233 5.428 12.85l50.247 50.248-186.147 186.151c-1.906 1.903-2.856 4.093-2.856 6.563 0 2.479.953 4.668 2.856 6.571l32.548 32.544c1.903 1.903 4.093 2.852 6.567 2.852s4.665-.948 6.567-2.852l186.148-186.148 50.251 50.248c3.614 3.617 7.898 5.426 12.847 5.426s9.233-1.809 12.851-5.426c3.617-3.616 5.424-7.898 5.424-12.847V54.818c-.001-4.952-1.814-9.232-5.428-12.847z'/%3E%3C/g%3E%3C/svg%3E\")}#onetrust-pc-sdk .category-host-list-handler,#onetrust-pc-sdk .ot-vlst-cntr,#onetrust-pc-sdk #ot-pc-desc+.category-vendors-list-handler{margin-top:8px}#onetrust-pc-sdk .ot-grp-hdr1+.ot-vlst-cntr{margin-top:0px;margin-bottom:10px}#onetrust-pc-sdk .ot-always-active-group h3.ot-cat-header,#onetrust-pc-sdk .ot-subgrp.ot-always-active-group>h4{max-width:70%}#onetrust-pc-sdk .ot-always-active-group .ot-tgl-cntr{max-width:28%}#onetrust-pc-sdk .ot-grp-desc ul,#onetrust-pc-sdk li.ot-subgrp p ul{margin:0px;margin-left:15px;padding-bottom:8px}#onetrust-pc-sdk .ot-grp-desc ul li,#onetrust-pc-sdk li.ot-subgrp p ul li{font-size:inherit;padding-top:8px;display:list-item;list-style:disc}#onetrust-pc-sdk ul.ot-subgrps{margin:0;font-size:inherit}#onetrust-pc-sdk ul.ot-subgrps li{padding:0;border:none;position:relative}#onetrust-pc-sdk ul.ot-subgrps li h4,#onetrust-pc-sdk ul.ot-subgrps li p{font-size:.82em;line-height:1.4}#onetrust-pc-sdk ul.ot-subgrps li p{color:dimgray;clear:both;float:left;margin-top:10px;margin-bottom:0;word-break:break-word;word-wrap:break-word}#onetrust-pc-sdk ul.ot-subgrps li h4{color:#263238;font-weight:bold;margin-bottom:0;float:left;position:relative;top:3px}#onetrust-pc-sdk li.ot-subgrp{margin-left:30px;display:inline-block;width:calc(100% - 30px)}#onetrust-pc-sdk .ot-subgrp-tgl{float:right}#onetrust-pc-sdk .ot-subgrp-tgl.ot-always-active-subgroup{width:auto}#onetrust-pc-sdk .ot-pc-footer-logo{height:30px;width:100%;text-align:right;background:#f4f4f4;border-radius:0 0 2px 2px}#onetrust-pc-sdk .ot-pc-footer-logo a{display:inline-block;margin-top:5px;margin-right:10px}#onetrust-pc-sdk #accept-recommended-btn-handler{float:right;text-align:center}#onetrust-pc-sdk .save-preference-btn-handler{min-width:155px;background-color:#68b631;border-radius:2px;color:#fff;font-size:.9em;line-height:1.1;text-align:center;margin-left:15px;margin-right:15px}#onetrust-pc-sdk .ot-btn-subcntr button{margin-right:16px}#onetrust-pc-sdk.ot-ftr-stacked .save-preference-btn-handler,#onetrust-pc-sdk.ot-ftr-stacked .ot-btn-subcntr{max-width:40%;white-space:normal;text-align:center}#onetrust-pc-sdk.ot-ftr-stacked .ot-btn-subcntr button{margin-left:auto;margin-right:auto;min-width:60%;max-width:90%}#onetrust-pc-sdk.ot-ftr-stacked .ot-btn-subcntr button:nth-child(2){margin-top:0}#onetrust-pc-sdk.ot-ftr-stacked #accept-recommended-btn-handler{float:none}#onetrust-pc-sdk.ot-ftr-stacked .ot-btn-container{overflow:hidden}#onetrust-pc-sdk #ot-pc-title{float:left;margin-left:10px;max-width:85%;overflow:hidden;position:relative;line-height:1.2;max-height:2.4em;padding-right:1em;font-size:1.37em}#onetrust-pc-sdk #ot-pc-title:before{content:\"...\";right:0px;bottom:0px;position:absolute}#onetrust-pc-sdk #ot-pc-title:after{position:absolute;content:\"\";width:1em;height:1em;right:0px;background:#fff}#onetrust-pc-sdk #ot-pc-lst{width:100%;position:relative}#onetrust-pc-sdk #ot-pc-lst .ot-acc-hdr{padding-top:17px;padding-right:15px;padding-bottom:17px;padding-left:20px;display:inline-block;width:calc(100% - 35px);vertical-align:middle}#onetrust-pc-sdk #ot-pc-lst .ot-acc-txt{padding-top:6px;padding-right:15px;padding-bottom:10px;padding-left:20px}#onetrust-pc-sdk .ot-lst-cntr{height:100%}#onetrust-pc-sdk #ot-pc-hdr{padding-top:15px;padding-right:30px;padding-bottom:15px;padding-left:20px;display:inline-block;width:calc(100% - 50px);height:20px;border-bottom:1px solid #d8d8d8}#onetrust-pc-sdk #ot-pc-hdr input{height:32px;width:100%;border-radius:50px;font-size:.8em;padding-right:35px;padding-left:15px;float:left}#onetrust-pc-sdk #ot-pc-hdr input::placeholder{color:#d4d4d4;font-style:italic}#onetrust-pc-sdk #ot-lst-cnt{height:calc(100% - 86px);padding-left:30px;padding-right:27px;padding-top:20px;margin-top:8px;margin-right:3px;margin-bottom:4px;margin-left:0;overflow-x:hidden;overflow-y:auto;transform:translate3d(0, 0, 0)}#onetrust-pc-sdk #ot-back-arw{height:12px;width:12px}#onetrust-pc-sdk #ot-lst-title{display:inline-block;font-size:1em}#onetrust-pc-sdk #ot-lst-title span{color:dimgray;font-weight:bold;margin-left:10px}#onetrust-pc-sdk #ot-lst-title span *{font-size:inherit}#onetrust-pc-sdk .ot-lst-subhdr{float:right;position:relative;bottom:6px}#onetrust-pc-sdk #ot-search-cntr{display:inline-block;vertical-align:middle;position:relative;width:300px}#onetrust-pc-sdk #ot-search-cntr svg{position:absolute;right:0px;width:30px;height:30px;font-size:1em;line-height:1;top:2px}#onetrust-pc-sdk #ot-fltr-cntr{display:inline-block;position:relative;margin-left:20px;vertical-align:middle;font-size:0}#onetrust-pc-sdk #filter-btn-handler{background-color:#3860be;border-radius:17px;-moz-transition:.1s ease;-o-transition:.1s ease;-webkit-transition:1s ease;transition:.1s ease;width:32px;height:32px;padding:0;margin:0;position:relative}#onetrust-pc-sdk #filter-btn-handler svg{cursor:pointer;width:15px;height:15px;position:absolute;left:50%;top:50%;transform:translate(-50%, -50%);padding-top:5px}#onetrust-pc-sdk #filter-btn-handler path{fill:#fff}#onetrust-pc-sdk #ot-sel-blk{min-width:200px;min-height:30px;padding-left:20px}#onetrust-pc-sdk #ot-selall-vencntr,#onetrust-pc-sdk #ot-selall-adtlvencntr{float:left;height:100%}#onetrust-pc-sdk #ot-selall-vencntr label,#onetrust-pc-sdk #ot-selall-adtlvencntr label{height:100%;padding-left:0}#onetrust-pc-sdk #ot-selall-hostcntr{width:21px;height:21px;position:relative;left:20px}#onetrust-pc-sdk #ot-selall-vencntr.line-through label::after,#onetrust-pc-sdk #ot-selall-adtlvencntr.line-through label::after,#onetrust-pc-sdk #ot-selall-licntr.line-through label::after,#onetrust-pc-sdk #ot-selall-hostcntr.line-through label::after,#onetrust-pc-sdk #ot-selall-gnvencntr.line-through label::after{height:auto;border-left:0;left:5px;top:10.5px;transform:none;-o-transform:none;-ms-transform:none;-webkit-transform:none}#onetrust-pc-sdk .ot-ven-name,#onetrust-pc-sdk .ot-host-name{color:#2c3643;font-weight:bold;font-size:.813em;line-height:1.2;margin:0;height:auto;text-align:left;word-break:break-word;word-wrap:break-word}#onetrust-pc-sdk .ot-ven-name *,#onetrust-pc-sdk .ot-host-name *{font-size:inherit}#onetrust-pc-sdk .ot-host-desc{font-size:.69em;line-height:1.4;margin-top:5px;margin-bottom:5px;float:left;color:dimgray}#onetrust-pc-sdk .ot-host-name>a{text-decoration:underline;position:relative;z-index:2;float:left;margin-bottom:5px;font-weight:bold}#onetrust-pc-sdk .ot-host-hdr .ot-host-name+a{margin-top:5px;font-size:.813em;text-decoration:underline}#onetrust-pc-sdk .ot-ven-hdr{width:88%;float:right}#onetrust-pc-sdk input:focus+.ot-acc-hdr{outline:#000 solid 1px !important}#onetrust-pc-sdk #ot-selall-hostcntr input[type=checkbox],#onetrust-pc-sdk #ot-selall-vencntr input[type=checkbox],#onetrust-pc-sdk #ot-selall-adtlvencntr input[type=checkbox]{position:absolute}#onetrust-pc-sdk .ot-host-item .ot-chkbox{float:left}#onetrust-pc-sdk.ot-addtl-vendors #ot-lst-cnt:not(.ot-host-cnt) .ot-sel-all-hdr{right:38px}#onetrust-pc-sdk.ot-addtl-vendors #ot-lst-cnt:not(.ot-host-cnt) #ot-sel-blk{background-color:#f9f9fc;border:1px solid #e2e2e2;width:auto;padding-bottom:5px;padding-top:5px}#onetrust-pc-sdk.ot-addtl-vendors #ot-lst-cnt:not(.ot-host-cnt) .ot-sel-all-chkbox{right:2px;width:auto}#onetrust-pc-sdk #ot-pc-lst .ot-acc-cntr{position:relative;border-left:1px solid #e2e2e2;border-right:1px solid #e2e2e2;border-bottom:1px solid #e2e2e2}#onetrust-pc-sdk #ot-pc-lst .ot-acc-cntr input{z-index:1}#onetrust-pc-sdk #ot-pc-lst .ot-acc-cntr>.ot-acc-hdr{background:#f9f9fc;padding-top:10px;padding-bottom:10px;background-color:#f9f9fc}#onetrust-pc-sdk #ot-pc-lst .ot-acc-cntr>.ot-acc-hdr input{z-index:2}#onetrust-pc-sdk #ot-pc-lst .ot-acc-cntr>input[type=checkbox]:checked~.ot-acc-hdr{border-bottom:1px solid #e2e2e2}#onetrust-pc-sdk #ot-pc-lst .ot-acc-cntr .ot-addtl-venbox{display:none}#onetrust-pc-sdk #ot-addtl-venlst .ot-tgl-cntr{margin-right:13px}#onetrust-pc-sdk .ot-vensec-title{font-size:.813em;display:inline-block}#onetrust-pc-sdk .ot-ven-item>button:focus,#onetrust-pc-sdk .ot-host-item>button:focus,#onetrust-pc-sdk .ot-acc-cntr>button:focus{outline:#000 solid 2px}#onetrust-pc-sdk .ot-ven-item>button,#onetrust-pc-sdk .ot-host-item>button,#onetrust-pc-sdk .ot-acc-cntr>button{position:absolute;cursor:pointer;width:100%;height:100%;border:0;opacity:0;margin:0;top:0;left:0}#onetrust-pc-sdk .ot-ven-item>button~.ot-acc-hdr,#onetrust-pc-sdk .ot-host-item>button~.ot-acc-hdr,#onetrust-pc-sdk .ot-acc-cntr>button~.ot-acc-hdr{cursor:pointer}#onetrust-pc-sdk .ot-ven-item>button[aria-expanded=false]~.ot-acc-txt,#onetrust-pc-sdk .ot-host-item>button[aria-expanded=false]~.ot-acc-txt,#onetrust-pc-sdk .ot-acc-cntr>button[aria-expanded=false]~.ot-acc-txt{margin-top:0;max-height:0;opacity:0;overflow:hidden;width:100%;transition:.25s ease-out;display:none}#onetrust-pc-sdk .ot-ven-item>button[aria-expanded=true]~.ot-acc-txt,#onetrust-pc-sdk .ot-host-item>button[aria-expanded=true]~.ot-acc-txt,#onetrust-pc-sdk .ot-acc-cntr>button[aria-expanded=true]~.ot-acc-txt{transition:.1s ease-in;display:block}#onetrust-pc-sdk .ot-hide-acc>button{pointer-events:none}#onetrust-pc-sdk .ot-hide-acc .ot-arw-cntr>*{visibility:hidden}#onetrust-pc-sdk #ot-ven-lst,#onetrust-pc-sdk #ot-host-lst,#onetrust-pc-sdk #ot-addtl-venlst,#onetrust-pc-sdk #ot-gn-venlst{width:100%}#onetrust-pc-sdk #ot-ven-lst li,#onetrust-pc-sdk #ot-host-lst li,#onetrust-pc-sdk #ot-addtl-venlst li,#onetrust-pc-sdk #ot-gn-venlst li{border:1px solid #d7d7d7;border-radius:2px;position:relative;margin-top:10px}#onetrust-pc-sdk #ot-ven-lst .ot-tgl-cntr{width:65%}#onetrust-pc-sdk #ot-host-lst .ot-tgl-cntr{width:65%;float:left}#onetrust-pc-sdk label{margin-bottom:0}#onetrust-pc-sdk .ot-host-notice{float:right}#onetrust-pc-sdk .ot-ven-link,#onetrust-pc-sdk .ot-host-expand{color:dimgray;font-size:.75em;line-height:.9;display:inline-block}#onetrust-pc-sdk .ot-ven-link *,#onetrust-pc-sdk .ot-host-expand *{font-size:inherit}#onetrust-pc-sdk .ot-ven-link{position:relative;z-index:2}#onetrust-pc-sdk .ot-ven-link:hover{text-decoration:underline}#onetrust-pc-sdk .ot-ven-dets{border-radius:2px;background-color:#f8f8f8}#onetrust-pc-sdk .ot-ven-dets div:first-child p:first-child{border-top:none}#onetrust-pc-sdk .ot-ven-dets .ot-ven-disc:not(:first-child){border-top:1px solid #e9e9e9}#onetrust-pc-sdk .ot-ven-dets .ot-ven-disc:nth-child(n+3) p{display:inline-block}#onetrust-pc-sdk .ot-ven-dets .ot-ven-disc:nth-child(n+3) p:nth-of-type(odd){width:30%}#onetrust-pc-sdk .ot-ven-dets .ot-ven-disc:nth-child(n+3) p:nth-of-type(even){width:50%;word-break:break-word;word-wrap:break-word}#onetrust-pc-sdk .ot-ven-dets .ot-ven-disc p{padding-top:5px;padding-bottom:5px;display:block}#onetrust-pc-sdk .ot-ven-dets p{font-size:.69em;text-align:left;vertical-align:middle;word-break:break-word;word-wrap:break-word;margin:0;padding-bottom:10px;padding-left:15px;color:#2e3644}#onetrust-pc-sdk .ot-ven-dets p:first-child{padding-top:5px}#onetrust-pc-sdk .ot-ven-dets .ot-ven-pur p:first-child{border-top:1px solid #e9e9e9;border-bottom:1px solid #e9e9e9;padding-bottom:5px;margin-bottom:5px;font-weight:bold}#onetrust-pc-sdk .ot-host-name{float:left;width:calc(100% - 50px)}#onetrust-pc-sdk .ot-host-opt{display:inline-block;width:100%;margin:0;font-size:inherit}#onetrust-pc-sdk .ot-host-opt li>div div{font-size:.81em;padding:5px 0}#onetrust-pc-sdk .ot-host-opt li>div div:nth-child(1){width:30%;float:left}#onetrust-pc-sdk .ot-host-opt li>div div:nth-child(2){width:70%;float:left;word-break:break-word;word-wrap:break-word}#onetrust-pc-sdk #ot-host-lst li.ot-host-info{border:none;font-size:.8em;color:dimgray;float:left;text-align:left;padding:10px;margin-bottom:10px;width:calc(100% - 10px);background-color:#f8f8f8}#onetrust-pc-sdk #ot-host-lst li.ot-host-info a{color:dimgray}#onetrust-pc-sdk #no-results{text-align:center;margin-top:30px}#onetrust-pc-sdk #no-results p{font-size:1em;color:#2e3644;word-break:break-word;word-wrap:break-word}#onetrust-pc-sdk #no-results p span{font-weight:bold}#onetrust-pc-sdk .ot-tgl-cntr{display:inline-block;vertical-align:middle}#onetrust-pc-sdk .ot-arw-cntr,#onetrust-pc-sdk .ot-tgl-cntr{float:right}#onetrust-pc-sdk .ot-desc-cntr{padding-top:0px;margin-top:20px;padding-right:0px;border-radius:3px;overflow:hidden;padding-bottom:10px}#onetrust-pc-sdk .ot-leg-border-color{border:1px solid #e9e9e9}#onetrust-pc-sdk .ot-leg-border-color .ot-subgrp-cntr{border-top:1px solid #e9e9e9;padding-bottom:10px}#onetrust-pc-sdk .ot-category-desc{padding-bottom:10px}#onetrust-pc-sdk .ot-grp-hdr1{padding-left:10px;width:calc(100% - 20px);padding-top:10px;margin-bottom:0px;padding-bottom:8px}#onetrust-pc-sdk .ot-subgrp-cntr{padding-top:10px}#onetrust-pc-sdk .ot-desc-cntr>*:not(.ot-grp-hdr1){padding-left:10px;padding-right:10px}#onetrust-pc-sdk .ot-pli-hdr{overflow:hidden;padding-top:7.5px;padding-bottom:7.5px;background-color:#f8f8f8;border:none;border-bottom:1px solid #e9e9e9}#onetrust-pc-sdk .ot-pli-hdr span:first-child{text-align:left;max-width:80px;padding-right:5px}#onetrust-pc-sdk .ot-pli-hdr span:last-child{padding-right:20px;text-align:center}#onetrust-pc-sdk .ot-li-title{float:right;font-size:.813em}#onetrust-pc-sdk .ot-desc-cntr .ot-tgl-cntr:first-of-type,#onetrust-pc-sdk .ot-cat-header+.ot-tgl{padding-left:55px;padding-right:7px}#onetrust-pc-sdk .ot-always-active-group .ot-grp-hdr1 .ot-tgl-cntr:first-of-type{padding-left:0px}#onetrust-pc-sdk .ot-cat-header,#onetrust-pc-sdk .ot-subgrp h4{max-width:calc(100% - 133px)}#onetrust-pc-sdk #ot-lst-cnt #ot-sel-blk{width:100%;display:inline-block;padding:0}#onetrust-pc-sdk .ot-sel-all{display:inline-block;width:100%}#onetrust-pc-sdk .ot-sel-all-hdr,#onetrust-pc-sdk .ot-sel-all-chkbox{width:100%;float:right;position:relative}#onetrust-pc-sdk .ot-sel-all-chkbox{z-index:1}#onetrust-pc-sdk :not(.ot-hosts-ui) .ot-sel-all-hdr,#onetrust-pc-sdk :not(.ot-hosts-ui) .ot-sel-all-chkbox{right:23px;width:calc(100% - 23px)}#onetrust-pc-sdk .ot-consent-hdr,#onetrust-pc-sdk .ot-li-hdr{float:right;font-size:.813em;position:relative;line-height:normal;text-align:center;word-break:break-word;word-wrap:break-word}#onetrust-pc-sdk .ot-hosts-ui .ot-consent-hdr{float:left;position:relative;left:5px}#onetrust-pc-sdk .ot-li-hdr{max-width:100px;margin-right:10px}#onetrust-pc-sdk .ot-consent-hdr{max-width:55px}#onetrust-pc-sdk .ot-ven-ctgl{margin-left:10px}#onetrust-pc-sdk .ot-ven-litgl{margin-right:55px}#onetrust-pc-sdk .ot-ven-litgl.ot-ven-litgl-only{margin-right:86px}#onetrust-pc-sdk .ot-ven-ctgl,#onetrust-pc-sdk .ot-ven-litgl,#onetrust-pc-sdk .ot-ven-gvctgl{float:left}#onetrust-pc-sdk .ot-ven-ctgl label,#onetrust-pc-sdk .ot-ven-litgl label,#onetrust-pc-sdk .ot-ven-gvctgl label{width:22px;padding:0}#onetrust-pc-sdk #ot-selall-licntr{display:block;width:21px;height:21px;position:relative;float:right;right:80px}#onetrust-pc-sdk #ot-selall-licntr input{position:absolute}#onetrust-pc-sdk #ot-selall-vencntr,#onetrust-pc-sdk #ot-selall-adtlvencntr,#onetrust-pc-sdk #ot-selall-gnvencntr{float:right;width:21px;height:21px;position:relative;right:15px}#onetrust-pc-sdk #ot-ven-lst .ot-tgl-cntr{float:right;width:auto}#onetrust-pc-sdk .ot-ven-hdr{float:left;width:60%}#onetrust-pc-sdk #vdr-lst-dsc{font-size:.812em;line-height:1.5;padding:10px 15px 5px 15px}#onetrust-pc-sdk #ot-anchor{border:12px solid transparent;display:none;position:absolute;z-index:2147483647;top:40px;right:35px;transform:rotate(45deg);-o-transform:rotate(45deg);-ms-transform:rotate(45deg);-webkit-transform:rotate(45deg);background-color:#fff;-webkit-box-shadow:-3px -3px 5px -2px #c7c5c7;-moz-box-shadow:-3px -3px 5px -2px #c7c5c7;box-shadow:-3px -3px 5px -2px #c7c5c7}#onetrust-pc-sdk #ot-fltr-modal{width:300px;position:absolute;z-index:2147483646;top:46px;height:90%;max-height:350px;display:none;-moz-transition:.2s ease;-o-transition:.2s ease;-webkit-transition:2s ease;transition:.2s ease;opacity:1;right:0}#onetrust-pc-sdk #ot-fltr-modal button{max-width:200px;line-height:1;word-break:break-word;white-space:normal;height:auto;font-weight:bold}#onetrust-pc-sdk #ot-fltr-cnt{background-color:#fff;margin:5px;border-radius:3px;height:100%;margin-right:10px;padding-right:10px;-webkit-box-shadow:0px 0px 12px 2px #c7c5c7;-moz-box-shadow:0px 0px 12px 2px #c7c5c7;box-shadow:0px 0px 12px 2px #c7c5c7}#onetrust-pc-sdk .ot-fltr-scrlcnt{overflow-y:auto;overflow-x:hidden;clear:both;max-height:calc(100% - 60px)}#onetrust-pc-sdk .ot-fltr-opt{margin-bottom:25px;margin-left:15px;clear:both}#onetrust-pc-sdk .ot-fltr-opt label{height:auto}#onetrust-pc-sdk .ot-fltr-opt span{cursor:pointer;color:dimgray;font-size:.8em;line-height:1.1;font-weight:normal}#onetrust-pc-sdk #clear-filters-handler{float:right;margin-top:15px;margin-bottom:10px;text-decoration:none;color:#3860be;font-size:.9em;border:none;padding:1px}#onetrust-pc-sdk #clear-filters-handler:hover{color:#1883fd}#onetrust-pc-sdk #clear-filters-handler:focus{outline:#000 solid 1px}#onetrust-pc-sdk #filter-apply-handler{margin-right:10px}#onetrust-pc-sdk .ot-grp-desc+.ot-leg-btn-container{margin-top:0}#onetrust-pc-sdk .ot-leg-btn-container{display:inline-block;width:100%;margin-top:10px}#onetrust-pc-sdk .ot-leg-btn-container button{height:auto;padding:6.5px 8px;margin-bottom:0;line-height:normal;letter-spacing:0}#onetrust-pc-sdk .ot-leg-btn-container svg{display:none;height:14px;width:14px;padding-right:5px;vertical-align:sub}#onetrust-pc-sdk .ot-active-leg-btn{cursor:default;pointer-events:none}#onetrust-pc-sdk .ot-active-leg-btn svg{display:inline-block}#onetrust-pc-sdk .ot-remove-objection-handler{border:none;text-decoration:underline;padding:0;font-size:.82em;font-weight:600;line-height:1.4;padding-left:10px}#onetrust-pc-sdk .ot-obj-leg-btn-handler span{font-weight:bold;text-align:center;font-size:.91em;line-height:1.5}#onetrust-pc-sdk.otPcTab[dir=rtl] input~.ot-acc-hdr .ot-arw,#onetrust-pc-sdk.otPcTab[dir=rtl] #ot-back-arw{transform:rotate(180deg);-o-transform:rotate(180deg);-ms-transform:rotate(180deg);-webkit-transform:rotate(180deg)}#onetrust-pc-sdk.otPcTab[dir=rtl] input:checked~.ot-acc-hdr .ot-arw{transform:rotate(270deg);-o-transform:rotate(270deg);-ms-transform:rotate(270deg);-webkit-transform:rotate(270deg)}#onetrust-pc-sdk.otPcTab[dir=rtl] #ot-search-cntr svg{right:15px}#onetrust-pc-sdk.otPcTab[dir=rtl] .ot-chkbox label::after{transform:rotate(45deg);-webkit-transform:rotate(45deg);-o-transform:rotate(45deg);-ms-transform:rotate(45deg);border-left:0;border-right:3px solid}#onetrust-pc-sdk #close-pc-btn-handler.ot-close-icon{padding:0;background-color:transparent;border:none;margin:0}@media(max-width: 767px){#onetrust-pc-sdk{width:100%;border:none}#onetrust-pc-sdk .ot-sdk-container,#onetrust-pc-sdk .ot-sdk-container{padding:0;margin:0}#onetrust-pc-sdk #ot-pc-title{margin-left:10px;max-width:60%}#onetrust-pc-sdk .ot-desc-cntr{margin:0;padding-top:20px;padding-right:20px;padding-bottom:15px;padding-left:20px;position:relative;left:auto}#onetrust-pc-sdk .ot-desc-cntr{margin-top:20px;margin-left:20px;padding:0;padding-bottom:10px}#onetrust-pc-sdk .ot-grps-cntr{max-height:none;overflow:hidden}#onetrust-pc-sdk #accept-recommended-btn-handler{float:none}}@media(min-width: 768px){#onetrust-pc-sdk.ot-tgl-with-label .ot-label-status{display:inline}#onetrust-pc-sdk.ot-tgl-with-label #ot-pc-lst .ot-label-status{display:none}#onetrust-pc-sdk.ot-tgl-with-label.ot-leg-opt-out .ot-pli-hdr{padding-right:8%}#onetrust-pc-sdk.ot-tgl-with-label .ot-cat-header{max-width:60%}#onetrust-pc-sdk.ot-tgl-with-label .ot-subgrp h4{max-width:58%}#onetrust-pc-sdk.ot-tgl-with-label .ot-desc-cntr .ot-tgl-cntr:first-of-type,#onetrust-pc-sdk.ot-tgl-with-label .ot-cat-header+.ot-tgl{padding-left:15px}}@media(max-width: 640px){#onetrust-pc-sdk{height:100%}#onetrust-pc-sdk .ot-pc-header{padding:10px;width:calc(100% - 20px)}#onetrust-pc-sdk #ot-pc-content{overflow:auto}#onetrust-pc-sdk .ot-sdk-row .ot-sdk-columns{width:100%}#onetrust-pc-sdk .ot-desc-cntr{margin:0;overflow:hidden}#onetrust-pc-sdk .ot-desc-cntr{margin-left:10px;width:calc(100% - 15px);margin-top:5px;margin-bottom:5px}#onetrust-pc-sdk .ot-ven-hdr{max-width:80%}#onetrust-pc-sdk #ot-lst-cnt{width:calc(100% - 18px);padding-top:13px;padding-right:5px;padding-left:10px}#onetrust-pc-sdk .ot-grps-cntr{width:100%}#onetrust-pc-sdk .ot-pc-footer{max-height:300px}#onetrust-pc-sdk #ot-pc-content,#onetrust-pc-sdk #ot-pc-lst{height:calc(100% - 322px)}}@media(max-width: 640px)and (orientation: portrait){#onetrust-pc-sdk #ot-pc-hdr{height:70px;padding:15px 0;width:100%}#onetrust-pc-sdk .ot-lst-subhdr{width:calc(100% - 15px);float:none;bottom:auto;display:inline-block;padding-top:8px;padding-left:15px}#onetrust-pc-sdk .ot-btn-subcntr{float:none}#onetrust-pc-sdk #ot-search-cntr{display:inline-block;width:calc(100% - 55px);position:relative}#onetrust-pc-sdk #ot-anchor{top:75px;right:30px}#onetrust-pc-sdk #ot-fltr-modal{top:81px}#onetrust-pc-sdk #ot-fltr-cntr{float:right;right:15px}#onetrust-pc-sdk #ot-lst-title{padding-left:15px}#onetrust-pc-sdk .ot-lst-cntr{overflow-y:scroll}#onetrust-pc-sdk #ot-lst-cnt{height:auto;overflow:hidden}#onetrust-pc-sdk .save-preference-btn-handler,#onetrust-pc-sdk #accept-recommended-btn-handler,#onetrust-pc-sdk .ot-pc-refuse-all-handler{width:calc(100% - 33px)}#onetrust-pc-sdk.ot-ftr-stacked .save-preference-btn-handler,#onetrust-pc-sdk.ot-ftr-stacked .ot-btn-subcntr{max-width:none}#onetrust-pc-sdk.ot-ftr-stacked .ot-pc-footer button{margin:15px}#onetrust-pc-sdk.ot-ftr-stacked .ot-btn-subcntr button{min-width:none;max-width:none}#onetrust-pc-sdk.ot-ftr-stacked .ot-btn-subcntr button:nth-child(2){margin-top:15px}#onetrust-pc-sdk.ot-ftr-stacked .ot-btn-container button:not(:last-child){margin-bottom:0}}@media(max-width: 425px){#onetrust-pc-sdk #ot-pc-lst .ot-acc-txt{padding-top:6px;padding-bottom:10px}#onetrust-pc-sdk #ot-pc-lst .ot-host-notice{float:left;margin-left:30px}#onetrust-pc-sdk #ot-pc-lst .ot-arw-cntr{float:none;display:inline}#onetrust-pc-sdk #ot-pc-lst .ot-ven-hdr{float:left;width:100%;max-width:85%}#onetrust-pc-sdk.ot-addtl-vendors #ot-pc-lst .ot-acc-cntr .ot-arw-cntr:first-of-type{float:right}#onetrust-pc-sdk #ot-pc-title{max-width:100%}#onetrust-pc-sdk .ot-subgrp-cntr li.ot-subgrp{margin-left:10px;width:calc(100% - 10px)}#onetrust-pc-sdk #ot-ven-lst .ot-tgl-cntr{width:auto;float:right}#onetrust-pc-sdk #ot-ven-lst .ot-arw-cntr{float:right}#onetrust-pc-sdk .ot-ven-hdr{max-width:47%}#onetrust-pc-sdk .ot-always-active-group .ot-tgl-cntr:first-of-type{max-width:none;padding-left:20px}}@media only screen and (max-height: 425px)and (max-width: 896px)and (orientation: landscape){#onetrust-pc-sdk{height:100%;width:100%;max-width:none}#onetrust-pc-sdk .ot-always-active-group .ot-tgl-cntr{max-width:none}#onetrust-pc-sdk .ot-pc-header{padding:10px;width:calc(100% - 20px)}#onetrust-pc-sdk .ot-lst-cntr{overflow-y:scroll}#onetrust-pc-sdk #ot-lst-cnt{height:auto;overflow:hidden}#onetrust-pc-sdk #accept-recommended-btn-handler{float:right}#onetrust-pc-sdk .save-preference-btn-handler,#onetrust-pc-sdk #accept-recommended-btn-handler,#onetrust-pc-sdk .ot-pc-refuse-all-handler{width:auto}#onetrust-pc-sdk #ot-pc-content,#onetrust-pc-sdk #ot-pc-lst{height:calc(100% - 155px)}#onetrust-pc-sdk .ot-pc-footer-logo{display:none}#onetrust-pc-sdk.ot-shw-fltr .ot-lst-cntr{overflow:hidden}#onetrust-pc-sdk.ot-shw-fltr #ot-pc-lst{position:static}#onetrust-pc-sdk.ot-shw-fltr #ot-fltr-modal{top:0;width:100%;height:100%;max-height:none}#onetrust-pc-sdk.ot-shw-fltr #ot-fltr-modal>div{margin:0;box-sizing:initial;height:100%;max-height:none}#onetrust-pc-sdk.ot-shw-fltr #clear-filters-handler{padding-right:20px}#onetrust-pc-sdk.ot-shw-fltr #ot-anchor{display:none !important}#onetrust-pc-sdk .ot-pc-footer button{margin:10px}}@media(max-width: 425px),(max-width: 896px)and (max-height: 425px)and (orientation: landscape){#onetrust-pc-sdk .ot-pc-header{padding-right:20px}#onetrust-pc-sdk .ot-pc-logo{margin-left:0px;margin-top:5px;width:150px}#onetrust-pc-sdk .ot-close-icon{width:44px;height:44px;background-size:12px}#onetrust-pc-sdk .ot-grp-hdr1{float:right;margin-left:10px;width:auto}#onetrust-pc-sdk .ot-grp-hdr1{margin-left:0px;padding-right:10px}#onetrust-pc-sdk #ot-pvcy-hdr,#onetrust-pc-sdk .ot-grp-hdr1 .ot-cat-header{display:none}#onetrust-pc-sdk .ot-grp-hdr1+.ot-vlst-cntr{padding-top:10px}}@media only screen and (max-height: 610px){#onetrust-pc-sdk{max-height:100%}}\n            #onetrust-consent-sdk #onetrust-pc-sdk,\n                #onetrust-consent-sdk #ot-search-cntr,\n                #onetrust-consent-sdk #onetrust-pc-sdk .ot-switch.ot-toggle,\n                #onetrust-consent-sdk #onetrust-pc-sdk ot-grp-hdr1 .checkbox,\n                #onetrust-consent-sdk #onetrust-pc-sdk #ot-pc-title:after\n                ,#onetrust-consent-sdk #onetrust-pc-sdk #ot-sel-blk,\n                        #onetrust-consent-sdk #onetrust-pc-sdk #ot-fltr-cnt,\n                        #onetrust-consent-sdk #onetrust-pc-sdk #ot-anchor {\n                    background-color: #FFFFFF;\n                }\n               \n            #onetrust-consent-sdk #onetrust-pc-sdk h3,\n                #onetrust-consent-sdk #onetrust-pc-sdk h4,\n                #onetrust-consent-sdk #onetrust-pc-sdk h5,\n                #onetrust-consent-sdk #onetrust-pc-sdk h6,\n                #onetrust-consent-sdk #onetrust-pc-sdk p,\n                #onetrust-consent-sdk #onetrust-pc-sdk #ot-ven-lst .ot-ven-opts p,\n                #onetrust-consent-sdk #onetrust-pc-sdk #ot-pc-desc,\n                #onetrust-consent-sdk #onetrust-pc-sdk #ot-pc-title,\n                #onetrust-consent-sdk #onetrust-pc-sdk .ot-li-title,\n                #onetrust-consent-sdk #onetrust-pc-sdk .ot-sel-all-hdr span,\n                #onetrust-consent-sdk #onetrust-pc-sdk #ot-host-lst .ot-host-info,\n                #onetrust-consent-sdk #onetrust-pc-sdk #ot-fltr-modal #modal-header,\n                #onetrust-consent-sdk #onetrust-pc-sdk .ot-checkbox label span,\n                #onetrust-consent-sdk #onetrust-pc-sdk #ot-pc-lst #ot-sel-blk p,\n                #onetrust-consent-sdk #onetrust-pc-sdk #ot-pc-lst #ot-lst-title span,\n                #onetrust-consent-sdk #onetrust-pc-sdk #ot-pc-lst .back-btn-handler p,\n                #onetrust-consent-sdk #onetrust-pc-sdk #ot-pc-lst .ot-ven-name,\n                #onetrust-consent-sdk #onetrust-pc-sdk #ot-pc-lst #ot-ven-lst .consent-category,\n                #onetrust-consent-sdk #onetrust-pc-sdk .ot-leg-btn-container .ot-inactive-leg-btn,\n                #onetrust-consent-sdk #onetrust-pc-sdk .ot-label-status,\n                #onetrust-consent-sdk #onetrust-pc-sdk .ot-chkbox label span,\n                #onetrust-consent-sdk #onetrust-pc-sdk #clear-filters-handler \n                {\n                    color: #302c2c;\n                }\n             #onetrust-consent-sdk #onetrust-pc-sdk .privacy-notice-link,\n                    #onetrust-consent-sdk #onetrust-pc-sdk .category-vendors-list-handler,\n                    #onetrust-consent-sdk #onetrust-pc-sdk .category-vendors-list-handler + a,\n                    #onetrust-consent-sdk #onetrust-pc-sdk .category-host-list-handler,\n                    #onetrust-consent-sdk #onetrust-pc-sdk .ot-ven-link,\n                    #onetrust-consent-sdk #onetrust-pc-sdk #ot-host-lst .ot-host-name a,\n                    #onetrust-consent-sdk #onetrust-pc-sdk #ot-host-lst .ot-acc-hdr .ot-host-expand,\n                    #onetrust-consent-sdk #onetrust-pc-sdk #ot-host-lst .ot-host-info a\n                    {\n                        color: #027bc5;\n                    }\n             #onetrust-consent-sdk #onetrust-banner-sdk a[href],\n                     #onetrust-consent-sdk #onetrust-banner-sdk .ot-link-btn\n                        {\n                            color: #027bc5;\n                        }           \n            #onetrust-consent-sdk #onetrust-pc-sdk .category-vendors-list-handler:hover { opacity: .7;}\n            #onetrust-consent-sdk #onetrust-pc-sdk .ot-acc-grpcntr.ot-acc-txt,\n            #onetrust-consent-sdk #onetrust-pc-sdk .ot-acc-txt .ot-subgrp-tgl .ot-switch.ot-toggle\n             {\n                background-color: #c3c3ca;\n            }\n            \n             #onetrust-consent-sdk #onetrust-pc-sdk #ot-host-lst .ot-host-info,\n                    #onetrust-consent-sdk #onetrust-pc-sdk .ot-acc-txt .ot-ven-dets\n                            {\n                                background-color: #c3c3ca;\n                            }\n        #onetrust-consent-sdk #onetrust-pc-sdk \n            button:not(#clear-filters-handler):not(.ot-close-icon):not(#filter-btn-handler):not(.ot-remove-objection-handler):not(.ot-obj-leg-btn-handler):not([aria-expanded]):not(.ot-link-btn),\n            #onetrust-consent-sdk #onetrust-pc-sdk .ot-leg-btn-container .ot-active-leg-btn {\n                background-color: #027bc5;border-color: #027bc5;\n                color: #FFFFFF;\n            }\n            #onetrust-consent-sdk #onetrust-pc-sdk .ot-active-menu {\n                border-color: #027bc5;\n            }\n            \n            #onetrust-consent-sdk #onetrust-pc-sdk .ot-leg-btn-container .ot-remove-objection-handler{\n                background-color: transparent;\n                border:1px solid transparent;\n            }\n            #onetrust-consent-sdk #onetrust-pc-sdk .ot-leg-btn-container .ot-inactive-leg-btn {\n                background-color: #FFFFFF;\n                color: #78808E; border-color: #78808E;\n            }\n            #onetrust-consent-sdk #onetrust-pc-sdk .category-menu-switch-handler {\n                    background-color: #F4F4F4\n                }#onetrust-consent-sdk #onetrust-pc-sdk .ot-active-menu {\n                    background-color: #FFFFFF\n                }.ot-sdk-cookie-policy{font-family:inherit;font-size:16px}.ot-sdk-cookie-policy h3,.ot-sdk-cookie-policy h4,.ot-sdk-cookie-policy h6,.ot-sdk-cookie-policy p,.ot-sdk-cookie-policy li,.ot-sdk-cookie-policy a,.ot-sdk-cookie-policy th,.ot-sdk-cookie-policy #cookie-policy-description,.ot-sdk-cookie-policy .ot-sdk-cookie-policy-group,.ot-sdk-cookie-policy #cookie-policy-title{color:dimgray}.ot-sdk-cookie-policy #cookie-policy-description{margin-bottom:1em}.ot-sdk-cookie-policy h4{font-size:1.2em}.ot-sdk-cookie-policy h6{font-size:1em;margin-top:2em}.ot-sdk-cookie-policy th{min-width:75px}.ot-sdk-cookie-policy a,.ot-sdk-cookie-policy a:hover{background:#fff}.ot-sdk-cookie-policy thead{background-color:#f6f6f4;font-weight:bold}.ot-sdk-cookie-policy .ot-mobile-border{display:none}.ot-sdk-cookie-policy section{margin-bottom:2em}.ot-sdk-cookie-policy table{border-collapse:inherit}#ot-sdk-cookie-policy-v2.ot-sdk-cookie-policy{font-family:inherit;font-size:16px}#ot-sdk-cookie-policy-v2.ot-sdk-cookie-policy h3,#ot-sdk-cookie-policy-v2.ot-sdk-cookie-policy h4,#ot-sdk-cookie-policy-v2.ot-sdk-cookie-policy h6,#ot-sdk-cookie-policy-v2.ot-sdk-cookie-policy p,#ot-sdk-cookie-policy-v2.ot-sdk-cookie-policy li,#ot-sdk-cookie-policy-v2.ot-sdk-cookie-policy a,#ot-sdk-cookie-policy-v2.ot-sdk-cookie-policy th,#ot-sdk-cookie-policy-v2.ot-sdk-cookie-policy #cookie-policy-description,#ot-sdk-cookie-policy-v2.ot-sdk-cookie-policy .ot-sdk-cookie-policy-group,#ot-sdk-cookie-policy-v2.ot-sdk-cookie-policy #cookie-policy-title{color:dimgray}#ot-sdk-cookie-policy-v2.ot-sdk-cookie-policy #cookie-policy-description{margin-bottom:1em}#ot-sdk-cookie-policy-v2.ot-sdk-cookie-policy .ot-sdk-subgroup{margin-left:1.5em}#ot-sdk-cookie-policy-v2.ot-sdk-cookie-policy #cookie-policy-description,#ot-sdk-cookie-policy-v2.ot-sdk-cookie-policy .ot-sdk-cookie-policy-group-desc,#ot-sdk-cookie-policy-v2.ot-sdk-cookie-policy .ot-table-header,#ot-sdk-cookie-policy-v2.ot-sdk-cookie-policy a,#ot-sdk-cookie-policy-v2.ot-sdk-cookie-policy span,#ot-sdk-cookie-policy-v2.ot-sdk-cookie-policy td{font-size:.9em}#ot-sdk-cookie-policy-v2.ot-sdk-cookie-policy td span,#ot-sdk-cookie-policy-v2.ot-sdk-cookie-policy td a{font-size:inherit}#ot-sdk-cookie-policy-v2.ot-sdk-cookie-policy .ot-sdk-cookie-policy-group{font-size:1em;margin-bottom:.6em}#ot-sdk-cookie-policy-v2.ot-sdk-cookie-policy .ot-sdk-cookie-policy-title{margin-bottom:1.2em}#ot-sdk-cookie-policy-v2.ot-sdk-cookie-policy>section{margin-bottom:1em}#ot-sdk-cookie-policy-v2.ot-sdk-cookie-policy th{min-width:75px}#ot-sdk-cookie-policy-v2.ot-sdk-cookie-policy a,#ot-sdk-cookie-policy-v2.ot-sdk-cookie-policy a:hover{background:#fff}#ot-sdk-cookie-policy-v2.ot-sdk-cookie-policy thead{background-color:#f6f6f4;font-weight:bold}#ot-sdk-cookie-policy-v2.ot-sdk-cookie-policy .ot-mobile-border{display:none}#ot-sdk-cookie-policy-v2.ot-sdk-cookie-policy section{margin-bottom:2em}#ot-sdk-cookie-policy-v2.ot-sdk-cookie-policy .ot-sdk-subgroup ul li{list-style:disc;margin-left:1.5em}#ot-sdk-cookie-policy-v2.ot-sdk-cookie-policy .ot-sdk-subgroup ul li h4{display:inline-block}#ot-sdk-cookie-policy-v2.ot-sdk-cookie-policy table{border-collapse:inherit;margin:auto;border:1px solid #d7d7d7;border-radius:5px;border-spacing:initial;width:100%;overflow:hidden}#ot-sdk-cookie-policy-v2.ot-sdk-cookie-policy table th,#ot-sdk-cookie-policy-v2.ot-sdk-cookie-policy table td{border-bottom:1px solid #d7d7d7;border-right:1px solid #d7d7d7}#ot-sdk-cookie-policy-v2.ot-sdk-cookie-policy table tr:last-child td{border-bottom:0px}#ot-sdk-cookie-policy-v2.ot-sdk-cookie-policy table tr th:last-child,#ot-sdk-cookie-policy-v2.ot-sdk-cookie-policy table tr td:last-child{border-right:0px}#ot-sdk-cookie-policy-v2.ot-sdk-cookie-policy table .ot-host,#ot-sdk-cookie-policy-v2.ot-sdk-cookie-policy table .ot-cookies-type{width:25%}.ot-sdk-cookie-policy[dir=rtl]{text-align:left}#ot-sdk-cookie-policy h3{font-size:1.5em}@media only screen and (max-width: 530px){.ot-sdk-cookie-policy:not(#ot-sdk-cookie-policy-v2) table,.ot-sdk-cookie-policy:not(#ot-sdk-cookie-policy-v2) thead,.ot-sdk-cookie-policy:not(#ot-sdk-cookie-policy-v2) tbody,.ot-sdk-cookie-policy:not(#ot-sdk-cookie-policy-v2) th,.ot-sdk-cookie-policy:not(#ot-sdk-cookie-policy-v2) td,.ot-sdk-cookie-policy:not(#ot-sdk-cookie-policy-v2) tr{display:block}.ot-sdk-cookie-policy:not(#ot-sdk-cookie-policy-v2) thead tr{position:absolute;top:-9999px;left:-9999px}.ot-sdk-cookie-policy:not(#ot-sdk-cookie-policy-v2) tr{margin:0 0 1em 0}.ot-sdk-cookie-policy:not(#ot-sdk-cookie-policy-v2) tr:nth-child(odd),.ot-sdk-cookie-policy:not(#ot-sdk-cookie-policy-v2) tr:nth-child(odd) a{background:#f6f6f4}.ot-sdk-cookie-policy:not(#ot-sdk-cookie-policy-v2) td{border:none;border-bottom:1px solid #eee;position:relative;padding-left:50%}.ot-sdk-cookie-policy:not(#ot-sdk-cookie-policy-v2) td:before{position:absolute;height:100%;left:6px;width:40%;padding-right:10px}.ot-sdk-cookie-policy:not(#ot-sdk-cookie-policy-v2) .ot-mobile-border{display:inline-block;background-color:#e4e4e4;position:absolute;height:100%;top:0;left:45%;width:2px}.ot-sdk-cookie-policy:not(#ot-sdk-cookie-policy-v2) td:before{content:attr(data-label);font-weight:bold}.ot-sdk-cookie-policy:not(#ot-sdk-cookie-policy-v2) li{word-break:break-word;word-wrap:break-word}#ot-sdk-cookie-policy-v2.ot-sdk-cookie-policy table{overflow:hidden}#ot-sdk-cookie-policy-v2.ot-sdk-cookie-policy table td{border:none;border-bottom:1px solid #d7d7d7}#ot-sdk-cookie-policy-v2.ot-sdk-cookie-policy table,#ot-sdk-cookie-policy-v2.ot-sdk-cookie-policy thead,#ot-sdk-cookie-policy-v2.ot-sdk-cookie-policy tbody,#ot-sdk-cookie-policy-v2.ot-sdk-cookie-policy th,#ot-sdk-cookie-policy-v2.ot-sdk-cookie-policy td,#ot-sdk-cookie-policy-v2.ot-sdk-cookie-policy tr{display:block}#ot-sdk-cookie-policy-v2.ot-sdk-cookie-policy table .ot-host,#ot-sdk-cookie-policy-v2.ot-sdk-cookie-policy table .ot-cookies-type{width:auto}#ot-sdk-cookie-policy-v2.ot-sdk-cookie-policy tr{margin:0 0 1em 0}#ot-sdk-cookie-policy-v2.ot-sdk-cookie-policy td:before{height:100%;width:40%;padding-right:10px}#ot-sdk-cookie-policy-v2.ot-sdk-cookie-policy td:before{content:attr(data-label);font-weight:bold}#ot-sdk-cookie-policy-v2.ot-sdk-cookie-policy li{word-break:break-word;word-wrap:break-word}#ot-sdk-cookie-policy-v2.ot-sdk-cookie-policy thead tr{position:absolute;top:-9999px;left:-9999px;z-index:-9999}#ot-sdk-cookie-policy-v2.ot-sdk-cookie-policy table tr:last-child td{border-bottom:1px solid #d7d7d7;border-right:0px}#ot-sdk-cookie-policy-v2.ot-sdk-cookie-policy table tr:last-child td:last-child{border-bottom:0px}}\n                \n                    #ot-sdk-cookie-policy-v2.ot-sdk-cookie-policy h5,\n                    #ot-sdk-cookie-policy-v2.ot-sdk-cookie-policy h6,\n                    #ot-sdk-cookie-policy-v2.ot-sdk-cookie-policy li,\n                    #ot-sdk-cookie-policy-v2.ot-sdk-cookie-policy p,\n                    #ot-sdk-cookie-policy-v2.ot-sdk-cookie-policy a,\n                    #ot-sdk-cookie-policy-v2.ot-sdk-cookie-policy span,\n                    #ot-sdk-cookie-policy-v2.ot-sdk-cookie-policy td,\n                    #ot-sdk-cookie-policy-v2.ot-sdk-cookie-policy #cookie-policy-description {\n                        color: #696969;\n                    }\n                    #ot-sdk-cookie-policy-v2.ot-sdk-cookie-policy th {\n                        color: #696969;\n                    }\n                    #ot-sdk-cookie-policy-v2.ot-sdk-cookie-policy .ot-sdk-cookie-policy-group {\n                        color: #696969;\n                    }\n                    \n                    #ot-sdk-cookie-policy-v2.ot-sdk-cookie-policy #cookie-policy-title {\n                            color: #696969;\n                        }\n                    \n            \n                    #ot-sdk-cookie-policy-v2.ot-sdk-cookie-policy table th {\n                            background-color: #F8F8F8;\n                        }\n                    \n            </style><script type=\"text/javascript\" class=\"optanon-category-C0002\">\nvar appInsights=window.appInsights||function(a){\n  function b(a){c[a]=function(){var b=arguments;c.queue.push(function(){c[a].apply(c,b)})}}var c={config:a},d=document,e=window;setTimeout(function(){var b=d.createElement(\"script\");b.src=a.url||\"https://az416426.vo.msecnd.net/scripts/a/ai.0.js\",d.getElementsByTagName(\"script\")[0].parentNode.appendChild(b)});try{c.cookie=d.cookie}catch(a){}c.queue=[];for(var f=[\"Event\",\"Exception\",\"Metric\",\"PageView\",\"Trace\",\"Dependency\"];f.length;)b(\"track\"+f.pop());if(b(\"setAuthenticatedUserContext\"),b(\"clearAuthenticatedUserContext\"),b(\"startTrackEvent\"),b(\"stopTrackEvent\"),b(\"startTrackPage\"),b(\"stopTrackPage\"),b(\"flush\"),!a.disableExceptionTracking){f=\"onerror\",b(\"_\"+f);var g=e[f];e[f]=function(a,b,d,e,h){var i=g&&g(a,b,d,e,h);return!0!==i&&c[\"_\"+f](a,b,d,e,h),i}}return c\n  }({\n    instrumentationKey:\"d1bff452-16c0-4f15-9744-223f27abe717\",\n    disableExceptionTracking: false,\n    verboseLogging: true\n  });\nwindow.appInsights=appInsights,appInsights.queue&&0===appInsights.queue.length&&appInsights.trackPageView();</script><script src=\"https://az416426.vo.msecnd.net/scripts/a/ai.0.js\"></script><style type=\"text/css\">.drift-conductor-item iframe,\n.drift-frame-controller {\n  display: block;\n  min-width: 0;\n  max-width: none;\n  min-height: 0;\n  max-height: none;\n}\n\n.drift-conductor-item iframe {\n  border: none !important;\n  background: transparent !important;\n  width: 100% !important;\n  height: 100% !important;\n  max-width: 100% !important;\n}\n\n.drift-conductor-item.with-transition {\n  -webkit-transition: opacity .5s ease, -webkit-transform .5s ease;\n  transition: opacity .5s ease, -webkit-transform .5s ease;\n  -o-transition: opacity .5s ease, -o-transform .5s ease;\n  -moz-transition: opacity .5s ease, transform .5s ease, -moz-transform .5s ease;\n  transition: opacity .5s ease, transform .5s ease;\n  transition: opacity .5s ease, transform .5s ease, -webkit-transform .5s ease, -moz-transform .5s ease, -o-transform .5s ease;\n}\n\n.drift-frame-chat.with-transition.drift-frame-chat__takeover {\n  -webkit-transition: opacity .5s ease;\n  -o-transition: opacity .5s ease;\n  -moz-transition: opacity .5s ease;\n  transition: opacity .5s ease;\n}\n\n.drift-conductor-item.with-height-transition {\n  -webkit-transition: max-height .5s ease, opacity .5s ease, -webkit-transform .5s ease;\n  transition: max-height .5s ease, opacity .5s ease, -webkit-transform .5s ease;\n  -o-transition: max-height .5s ease, opacity .5s ease, -o-transform .5s ease;\n  -moz-transition: max-height .5s ease, opacity .5s ease, transform .5s ease, -moz-transform .5s ease;\n  transition: max-height .5s ease, opacity .5s ease, transform .5s ease;\n  transition: max-height .5s ease, opacity .5s ease, transform .5s ease, -webkit-transform .5s ease, -moz-transform .5s ease, -o-transform .5s ease;\n}\n\n.drift-conductor-item {\n  -webkit-transform: translateZ(0);\n     -moz-transform: translateZ(0);\n          transform: translateZ(0);\n}\n\n.drift-frame-controller-align-left {\n  left: 24px;\n}\n\n.drift-frame-controller-align-right {\n  right: 24px;\n}\n\n.drift-frame-chat:not(iframe) {\n  bottom: -webkit-calc(12px + 52px + 24px);\n  bottom: -moz-calc(12px + 52px + 24px);\n  bottom: calc(12px + 52px + 24px); /* vertical margin x 2 + widget icon size*/\n  width: 100%;\n  height: 100%;\n  min-width: 320px;\n  opacity: 0;\n  visibility: hidden;\n\n  -webkit-transform: translateY(88px);\n\n     -moz-transform: translateY(88px);\n\n      -ms-transform: translateY(88px);\n\n       -o-transform: translateY(88px);\n\n          transform: translateY(88px);\n}\n\n.drift-frame-chat:not(iframe).drift-chat-open {\n  opacity: 1;\n  visibility: visible;\n  -webkit-transform: translateY(0px);\n     -moz-transform: translateY(0px);\n      -ms-transform: translateY(0px);\n       -o-transform: translateY(0px);\n          transform: translateY(0px);\n}\n\n.drift-frame-chat.drift-frame-chat__takeover:not(iframe):not(.drift-frame--mobile) {\n  min-width: 600px;\n}\n\n.drift-frame-chat-align-left {\n  left: 12px;\n}\n\n.drift-frame-chat-align-right {\n  right: 12px;\n}\n\n.drift-frame-takeover,\n.drift-frame-chat__takeover {\n  left: 0;\n  top: 0;\n  width: 100%;\n  height: 100%;\n  min-height: 100%;\n  min-width: 100%;\n}\n\n.drift-frame-chat__takeover.hide-frame {\n  visibility: hidden;\n}\n\n.drift-frame-emailCapture {\n  bottom: 0;\n  height: -webkit-calc(100% + 30px);\n  height: -moz-calc(100% + 30px);\n  height: calc(100% + 30px);\n}\n\n.drift-frame-emailCapture-align-left {\n  left: 0;\n}\n\n.drift-frame-emailCapture-align-right {\n  right: 0;\n}\n\n.drift-frame-slider {\n  bottom: 18px;\n  height: -webkit-calc(100% + 20px);\n  height: -moz-calc(100% + 20px);\n  height: calc(100% + 20px);\n}\n\n.drift-frame-slider-align-left {\n  left: 96px;\n}\n\n.drift-frame-slider-align-right {\n  right: 96px;\n}\n\n.drift-frame--mobile.drift-frame-slider-align-left {\n  left: 68px;\n}\n\n.drift-frame--mobile.drift-frame-slider-align-right {\n  right: 68px;\n}\n\n.drift-widget-content-box {\n  background: rgba(0,0,0,.5);\n  width: 100%;\n  height: 100%;\n\n  position: fixed;\n  z-index: 2147483647;\n  top: 0;\n  left: 0;\n}\n\n.drift-widget-content-box > * {\n  margin: 0 auto;\n  position: relative;\n  left: 50%;\n  top: 50%;\n  -webkit-transform: translate(-50%, -50%);\n     -moz-transform: translate(-50%, -50%);\n      -ms-transform: translate(-50%, -50%);\n       -o-transform: translate(-50%, -50%);\n          transform: translate(-50%, -50%);\n  max-height: 80%;\n\n}\n\n@media\n(max-height: 685px),\n(min-width: 369px) and (max-width: 380px),\n(max-width: 367px),\n\n(min-device-width: 320px)\nand (max-device-width: 480px)\nand (-webkit-min-device-pixel-ratio: 2)\nand (orientation: portrait)\n\n{\n  .drift-frame--mobile.drift-frame-chat.drift-chat-open {\n    bottom: 0;\n    right: 0;\n    left: 0;\n    z-index: 2147483647 !important;\n  }\n\n  .drift-frame--mobile.drift-frame-controller {\n    bottom: 0;\n  }\n\n  .drift-frame--mobile.drift-frame-controller.drift-frame-controller-align-left {\n    left: 0;\n  }\n\n  .drift-frame--mobile.drift-frame-controller.drift-frame-controller-align-right {\n    right: 0;\n  }\n\n\n  .drift-frame--mobile.drift-frame-controller.drift-chat-open {\n    display: none;\n  }\n\n  .drift-frame--mobile.drift-frame-slider {\n    bottom: 0;\n    max-width: 320px;\n  }\n\n  .drift-frame--mobile iframe.drift-frame-chat {\n    width: 1px !important;\n    min-width: 100%;\n\n    height: 1px !important;\n    min-height: 100%;\n  }\n\n  html.drift-widget-parent--chat-open,\n  html.drift-widget-parent--chat-open > body {\n    overflow: hidden !important;\n    position: static !important;\n  }\n}\n</style></head><body><div id=\"__next\"><style>\n      .__react_component_tooltip { background: #222; color: #fff }\n    </style><div class=\"page  research-center-web jobv2\"><nav id=\"navbar\" class=\"pxl-navbar pxl-navbar--white\"><div class=\"pxl-navbar__branding\"><a href=\"/\"><svg id=\"psLogo\" class=\"pxl-navbar__logo\" xmlns=\"http://www.w3.org/2000/svg\" xmlns:xlink=\"http://www.w3.org/1999/xlink\" viewBox=\"0 0 89 31\" version=\"1.1\"><g stroke=\"none\" stroke-width=\"1\" fill=\"none\" fill-rule=\"evenodd\"><g id=\"TopDots\" transform=\"translate(18.760000, 0.170000)\" class=\"top-dot\"><path d=\"M4.56,2.27 C4.56265817,2.85613025 4.33236504,3.41930945 3.91978371,3.83564152 C3.50720238,4.25197358 2.94613031,4.48735383 2.36,4.49 C1.77040373,4.49267395 1.2041831,4.25964052 0.787271286,3.84272871 C0.370359476,3.4258169 0.137326053,2.85959627 0.14,2.27 C0.14,1.04 1.12,0.05 2.34,0.05 C3.56,0.05 4.54,1.05 4.54,2.27\" id=\"Shape\" fill-rule=\"nonzero\"></path><path d=\"M4.55,8.61 C4.55265817,9.19613025 4.32236504,9.75930945 3.90978371,10.1756415 C3.49720238,10.5919736 2.93613031,10.8273538 2.35,10.83 C1.76386969,10.8273538 1.20279762,10.5919736 0.790216289,10.1756415 C0.377634962,9.75930945 0.147341826,9.19613025 0.15,8.61 C0.15,7.38 1.13,6.39 2.35,6.39 C3.57,6.39 4.55,7.39 4.55,8.61\" id=\"Path\"></path></g><g id=\"BottomDots\" class=\"bottom-dot\" transform=\"translate(0.000000, 12.900000)\"><path d=\"M4.41,2.22 C4.41,3.44 3.42,4.44 2.21,4.44 C1.62213483,4.44000602 1.05849188,4.20579519 0.643747826,3.78917446 C0.229003769,3.37255373 -0.00266002345,2.80785916 3.38813179e-21,2.22 C3.38813179e-21,0.99 0.99,0 2.2,0 C3.42,0 4.41,1 4.41,2.22\" fill-rule=\"nonzero\"></path><path d=\"M4.41,8.55 C4.41,9.78 3.42,10.77 2.21,10.77 C1.62213483,10.770006 1.05849188,10.5357952 0.643747826,10.1191745 C0.229003769,9.70255373 -0.00266002345,9.13785916 2.45361728e-16,8.55 C3.38813179e-21,7.34 0.99,6.34 2.2,6.34 C3.42,6.34 4.41,7.33 4.41,8.55\" fill-rule=\"nonzero\"></path><path d=\"M4.41,14.89 C4.41265817,15.4761303 4.18236504,16.0393094 3.76978371,16.4556415 C3.35720238,16.8719736 2.79613031,17.1073538 2.21,17.11 C1.62213483,17.110006 1.05849188,16.8757952 0.643747826,16.4591745 C0.229003769,16.0425537 -0.00266002345,15.4778592 0,14.89 C0,13.66 0.99,12.67 2.2,12.67 C3.42,12.67 4.41,13.67 4.41,14.89\" fill-rule=\"nonzero\"></path><path d=\"M23.32,2.22 C23.32,3.44 22.32,4.43 21.12,4.43 C20.5304037,4.43267395 19.9641831,4.19964052 19.5472713,3.78272871 C19.1303595,3.3658169 18.8973261,2.79959627 18.9,2.21 C18.9781795,1.04975186 19.9421209,0.14857658 21.105,0.14857658 C22.2678791,0.14857658 23.2318205,1.04975186 23.31,2.21\"></path><path d=\"M23.31,8.55 C23.31,9.78 22.33,10.77 21.11,10.77 C20.5238697,10.7673538 19.9627976,10.5319736 19.5502163,10.1156415 C19.137635,9.69930945 18.9073418,9.13613025 18.91,8.55 C18.91,7.33 19.89,6.34 21.11,6.34 C22.33,6.34 23.31,7.33 23.31,8.55\"></path><path d=\"M23.31,14.9 C23.3100125,16.1166547 22.3266422,17.1044948 21.11,17.11 C19.8933578,17.1044948 18.9099875,16.1166547 18.91,14.9 C18.91,13.67 19.89,12.68 21.11,12.68 C22.33,12.68 23.31,13.67 23.31,14.9\"></path><path d=\"M17.01,2.21 C17.01,3.44 16.03,4.43 14.81,4.43 C14.2238697,4.42735383 13.6627976,4.19197358 13.2502163,3.77564152 C12.837635,3.35930945 12.6073418,2.79613025 12.61,2.21 C12.61,0.99 13.59,1.77635684e-15 14.81,1.77635684e-15 C16.03,1.77635684e-15 17.01,0.99 17.01,2.21\"></path><path d=\"M17.01,8.55 C17.01,9.78 16.03,10.77 14.81,10.77 C14.2238697,10.7673538 13.6627976,10.5319736 13.2502163,10.1156415 C12.837635,9.69930945 12.6073418,9.13613025 12.61,8.55 C12.61,7.33 13.59,6.34 14.81,6.34 C16.03,6.34 17.01,7.33 17.01,8.55\"></path><path d=\"M17.01,14.9 C17.0100125,16.1166547 16.0266422,17.1044948 14.81,17.11 C13.5933578,17.1044948 12.6099875,16.1166547 12.61,14.9 C12.61,13.67 13.59,12.68 14.81,12.68 C16.03,12.68 17.01,13.67 17.01,14.9\"></path><path d=\"M10.71,8.55 C10.71,9.78 9.72,10.77 8.51,10.77 C7.92386969,10.7673538 7.36279762,10.5319736 6.95021629,10.1156415 C6.53763496,9.69930945 6.30734183,9.13613025 6.31,8.55 C6.23019356,7.71393256 6.63119079,6.90488544 7.34485273,6.46209376 C8.05851467,6.01930208 8.96148533,6.01930208 9.67514727,6.46209376 C10.3888092,6.90488544 10.7898064,7.71393256 10.71,8.55\"></path><path d=\"M10.71,14.9 C10.7045247,16.1127513 9.72275128,17.0945247 8.51,17.1 C7.29335776,17.0944948 6.30998754,16.1066547 6.31,14.89 C6.31,13.66 7.29,12.67 8.51,12.67 C9.73,12.67 10.71,13.66 10.71,14.89\"></path></g><g id=\"PayScale\" transform=\"translate(29.000000, 12.000000)\" fill-rule=\"nonzero\"><path d=\"M49.21,11.76 L51.45,11.76 L51.45,0.76 L49.2,0.76 L49.21,11.76 Z M57.68,7.08 C57.68,6.73 57.65,5.36 56.35,5.36 C55.37,5.36 54.95,6.18 54.91,7.08 L57.68,7.08 Z M54.84,8.49 C54.84,8.99 54.84,10.55 56.39,10.55 C56.96,10.55 57.51,10.33 57.73,9.48 L59.77,9.48 C59.74,9.85 59.66,10.5 59.04,11.16 C58.48,11.75 57.48,12.06 56.36,12.06 C55.71,12.06 54.47,11.93 53.67,11.09 C52.97,10.36 52.69,9.33 52.69,8.09 C52.69,6.85 52.99,5.36 54.28,4.46 C54.9159355,4.03517463 55.6652592,3.81212012 56.43,3.82 C57.51,3.82 58.82,4.25 59.51,5.68 C60.02,6.73 59.99,7.85 59.96,8.49 L54.84,8.49 L54.84,8.49 Z\"></path><path d=\"M2.58,5.74 L3.52,5.74 C4.28,5.74 5.68,5.74 5.68,4.06 C5.68,2.57 4.43,2.57 3.76,2.57 L2.58,2.57 L2.58,5.74 L2.58,5.74 Z M0.27,0.79 L3.64,0.79 C5.21,0.79 6.53,0.9 7.44,1.99 C8.01,2.69 8.1,3.49 8.1,4.05 C8.1,5.33 7.57,6.41 6.71,6.97 C5.91,7.49 4.81,7.49 4.23,7.49 L2.64,7.49 L2.64,11.76 L0.27,11.76 L0.27,0.8 L0.27,0.79 Z M10.36,9.39 C10.36,9.91 10.7,10.49 11.45,10.49 C11.85,10.49 12.19,10.29 12.4,10.04 C12.8,9.56 12.82,8.96 12.82,8.14 C11.39,8.02 10.36,8.48 10.36,9.39 L10.36,9.39 Z M13.11,11.76 C13.03,11.29 13.03,10.94 13.03,10.71 C12.19,11.95 11.17,11.95 10.75,11.95 C9.65,11.95 9.15,11.63 8.85,11.34 C8.41124567,10.8838455 8.17043902,10.2728435 8.18,9.64 C8.18,9.04 8.39,7.91 9.8,7.26 C10.74,6.83 12.08,6.83 12.82,6.83 C12.8454205,6.48678098 12.8115936,6.14174705 12.72,5.81 C12.52,5.25 11.92,5.18 11.72,5.18 C11.4,5.18 11.07,5.31 10.87,5.6 C10.71,5.86 10.71,6.1 10.71,6.29 L8.51,6.29 C8.53,5.87 8.57,5.05 9.37,4.47 C10.07,3.97 10.97,3.85 11.69,3.85 C12.32,3.85 13.65,3.95 14.41,4.84 C14.92,5.46 14.94,6.38 14.94,6.73 L14.96,9.82 C14.96,10.47 15,11.12 15.14,11.76 L13.1,11.76 L13.11,11.76 Z M17.91,4.08 L19.58,9.41 L21.29,4.08 L23.52,4.08 L20.62,11.62 L19.71,14.68 L17.45,14.68 L18.42,11.79 L15.49,4.08 L17.91,4.08 Z M26.22,8.25 C26.22,8.76 26.19,10.33 28.2,10.33 C29.47,10.33 30.28,9.73 30.28,8.8 C30.28,7.82 29.6,7.65 28.2,7.32 C25.6,6.67 24.18,5.89 24.18,3.99 C24.18,2.18 25.47,0.59 28.51,0.59 C29.64,0.59 30.98,0.84 31.81,1.87 C32.42,2.67 32.42,3.55 32.42,3.91 L30.17,3.91 C30.14,3.53 30.02,2.24 28.4,2.24 C27.31,2.24 26.6,2.84 26.6,3.73 C26.6,4.75 27.4,4.93 28.97,5.33 C30.87,5.78 32.73,6.3 32.73,8.45 C32.73,10.42 31.09,12.02 28.07,12.02 C23.87,12.02 23.85,9.32 23.83,8.25 L26.23,8.25 L26.22,8.25 Z M38.03,6.8 C38,6.53 37.91,5.39 37.05,5.39 C35.99,5.39 35.87,7 35.87,7.84 C35.87,8.26 35.92,9.64 36.43,10.2 C36.5814395,10.3715036 36.8013049,10.4667786 37.03,10.46 C37.28,10.46 37.97,10.35 38.06,8.98 L40.27,8.98 C40.24,9.58 40.17,10.5 39.29,11.25 C38.74,11.72 38.06,11.96 37.11,11.96 C36.1,11.96 35.21,11.73 34.51,10.94 C33.8268517,10.0826896 33.4817396,9.00465906 33.54,7.91 C33.54,4.41 36.1,3.93 37.11,3.93 C38.53,3.93 40.21,4.72 40.23,6.8 L38.03,6.8 L38.03,6.8 Z M43.01,9.39 C43.01,9.91 43.36,10.49 44.1,10.49 C44.5,10.49 44.84,10.29 45.05,10.04 C45.45,9.56 45.47,8.96 45.47,8.14 C44.04,8.02 43.01,8.48 43.01,9.39 L43.01,9.39 Z M45.76,11.76 C45.69,11.29 45.69,10.94 45.69,10.71 C44.84,11.95 43.83,11.95 43.4,11.95 C42.3,11.95 41.8,11.63 41.5,11.34 C41.0612457,10.8838455 40.820439,10.2728435 40.83,9.64 C40.83,9.04 41.04,7.91 42.45,7.26 C43.39,6.83 44.73,6.83 45.47,6.83 C45.47,6.33 45.46,6.09 45.37,5.81 C45.17,5.25 44.58,5.18 44.37,5.18 C44.05,5.18 43.72,5.31 43.52,5.6 C43.36,5.86 43.36,6.1 43.36,6.29 L41.16,6.29 C41.18,5.87 41.22,5.05 42.03,4.47 C42.73,3.97 43.61,3.85 44.34,3.85 C44.97,3.85 46.31,3.95 47.06,4.84 C47.58,5.46 47.59,6.38 47.59,6.73 L47.61,9.82 C47.61,10.47 47.65,11.12 47.79,11.76 L45.76,11.76 L45.76,11.76 Z\"></path></g></g></svg></a></div><div class=\"pxl-navbar__content\"><ul class=\"pxl-navbar__nav\"><li id=\"personal-nav\" class=\"pxl-navbar__nav-item \"><div class=\"pxl-navbar__nav-item-content\"><a class=\"pxl-navbar__nav-link\">For You</a><div class=\"pxl-navbar__nav-x\"></div></div><div class=\"pxl-navbar__subnav\"><div class=\"pxl-navbar__cta\"><div class=\"pxl-navbar__cta-item\" id=\"foryoucta\"><div class=\"pxl-navbar__cta-desktop\"><div class=\"pxl-navbar__cta-header\">Start Salary Survey</div><p class=\"pxl-navbar__cta-copy\">Do you know what people like you are earning? Stop guessing.</p><a href=\"/wizards/choose.aspx?tk=header-ps\" id=\"foryouctacta\" class=\"pxl-navbar__cta-btn pxl-navbar__cta-btn--consumer\">Find Out</a></div><a href=\"/wizards/choose.aspx?tk=header-ps\" id=\"foryouctactaMobile\" class=\"pxl-navbar__cta-btn pxl-navbar__cta-mobile pxl-navbar__cta-btn--consumer\">Start Salary Survey</a></div></div><div class=\"pxl-navbar__subnav-sections\"><div id=\"salaryResearch\" class=\"pxl-navbar__subnav-section\"><a class=\"pxl-navbar__subnav-header\" href=\"/research/IN/Country=India/Salary\">Salary Research</a><div class=\"pxl-navbar__subnav-items\"><a href=\"/research/IN/Employer\" class=\"pxl-navbar__subnav-item\"><div class=\"pxl-navbar__subnav-text\"><div class=\"pxl-navbar__subnav-link\">By Company</div></div></a><a href=\"/research/IN/Job\" class=\"pxl-navbar__subnav-item\"><div class=\"pxl-navbar__subnav-text\"><div class=\"pxl-navbar__subnav-link\">By Job Title</div></div></a><a href=\"/research/IN/Degree\" class=\"pxl-navbar__subnav-item\"><div class=\"pxl-navbar__subnav-text\"><div class=\"pxl-navbar__subnav-link\">By Degree</div></div></a><a href=\"/research/IN/Certification\" class=\"pxl-navbar__subnav-item\"><div class=\"pxl-navbar__subnav-text\"><div class=\"pxl-navbar__subnav-link\">By Certification</div></div></a><a href=\"/research/IN/Skill\" class=\"pxl-navbar__subnav-item\"><div class=\"pxl-navbar__subnav-text\"><div class=\"pxl-navbar__subnav-link\">By Skill</div></div></a><a href=\"/research/IN/Industry\" class=\"pxl-navbar__subnav-item\"><div class=\"pxl-navbar__subnav-text\"><div class=\"pxl-navbar__subnav-link\">By Industry</div></div></a><a href=\"/research/IN/School\" class=\"pxl-navbar__subnav-item\"><div class=\"pxl-navbar__subnav-text\"><div class=\"pxl-navbar__subnav-link\">By School</div></div></a></div><a href=\"/research/IN/Country=India/Salary\" class=\"pxl-navbar__subnav-all\">See all »</a></div><div id=\"personalResources\" class=\"pxl-navbar__subnav-section\"><a class=\"pxl-navbar__subnav-header\" href=\"/salary\">Resources</a><div class=\"pxl-navbar__subnav-items\"><a href=\"/salary-negotiation-guide\" class=\"pxl-navbar__subnav-item\"><div class=\"pxl-navbar__subnav-text\"><div class=\"pxl-navbar__subnav-link\">Salary Negotiation Guide</div></div></a><a href=\"/cost-of-living-calculator\" class=\"pxl-navbar__subnav-item\"><div class=\"pxl-navbar__subnav-text\"><div class=\"pxl-navbar__subnav-link\">Cost of Living Calculator</div></div></a><a href=\"/career-path-planner\" class=\"pxl-navbar__subnav-item\"><div class=\"pxl-navbar__subnav-text\"><div class=\"pxl-navbar__subnav-link\">Career Path Planner</div></div></a><a href=\"/job-search-engine\" class=\"pxl-navbar__subnav-item\"><div class=\"pxl-navbar__subnav-text\"><div class=\"pxl-navbar__subnav-link\">Find Jobs</div></div></a><a href=\"/events?type=b2c\" class=\"pxl-navbar__subnav-item\"><div class=\"pxl-navbar__subnav-text\"><div class=\"pxl-navbar__subnav-link\">Events</div></div></a></div><a href=\"/salary\" class=\"pxl-navbar__subnav-all\">See all »</a></div><div id=\"careerNews\" class=\"pxl-navbar__subnav-section\"><a class=\"pxl-navbar__subnav-header\" href=\"/career-news\">Career News Blog</a><div class=\"pxl-navbar__subnav-subheader\">Browse By Topic:</div><div class=\"pxl-navbar__subnav-items\"><a href=\"/career-news/topic/career-advice\" class=\"pxl-navbar__subnav-item\"><div class=\"pxl-navbar__subnav-text\"><div class=\"pxl-navbar__subnav-link\">Career Advice</div></div></a><a href=\"/career-news/topic/career-profiles\" class=\"pxl-navbar__subnav-item\"><div class=\"pxl-navbar__subnav-text\"><div class=\"pxl-navbar__subnav-link\">Career Profiles</div></div></a><a href=\"/career-news/topic/current-events\" class=\"pxl-navbar__subnav-item\"><div class=\"pxl-navbar__subnav-text\"><div class=\"pxl-navbar__subnav-link\">Current Events</div></div></a><a href=\"/career-news/topic/data-research\" class=\"pxl-navbar__subnav-item\"><div class=\"pxl-navbar__subnav-text\"><div class=\"pxl-navbar__subnav-link\">Data &amp; Research</div></div></a><a href=\"/career-news/topic/negotiation\" class=\"pxl-navbar__subnav-item\"><div class=\"pxl-navbar__subnav-text\"><div class=\"pxl-navbar__subnav-link\">Negotiation</div></div></a><a href=\"/career-news/topic/work-culture\" class=\"pxl-navbar__subnav-item\"><div class=\"pxl-navbar__subnav-text\"><div class=\"pxl-navbar__subnav-link\">Work Culture</div></div></a></div><a href=\"/career-news\" class=\"pxl-navbar__subnav-all\">See all »</a></div></div></div></li><li id=\"business-nav\" class=\"pxl-navbar__nav-item \"><div class=\"pxl-navbar__nav-item-content\"><a class=\"pxl-navbar__nav-link\">For Your Business</a><div class=\"pxl-navbar__nav-x\"></div></div><div class=\"pxl-navbar__subnav\"><div class=\"pxl-navbar__cta\"><div class=\"pxl-navbar__cta-item\" id=\"getademo\"><div class=\"pxl-navbar__cta-desktop\"><div class=\"pxl-navbar__cta-header\">Get a Demo</div><p class=\"pxl-navbar__cta-copy\">Get pay right with PayScale.</p><a href=\"http://resources.payscale.com/hr-request-a-demo.html?tk=rc-header-nav\" id=\"getademocta\" class=\"pxl-navbar__cta-btn pxl-navbar__cta-btn--b2b\">Get Started</a></div><a href=\"http://resources.payscale.com/hr-request-a-demo.html?tk=rc-header-nav\" id=\"getademoctaMobile\" class=\"pxl-navbar__cta-btn pxl-navbar__cta-mobile pxl-navbar__cta-btn--b2b\">Get a Demo</a></div><div class=\"pxl-navbar__cta-item\" id=\"priceajob\"><div class=\"pxl-navbar__cta-desktop\"><div class=\"pxl-navbar__cta-header\">Price a Job for Free</div><p class=\"pxl-navbar__cta-copy\">Eliminate compensation guesswork.</p><a href=\"/price-a-job?tk=ps-header\" id=\"priceajobcta\" class=\"pxl-navbar__cta-btn pxl-navbar__cta-btn--b2b\">Get Started</a></div><a href=\"/price-a-job?tk=ps-header\" id=\"priceajobctaMobile\" class=\"pxl-navbar__cta-btn pxl-navbar__cta-mobile pxl-navbar__cta-btn--b2b\">Price a Job</a></div></div><div class=\"pxl-navbar__subnav-sections\"><div id=\"compensationSoftware\" class=\"pxl-navbar__subnav-section\"><a class=\"pxl-navbar__subnav-header\" href=\"/hr/compensation-software\">Compensation Software</a><div class=\"pxl-navbar__subnav-items\"><a href=\"/hr/insight-lab\" class=\"pxl-navbar__subnav-item pxl-navbar__subnav-item--desc\"><div class=\"pxl-navbar__subnav-text\"><div class=\"pxl-navbar__subnav-link\">Insight Lab</div><p class=\"pxl-navbar__subnav-desc\">Fresh market data paired with robust analytics.</p></div></a><a href=\"/hr/team\" class=\"pxl-navbar__subnav-item pxl-navbar__subnav-item--desc\"><div class=\"pxl-navbar__subnav-text\"><div class=\"pxl-navbar__subnav-link\">Team</div><p class=\"pxl-navbar__subnav-desc\">Compensation collaboration in the cloud.</p></div></a><a href=\"/hr/marketpay\" class=\"pxl-navbar__subnav-item pxl-navbar__subnav-item--desc\"><div class=\"pxl-navbar__subnav-text\"><div class=\"pxl-navbar__subnav-link\">MarketPay</div><p class=\"pxl-navbar__subnav-desc\">Survey management &amp; participation.</p></div></a></div><a href=\"/hr/compensation-software\" class=\"pxl-navbar__subnav-all\">See all »</a></div><div id=\"hrResources\" class=\"pxl-navbar__subnav-section\"><a class=\"pxl-navbar__subnav-header\" href=\"/hr\">Resources</a><div class=\"pxl-navbar__subnav-items\"><a href=\"/compference\" class=\"pxl-navbar__subnav-item pxl-navbar__subnav-item--desc\"><div class=\"pxl-navbar__subnav-text\"><div class=\"pxl-navbar__subnav-link\">Compference</div><p class=\"pxl-navbar__subnav-desc\">The event for compensation changemakers.</p></div></a><a href=\"https://payscale.influitive.com/users/sign_in\" class=\"pxl-navbar__subnav-item pxl-navbar__subnav-item--desc\"><div class=\"pxl-navbar__subnav-text\"><div class=\"pxl-navbar__subnav-link\">Comptopia Community</div><p class=\"pxl-navbar__subnav-desc\">Plug into compensation conversations.</p></div></a><a href=\"/hr/customers\" class=\"pxl-navbar__subnav-item pxl-navbar__subnav-item--desc\"><div class=\"pxl-navbar__subnav-text\"><div class=\"pxl-navbar__subnav-link\">Customer Testimonials</div><p class=\"pxl-navbar__subnav-desc\">See why customers love us.</p></div></a></div><a href=\"/hr\" class=\"pxl-navbar__subnav-all\">See all »</a></div><div id=\"compensationAdvice\" class=\"pxl-navbar__subnav-section\"><a class=\"pxl-navbar__subnav-header\" href=\"/compensation-today\">Compensation Today Blog</a><div class=\"pxl-navbar__subnav-subheader\">Browse By Topic:</div><div class=\"pxl-navbar__subnav-items\"><a href=\"/compensation-today/topic/comp-communication\" class=\"pxl-navbar__subnav-item\"><div class=\"pxl-navbar__subnav-text\"><div class=\"pxl-navbar__subnav-link\">Comp Communication</div></div></a><a href=\"/compensation-today/topic/comp-strategy\" class=\"pxl-navbar__subnav-item\"><div class=\"pxl-navbar__subnav-text\"><div class=\"pxl-navbar__subnav-link\">Comp Strategy</div></div></a><a href=\"/compensation-today/topic/growth\" class=\"pxl-navbar__subnav-item\"><div class=\"pxl-navbar__subnav-text\"><div class=\"pxl-navbar__subnav-link\">Growth</div></div></a><a href=\"/compensation-today/topic/modernizing-comp\" class=\"pxl-navbar__subnav-item\"><div class=\"pxl-navbar__subnav-text\"><div class=\"pxl-navbar__subnav-link\">Modernizing Comp</div></div></a><a href=\"/compensation-today/topic/pay-equity\" class=\"pxl-navbar__subnav-item\"><div class=\"pxl-navbar__subnav-text\"><div class=\"pxl-navbar__subnav-link\">Pay Equity</div></div></a><a href=\"/compensation-today/topic/recession\" class=\"pxl-navbar__subnav-item\"><div class=\"pxl-navbar__subnav-text\"><div class=\"pxl-navbar__subnav-link\">Recession</div></div></a><a href=\"/compensation-today/topic/retention\" class=\"pxl-navbar__subnav-item\"><div class=\"pxl-navbar__subnav-text\"><div class=\"pxl-navbar__subnav-link\">Retention</div></div></a></div><a href=\"/compensation-today\" class=\"pxl-navbar__subnav-all\">See all »</a></div></div></div></li><li id=\"data-nav\" class=\"pxl-navbar__nav-item \"><div class=\"pxl-navbar__nav-item-content\"><a class=\"pxl-navbar__nav-link\">Compensation Research</a><div class=\"pxl-navbar__nav-x\"></div></div><div class=\"pxl-navbar__subnav\"><div class=\"pxl-navbar__cta\"><div class=\"pxl-navbar__cta-item\" id=\"compbestpractices\"><div class=\"pxl-navbar__cta-desktop\"><div class=\"pxl-navbar__cta-header\">Annual Compensation Best Practices Report</div><p class=\"pxl-navbar__cta-copy\">The largest survey in North America on compensation topics and trends</p><a href=\"/cbpr\" id=\"compbestpracticescta\" class=\"pxl-navbar__cta-btn pxl-navbar__cta-btn--b2b\">Learn More</a></div><a href=\"/cbpr\" id=\"compbestpracticesctaMobile\" class=\"pxl-navbar__cta-btn pxl-navbar__cta-mobile pxl-navbar__cta-btn--b2b\">See Comp Best Practices</a></div></div><div class=\"pxl-navbar__subnav-sections\"><div id=\"originalResearch\" class=\"pxl-navbar__subnav-section\"><a class=\"pxl-navbar__subnav-header\" href=\"/data\">Original PayScale Research</a><div class=\"pxl-navbar__subnav-items\"><a href=\"/data/remote-work\" class=\"pxl-navbar__subnav-item pxl-navbar__subnav-item--desc\"><i class=\"icon-paper-plane pxl-navbar__subnav-icon\"></i><div class=\"pxl-navbar__subnav-text\"><div class=\"pxl-navbar__subnav-link\">Remote Work Report</div><p class=\"pxl-navbar__subnav-desc\">Guidance on remote work pay strategies.</p></div></a><a href=\"/data/covid-wage-growth\" class=\"pxl-navbar__subnav-item pxl-navbar__subnav-item--desc\"><i class=\"icon-temperatire pxl-navbar__subnav-icon\"></i><div class=\"pxl-navbar__subnav-text\"><div class=\"pxl-navbar__subnav-link\">COVID Wage Growth Report</div><p class=\"pxl-navbar__subnav-desc\">How the pandemic has affected wages across the U.S.</p></div></a><a href=\"/data/pay-transparency\" class=\"pxl-navbar__subnav-item pxl-navbar__subnav-item--desc\"><i class=\"icon-gauge pxl-navbar__subnav-icon\"></i><div class=\"pxl-navbar__subnav-text\"><div class=\"pxl-navbar__subnav-link\">Pay Transparency Report </div><p class=\"pxl-navbar__subnav-desc\">Does pay transparency close the gender wage gap?</p></div></a><a href=\"/college-salary-report\" class=\"pxl-navbar__subnav-item pxl-navbar__subnav-item--desc\"><i class=\"icon-graduation-cap pxl-navbar__subnav-icon\"></i><div class=\"pxl-navbar__subnav-text\"><div class=\"pxl-navbar__subnav-link\">College Salary Report</div><p class=\"pxl-navbar__subnav-desc\">Which alumni earn the most after graduation?</p></div></a><a href=\"/data/employee-engagement\" class=\"pxl-navbar__subnav-item pxl-navbar__subnav-item--desc\"><i class=\"icon-star pxl-navbar__subnav-icon\"></i><div class=\"pxl-navbar__subnav-text\"><div class=\"pxl-navbar__subnav-link\">Employee Engagement</div><p class=\"pxl-navbar__subnav-desc\">The formula for a winning company culture.</p></div></a><a href=\"/data/gender-pay-gap\" class=\"pxl-navbar__subnav-item pxl-navbar__subnav-item--desc\"><i class=\"icon-chart-line pxl-navbar__subnav-icon\"></i><div class=\"pxl-navbar__subnav-text\"><div class=\"pxl-navbar__subnav-link\">Gender Pay Gap</div><p class=\"pxl-navbar__subnav-desc\">Understand what's truly driving the gender pay gap.</p></div></a><a href=\"/payscale-index\" class=\"pxl-navbar__subnav-item pxl-navbar__subnav-item--desc\"><i class=\"icon-chart-arrow pxl-navbar__subnav-icon\"></i><div class=\"pxl-navbar__subnav-text\"><div class=\"pxl-navbar__subnav-link\">PayScale Index</div><p class=\"pxl-navbar__subnav-desc\">Follow wage changes of U.S. workers over time.</p></div></a><a href=\"/data/salary-history\" class=\"pxl-navbar__subnav-item pxl-navbar__subnav-item--desc\"><i class=\"icon-clock pxl-navbar__subnav-icon\"></i><div class=\"pxl-navbar__subnav-text\"><div class=\"pxl-navbar__subnav-link\">Salary History</div><p class=\"pxl-navbar__subnav-desc\">How the salary history question affects pay equity.</p></div></a><a href=\"/data/how-to-ask-for-a-raise\" class=\"pxl-navbar__subnav-item pxl-navbar__subnav-item--desc\"><i class=\"icon-dollar pxl-navbar__subnav-icon\"></i><div class=\"pxl-navbar__subnav-text\"><div class=\"pxl-navbar__subnav-link\">Raise Anatomy Report</div><p class=\"pxl-navbar__subnav-desc\">Improve your chances of receiving a raise when you ask.</p></div></a></div><a href=\"/data\" class=\"pxl-navbar__subnav-all\">See all »</a></div></div></div></li></ul><div id=\"ps-navbar-logout\" class=\"pxl-navbar__logout pxl-navbar__logout--out\"><a class=\"pxl-navbar__logout-link\" href=\"/logout.aspx\">Logout</a></div></div><div class=\"pxl-navbar__right\"><div class=\"pxl-navbar__search\"><div class=\"pxl-navbar__search-btn\"><i class=\"icon-search\"></i></div><form method=\"get\" action=\"/rcsearch.aspx\" class=\"pxl-navbar__search-bar\" __bizdiag=\"-1587827856\" __biza=\"W___\"><input type=\"hidden\" name=\"CountryName\" value=\"India\"><input type=\"hidden\" name=\"str\" value=\"\"><i class=\"icon-search pxl-navbar__search-icon\"></i><div role=\"combobox\" aria-haspopup=\"listbox\" aria-owns=\"react-autowhatever-psNavSearchbar\" aria-expanded=\"false\" class=\"react-autosuggest__container\"><input type=\"text\" value=\"\" autocomplete=\"off\" aria-autocomplete=\"list\" aria-controls=\"react-autowhatever-psNavSearchbar\" class=\"pxl-form-control pxl-navbar__search-input\" tabindex=\"0\" id=\"psNavSearchbar\" placeholder=\"Search salaries\"><div id=\"react-autowhatever-psNavSearchbar\" role=\"listbox\" class=\"react-autosuggest__suggestions-container\"></div></div></form></div><a id=\"psNavBarDashboard\" href=\"https://insight.payscale.com\" class=\"pxl-navbar__business-dashboard hidden\">Business Dashboard</a><li id=\"psNavBarMyAccount\" class=\"pxl-navbar__nav-item pxl-navbar__account hidden\"><div class=\"pxl-navbar__nav-item-content\"><a class=\"pxl-navbar__nav-link\">My Account</a></div><div class=\"pxl-navbar__subnav pxl-navbar__subnav--account\"><a class=\"pxl-navbar__subnav-link\" href=\"/logout.aspx\">Logout</a><a class=\"pxl-navbar__subnav-link\" href=\"/myaccount.aspx\">My Account</a><a class=\"pxl-navbar__subnav-link\" href=\"/mypayscale.aspx\">My Salary Reports</a></div></li><a id=\"psNavBarLogin\" href=\"/login.aspx\" class=\"pxl-navbar__login\">Log In</a><a id=\"psNavBarMyReports\" href=\"/mypayscale.aspx\" class=\"pxl-navbar__myreports hidden\">My Reports</a><a id=\"psNavBarLogout\" href=\"/logout.aspx\" class=\"pxl-navbar__logout hidden\">Log Out</a><div id=\"psNavBarMobileMenu\" class=\"pxl-navbar__menu--mobile\">Menu</div></div></nav><div class=\"jobv2__header-content\"><div class=\"jobv2__header-content__container\"><div class=\"breadcrumbs jobv2__breadcrumbs\"><div class=\"breadcrumbs__item\"><a href=\"/research/IN/Country=India/Salary\">India</a></div><div class=\"breadcrumbs__item\"><a href=\"/research/IN/Job\">Job</a></div><div class=\"breadcrumbs__item\">Accountant</div></div><h1 class=\"jobv2__title\">Average Accountant Salary in India</h1></div></div><div class=\"rc-sub-nav\"><div class=\"rc-sub-nav__contents\"><h1 class=\"rc-sub-nav__title\">Average Accountant Salary in India</h1><div class=\"nav-container \"><div role=\"button\" class=\"nav-container__scroll-icon nav-container__scroll-icon--hide\"><i class=\"icon-left-open\"></i></div><div class=\"nav-container__tabs\"><div class=\"tabs-container\"><div class=\"tabs-container__tabs nav-container__tabs-override\"><div class=\"tab tab--selected rc-sub-nav__active-tab\" role=\"button\" tabindex=\"0\"><span class=\"tab__name\">Pay</span></div><div class=\"tab rc-sub-nav__inactive-tab\" role=\"button\" tabindex=\"0\"><span class=\"tab__name\">Job Details</span></div><div class=\"tab rc-sub-nav__inactive-tab\" role=\"button\" tabindex=\"0\"><span class=\"tab__name\">Skills</span></div><div class=\"tab rc-sub-nav__inactive-tab\" role=\"button\" tabindex=\"0\"><span class=\"tab__name\">Job Listings</span></div><div class=\"tab rc-sub-nav__inactive-tab\" role=\"button\" tabindex=\"0\"><span class=\"tab__name\">Employers</span></div></div></div></div><div role=\"button\" class=\"nav-container__scroll-icon nav-container__scroll-icon--hide\"><i class=\"icon-right-open\"></i></div><div class=\"nav-container__inline-area\"><div class=\"rc-sub-nav__ctas\"><div class=\"rc-sub-nav__cta-wrapper\"><span class=\"rc-sub-nav__cta-label\">How should I pay?</span><a href=\"/price-a-job?tk=rc-subnav\" class=\"rc-sub-nav__cta-paj\">Price a Job</a></div><div class=\"rc-sub-nav__cta-wrapper\"><span class=\"rc-sub-nav__cta-label\">What am I worth?</span><a href=\"/wizards/choose.aspx?tk=rc-subnav\" class=\"rc-sub-nav__cta-survey\">Find market worth</a></div></div></div></div></div></div><div class=\"content one-col\"><div class=\"maincontent\"><section class=\"page-section\"><div class=\"paycharts\"><div class=\"paycharts__percentile\"><div class=\"paycharts__percentile--text\"><span class=\"paycharts__value\">₹249,659</span><div class=\"Dropdown-root\"><div class=\"Dropdown-control\" aria-haspopup=\"listbox\"><div class=\"Dropdown-placeholder\">/ year</div><div class=\"Dropdown-arrow-wrapper\"><span class=\"Dropdown-arrow\"></span></div></div></div><br>Avg. Base <!-- --> Salary<!-- --> (<!-- -->INR<!-- -->)</div><div id=\"percentile-chart-container\"><div class=\"percentile-chart \"><div class=\"percentile-chart__edge\"><div class=\"percentile-chart__edge-bar-low\"><svg class=\"gradient-bar\"><defs><linearGradient id=\"default-startEdge\"><stop class=\"svg-white\"></stop><stop offset=\"100%\" class=\"svg-ghost-light\"></stop></linearGradient></defs><rect x=\"0\" y=\"0\" width=\"100%\" height=\"100%\" fill=\"url('#default-startEdge')\"></rect></svg></div></div><div class=\"percentile-chart__low\"><div class=\"percentile-chart__low-bar\"></div><div class=\"percentile-chart__label\"><div>10%</div><div>₹123k</div></div></div><div class=\"percentile-chart__middle\"><div class=\"percentile-chart__middle-bar\"><svg class=\"gradient-bar\"><defs><linearGradient id=\"default-middleBar\"><stop class=\"svg-turqoise-dark\"></stop><stop offset=\"100%\" class=\"svg-cerulean\"></stop></linearGradient></defs><rect x=\"0\" y=\"0\" width=\"100%\" height=\"100%\" fill=\"url('#default-middleBar')\"></rect></svg></div><div class=\"percentile-chart__label--median\"><div class=\"percentile-chart__label percentile-chart__label--quartile\"></div><div class=\"percentile-chart__label\"><div>MEDIAN</div><div class=\"percentile-chart__median\">₹250k</div></div><div class=\"percentile-chart__label percentile-chart__label--quartile\"></div></div></div><div class=\"percentile-chart__high\"><div class=\"percentile-chart__high-bar\"></div><div class=\"percentile-chart__label\"><div>90%</div><div>₹506k</div></div></div><div class=\"percentile-chart__edge\"><div class=\"percentile-chart__edge-bar-high\"><svg class=\"gradient-bar\"><defs><linearGradient id=\"default-endEdge\"><stop class=\"svg-ghost-light\"></stop><stop offset=\"100%\" class=\"svg-white\"></stop></linearGradient></defs><rect x=\"0\" y=\"0\" width=\"100%\" height=\"100%\" fill=\"url('#default-endEdge')\"></rect></svg></div></div></div></div><p class=\"paycharts__percentile--desc\">The average <!-- -->salary<!-- --> for <!-- -->an Accountant<!-- --> is <!-- -->₹249,659</p></div><div class=\"paycharts__charttable\"><div class=\"charttable\"><div class=\"charttable__rows\"><div class=\"tablerow\"><div class=\"tablerow__title-wrapper\"><div class=\"tablerow__title\">Base Salary<span class=\"pxl-tooltip  \" data-tip=\"Salary represents gross income before taxes and deductions. It does not include additional pay such as benefits, bonuses, profit sharing or commissions.\" data-for=\"Base Salary\" currentitem=\"false\"><span class=\"icon-reverse icon-circle\"><i class=\"icon-info\"></i></span></span><div class=\"__react_component_tooltip t9675398-838c-426e-a958-8964ef34aaf7 place-right type-dark\" id=\"Base Salary\" data-id=\"tooltip\"><style>\n  \t.t9675398-838c-426e-a958-8964ef34aaf7 {\n\t    color: #fff;\n\t    background: #222;\n\t    border: 1px solid transparent;\n  \t}\n\n  \t.t9675398-838c-426e-a958-8964ef34aaf7.place-top {\n        margin-top: -10px;\n    }\n    .t9675398-838c-426e-a958-8964ef34aaf7.place-top::before {\n        border-top: 8px solid transparent;\n    }\n    .t9675398-838c-426e-a958-8964ef34aaf7.place-top::after {\n        border-left: 8px solid transparent;\n        border-right: 8px solid transparent;\n        bottom: -6px;\n        left: 50%;\n        margin-left: -8px;\n        border-top-color: #222;\n        border-top-style: solid;\n        border-top-width: 6px;\n    }\n\n    .t9675398-838c-426e-a958-8964ef34aaf7.place-bottom {\n        margin-top: 10px;\n    }\n    .t9675398-838c-426e-a958-8964ef34aaf7.place-bottom::before {\n        border-bottom: 8px solid transparent;\n    }\n    .t9675398-838c-426e-a958-8964ef34aaf7.place-bottom::after {\n        border-left: 8px solid transparent;\n        border-right: 8px solid transparent;\n        top: -6px;\n        left: 50%;\n        margin-left: -8px;\n        border-bottom-color: #222;\n        border-bottom-style: solid;\n        border-bottom-width: 6px;\n    }\n\n    .t9675398-838c-426e-a958-8964ef34aaf7.place-left {\n        margin-left: -10px;\n    }\n    .t9675398-838c-426e-a958-8964ef34aaf7.place-left::before {\n        border-left: 8px solid transparent;\n    }\n    .t9675398-838c-426e-a958-8964ef34aaf7.place-left::after {\n        border-top: 5px solid transparent;\n        border-bottom: 5px solid transparent;\n        right: -6px;\n        top: 50%;\n        margin-top: -4px;\n        border-left-color: #222;\n        border-left-style: solid;\n        border-left-width: 6px;\n    }\n\n    .t9675398-838c-426e-a958-8964ef34aaf7.place-right {\n        margin-left: 10px;\n    }\n    .t9675398-838c-426e-a958-8964ef34aaf7.place-right::before {\n        border-right: 8px solid transparent;\n    }\n    .t9675398-838c-426e-a958-8964ef34aaf7.place-right::after {\n        border-top: 5px solid transparent;\n        border-bottom: 5px solid transparent;\n        left: -6px;\n        top: 50%;\n        margin-top: -4px;\n        border-right-color: #222;\n        border-right-style: solid;\n        border-right-width: 6px;\n    }\n  </style></div></div><div class=\"tablerow__detail\"></div></div><div class=\"tablerow__value\">₹123k - ₹506k</div><div class=\"tablerow__bar\"><div class=\"tablerow__bar-range\" style=\"width:66%;margin-left:21%\"></div></div></div><div class=\"tablerow\"><div class=\"tablerow__title-wrapper\"><div class=\"tablerow__title\">Bonus</div><div class=\"tablerow__detail\"></div></div><div class=\"tablerow__value\">₹5k - ₹60k</div><div class=\"tablerow__bar\"><div class=\"tablerow__bar-range\" style=\"width:9%;margin-left:1%\"></div></div></div><div class=\"tablerow\"><div class=\"tablerow__title-wrapper\"><div class=\"tablerow__title\">Profit Sharing</div><div class=\"tablerow__detail\"></div></div><div class=\"tablerow__value\">₹41 - ₹198k</div><div class=\"tablerow__bar\"><div class=\"tablerow__bar-range\" style=\"width:34%;margin-left:0%\"></div></div></div><div class=\"tablerow\"><div class=\"tablerow__title-wrapper\"><div class=\"tablerow__title\">Commission</div><div class=\"tablerow__detail\"></div></div><div class=\"tablerow__value\">₹3k - ₹99k</div><div class=\"tablerow__bar\"><div class=\"tablerow__bar-range\" style=\"width:17%;margin-left:0%\"></div></div></div><div class=\"tablerow\"><div class=\"tablerow__title-wrapper\"><div class=\"tablerow__title\">Total Pay<span class=\"pxl-tooltip  \" data-tip=\"Total Pay combines base annual salary or hourly wage, bonuses, profit sharing, tips, commissions, overtime pay and other forms of cash earnings, as applicable for this job. It does not include equity (stock) compensation, cash value of retirement benefits, or the value of other non-cash benefits (e.g. healthcare).\" data-for=\"Total Pay\" currentitem=\"false\"><span class=\"icon-reverse icon-circle\"><i class=\"icon-info\"></i></span></span><div class=\"__react_component_tooltip t4fef1cf-42f8-4a54-9a83-f063f019fb85 place-right type-dark\" id=\"Total Pay\" data-id=\"tooltip\"><style>\n  \t.t4fef1cf-42f8-4a54-9a83-f063f019fb85 {\n\t    color: #fff;\n\t    background: #222;\n\t    border: 1px solid transparent;\n  \t}\n\n  \t.t4fef1cf-42f8-4a54-9a83-f063f019fb85.place-top {\n        margin-top: -10px;\n    }\n    .t4fef1cf-42f8-4a54-9a83-f063f019fb85.place-top::before {\n        border-top: 8px solid transparent;\n    }\n    .t4fef1cf-42f8-4a54-9a83-f063f019fb85.place-top::after {\n        border-left: 8px solid transparent;\n        border-right: 8px solid transparent;\n        bottom: -6px;\n        left: 50%;\n        margin-left: -8px;\n        border-top-color: #222;\n        border-top-style: solid;\n        border-top-width: 6px;\n    }\n\n    .t4fef1cf-42f8-4a54-9a83-f063f019fb85.place-bottom {\n        margin-top: 10px;\n    }\n    .t4fef1cf-42f8-4a54-9a83-f063f019fb85.place-bottom::before {\n        border-bottom: 8px solid transparent;\n    }\n    .t4fef1cf-42f8-4a54-9a83-f063f019fb85.place-bottom::after {\n        border-left: 8px solid transparent;\n        border-right: 8px solid transparent;\n        top: -6px;\n        left: 50%;\n        margin-left: -8px;\n        border-bottom-color: #222;\n        border-bottom-style: solid;\n        border-bottom-width: 6px;\n    }\n\n    .t4fef1cf-42f8-4a54-9a83-f063f019fb85.place-left {\n        margin-left: -10px;\n    }\n    .t4fef1cf-42f8-4a54-9a83-f063f019fb85.place-left::before {\n        border-left: 8px solid transparent;\n    }\n    .t4fef1cf-42f8-4a54-9a83-f063f019fb85.place-left::after {\n        border-top: 5px solid transparent;\n        border-bottom: 5px solid transparent;\n        right: -6px;\n        top: 50%;\n        margin-top: -4px;\n        border-left-color: #222;\n        border-left-style: solid;\n        border-left-width: 6px;\n    }\n\n    .t4fef1cf-42f8-4a54-9a83-f063f019fb85.place-right {\n        margin-left: 10px;\n    }\n    .t4fef1cf-42f8-4a54-9a83-f063f019fb85.place-right::before {\n        border-right: 8px solid transparent;\n    }\n    .t4fef1cf-42f8-4a54-9a83-f063f019fb85.place-right::after {\n        border-top: 5px solid transparent;\n        border-bottom: 5px solid transparent;\n        left: -6px;\n        top: 50%;\n        margin-top: -4px;\n        border-right-color: #222;\n        border-right-style: solid;\n        border-right-width: 6px;\n    }\n  </style></div></div><div class=\"tablerow__detail\"></div></div><div class=\"tablerow__value\">₹129k - ₹584k</div><div class=\"tablerow__bar\"><div class=\"tablerow__bar-range\" style=\"width:78%;margin-left:22%\"></div></div></div></div><div class=\"charttable__bottom\"><div class=\"charttable__footer\"><div class=\"charttable__footer-item\">Currency: INR<div class=\"charttable__footer-divider\">•</div></div><div class=\"charttable__footer-item\">Updated: Wed Feb 24 2021<div class=\"charttable__footer-divider\">•</div></div><div class=\"charttable__footer-item\">Individuals Reporting: 5,331</div></div></div></div><div class=\"paycharts__footer\">Based on 5,331 salary profiles <!-- -->(last updated<!-- --> <!-- -->Feb 24 2021<!-- -->)</div></div></div></section><section class=\"page-section\"><div id=\"\" class=\"single-row-start \"><div class=\"single-row-start__header\">Is Average Accountant Salary in India your job title? <!-- -->Find out what you should be paid<div class=\"single-row-start__sub-header\">Use our tool to get a personalized report on your market worth.<span class=\"single-row-start__highlights\">What's this?</span></div></div><form class=\"single-row-start__form survey-start__form\" method=\"post\" action=\"/wizards/choose.aspx?tk=singlerow-ps-rc-job\" __bizdiag=\"614252771\" __biza=\"W___\"><input type=\"hidden\" value=\"Mehdipatnam\" name=\"city\"><input type=\"hidden\" value=\"Telangana\" name=\"state\"><input type=\"hidden\" value=\"India\" name=\"confCountry\"><input type=\"hidden\" value=\"Mehdipatnam, Telangana\" name=\"location\"><input type=\"hidden\" value=\"\" name=\"job\"><input type=\"hidden\" value=\"\" name=\"yearsexp\"><div class=\"single-row-start__form-container\"><div id=\"\" class=\"pxl-form-group single-row-start__form-group single-row-start__form-group--location\"><label class=\" pxl-control-label\" for=\"\">Location:</label><div class=\"survey-start__citystate\"><div role=\"combobox\" aria-haspopup=\"listbox\" aria-owns=\"react-autowhatever-CityState\" aria-expanded=\"false\" class=\"react-autosuggest__container\"><input type=\"text\" value=\"Mehdipatnam, Telangana\" autocomplete=\"off\" aria-autocomplete=\"list\" aria-controls=\"react-autowhatever-CityState\" class=\"pxl-form-control survey-start__citystate-ac\" tabindex=\"0\" id=\"CityState\"><div id=\"react-autowhatever-CityState\" role=\"listbox\" class=\"react-autosuggest__suggestions-container\"></div></div></div></div><div class=\"survey-start__country\"><div class=\"survey-start__country-label\">India<!-- --> <a class=\"survey-start__country-link\">(change)</a></div><div role=\"combobox\" aria-haspopup=\"listbox\" aria-owns=\"react-autowhatever-Country\" aria-expanded=\"false\" class=\"react-autosuggest__container\"><input type=\"text\" value=\"India\" autocomplete=\"off\" aria-autocomplete=\"list\" aria-controls=\"react-autowhatever-Country\" class=\"pxl-form-control survey-start__country-ac survey-start--hidden\" id=\"Country\"><div id=\"react-autowhatever-Country\" role=\"listbox\" class=\"react-autosuggest__suggestions-container\"></div></div></div><div id=\"\" class=\"pxl-form-group single-row-start__form-group single-row-start__form-group--yoe\"><label class=\" pxl-control-label\" for=\"\">Years in Field/Career:</label><div class=\"pxl-input-number\"><input type=\"number\" class=\"survey-start__yearsexp pxl-input-number__input pxl-form-control\" value=\"\" min=\"0\" max=\"80\" step=\"any\" tabindex=\"0\"></div><div class=\"single-row-start__btn\"><input type=\"submit\" class=\"single-row-start__btn-input\" value=\"Find your market worth »\"></div></div></div><div class=\"single-row-start__right-bullets\"><div class=\"single-row-start__header\">How it works:</div><div class=\"single-row-start__right-bullets-list\"><div class=\"single-row-start__right-bullets-item\"><span>1</span><div>Enter city &amp; years of experience</div></div><div class=\"single-row-start__right-bullets-item\"><span>2</span><div>Add pay factors like skills &amp; education</div></div><div class=\"single-row-start__right-bullets-item\"><span>3</span><div>Find your market worth with a report tailored to you</div></div></div></div></form></div></section><section class=\"page-section\"><div class=\"explore-dropdowns-label\">EXPLORE BY:</div><div class=\"dropdowns\"><div class=\"dropdowns__dropdown dropdowns__dropdown--city\"><div class=\"pxl-dropdown   \"><button class=\"pxl-btn dropdown pxl-dropdown__toggle\" type=\"button\" id=\"\" data-toggle=\"dropdown\" aria-haspopup=\"true\" aria-expanded=\"false\">City</button><div class=\"pxl-dropdown__menu\" aria-labelledby=\"\"><a class=\"dropdowns__row\" href=\"/research/IN/Job=Accountant/Salary/24a57e4d/Mumbai\">Mumbai</a><a class=\"dropdowns__row\" href=\"/research/IN/Job=Accountant/Salary/8ffcc4f2/New-Delhi\">New Delhi</a><a class=\"dropdowns__row\" href=\"/research/IN/Job=Accountant/Salary/17594d0f/Bangalore\">Bangalore</a><a class=\"dropdowns__row\" href=\"/research/IN/Job=Accountant/Salary/d2b4b9ae/Pune\">Pune</a><a class=\"dropdowns__row\" href=\"/research/IN/Job=Accountant/Salary/66d63eb1/Chennai\">Chennai</a><a class=\"dropdowns__row\" href=\"/research/IN/Job=Accountant/Salary/e511fdf2/Kolkata\">Kolkata</a><a class=\"dropdowns__row\" href=\"/research/IN/Job=Accountant/Salary/9a8093f3/Delhi\">Delhi</a><a class=\"dropdowns__row\" href=\"/research/IN/Job=Accountant/Salary/20cb79cf/Hyderabad\">Hyderabad</a><a class=\"dropdowns__row\" href=\"/research/IN/Job=Accountant/Salary/2be3e760/Ahmedabad\">Ahmedabad</a><a class=\"dropdowns__row\" href=\"/research/IN/Job=Accountant/Salary/53d1a23e/Jaipur\">Jaipur</a><a class=\"dropdowns__row\" href=\"/research/IN/Job=Accountant/City\">See All Cities</a><div class=\"dropdowns__dropdown--survey-start\"><a href=\"/wizards/choose.aspx?tk=dropdown-ps-rc-job\">Don't see what you are looking for?<br><strong>Get A Free Custom Salary Report »</strong></a></div></div></div></div><div class=\"dropdowns__dropdown dropdowns__dropdown--experience\"><div class=\"pxl-dropdown   \"><button class=\"pxl-btn dropdown pxl-dropdown__toggle\" type=\"button\" id=\"\" data-toggle=\"dropdown\" aria-haspopup=\"true\" aria-expanded=\"false\">Experience</button><div class=\"pxl-dropdown__menu\" aria-labelledby=\"\"><a class=\"dropdowns__row\" href=\"/research/IN/Job=Accountant/Salary/71120446/Entry-Level\">Entry Level</a><a class=\"dropdowns__row\" href=\"/research/IN/Job=Accountant/Salary/71120446/Early-Career\">Early Career</a><a class=\"dropdowns__row\" href=\"/research/IN/Job=Accountant/Salary/26cf0477/Mid-Career\">Mid Career</a><a class=\"dropdowns__row\" href=\"/research/IN/Job=Accountant/Salary/76750e49/Experienced\">Experienced</a><a class=\"dropdowns__row\" href=\"/research/IN/Job=Accountant/Salary/f7365459/Late-Career\">Late Career</a><div class=\"dropdowns__dropdown--survey-start\"><a href=\"/wizards/choose.aspx?tk=dropdown-ps-rc-job\">Don't see what you are looking for?<br><strong>Get A Free Custom Salary Report »</strong></a></div></div></div></div><div class=\"dropdowns__dropdown dropdowns__dropdown--skill\"><div class=\"pxl-dropdown   \"><button class=\"pxl-btn dropdown pxl-dropdown__toggle\" type=\"button\" id=\"\" data-toggle=\"dropdown\" aria-haspopup=\"true\" aria-expanded=\"false\">Skill</button><div class=\"pxl-dropdown__menu\" aria-labelledby=\"\"><a class=\"dropdowns__row\" href=\"/research/IN/Job=Accountant/Salary/ebd17157/General-Ledger-Accounting\">General Ledger Accounting</a><a class=\"dropdowns__row\" href=\"/research/IN/Job=Accountant/Salary/35a6591b/Accounts-Payable\">Accounts Payable</a><a class=\"dropdowns__row\" href=\"/research/IN/Job=Accountant/Salary/dc47b68a/Microsoft-Excel\">Microsoft Excel</a><a class=\"dropdowns__row\" href=\"/research/IN/Job=Accountant/Salary/d440a488/Bookkeeping\">Bookkeeping</a><a class=\"dropdowns__row\" href=\"/research/IN/Job=Accountant/Salary/60f5899c/Financial-Reporting\">Financial Reporting</a><a class=\"dropdowns__row\" href=\"/research/IN/Job=Accountant/Salary/e8eeb682/Accounts-Receivable\">Accounts Receivable</a><a class=\"dropdowns__row\" href=\"/research/IN/Job=Accountant/Salary/5daf93b6/Account-Management\">Account Management</a><a class=\"dropdowns__row\" href=\"/research/IN/Job=Accountant/Salary/7c2820eb/Tax-Preparation\">Tax Preparation</a><a class=\"dropdowns__row\" href=\"/research/IN/Job=Accountant/Salary/0699604f/Sales-Tax-Reports\">Sales Tax Reports</a><a class=\"dropdowns__row\" href=\"/research/IN/Job=Accountant/Salary/9a4745f8/Microsoft-Office\">Microsoft Office</a><a class=\"dropdowns__row\" href=\"/research/IN/Job=Accountant/Skill\">See All Skills</a><div class=\"dropdowns__dropdown--survey-start\"><a href=\"/wizards/choose.aspx?tk=dropdown-ps-rc-job\">Don't see what you are looking for?<br><strong>Get A Free Custom Salary Report »</strong></a></div></div></div></div><div class=\"dropdowns__dropdown dropdowns__dropdown--employer\"><div class=\"pxl-dropdown   \"><button class=\"pxl-btn dropdown pxl-dropdown__toggle\" type=\"button\" id=\"\" data-toggle=\"dropdown\" aria-haspopup=\"true\" aria-expanded=\"false\">Employer</button><div class=\"pxl-dropdown__menu\" aria-labelledby=\"\"><a class=\"dropdowns__row\" href=\"/research/IN/Job=Accountant/Salary/b6f5ce68/Genpact\">Genpact</a><a class=\"dropdowns__row\" href=\"/research/IN/Job=Accountant/Salary/d9543e73/Infosys-BPO\">Infosys BPO</a><a class=\"dropdowns__row\" href=\"/research/IN/Job=Accountant/Salary/d2735610/Accountants-Inc.\">Accountants, Inc.</a><a class=\"dropdowns__row\" href=\"/research/IN/Job=Accountant/Salary/3d28b805/Abc-Corp.\">Abc Corp.</a><a class=\"dropdowns__row\" href=\"/research/IN/Job=Accountant/Salary/5c209f29/Any-Company\">Any Company</a><a class=\"dropdowns__row\" href=\"/research/IN/Job=Accountant/Salary/8b34aa19/Larsen-%26-Toubro-Limited\">Larsen &amp; Toubro Limited</a><a class=\"dropdowns__row\" href=\"/research/IN/Job=Accountant/Salary/2e1a871d/CA-Inc.\">CA, Inc.</a><a class=\"dropdowns__row\" href=\"/research/IN/Job=Accountant/Salary/d3e83f95/Capgemini\">Capgemini</a><a class=\"dropdowns__row\" href=\"/research/IN/Job=Accountant/Salary/38566bc7/Sobha-Developers-Ltd\">Sobha Developers Ltd</a><a class=\"dropdowns__row\" href=\"/research/IN/Job=Accountant/Salary/e746cf4b/Tata-Motors\">Tata Motors</a><a class=\"dropdowns__row\" href=\"/research/IN/Job=Accountant/Employer\">See All Employers</a><div class=\"dropdowns__dropdown--survey-start\"><a href=\"/wizards/choose.aspx?tk=dropdown-ps-rc-job\">Don't see what you are looking for?<br><strong>Get A Free Custom Salary Report »</strong></a></div></div></div></div><div class=\"dropdowns__dropdown dropdowns__dropdown--job\"><div class=\"pxl-dropdown   \"><button class=\"pxl-btn dropdown pxl-dropdown__toggle\" type=\"button\" id=\"\" data-toggle=\"dropdown\" aria-haspopup=\"true\" aria-expanded=\"false\">Job</button><div class=\"pxl-dropdown__menu\" aria-labelledby=\"\"><a class=\"dropdowns__row\" href=\"/research/IN/Job=Chartered_Accountant/Salary\">Chartered Accountant</a><a class=\"dropdowns__row\" href=\"/research/IN/Job=Financial_Analyst/Salary\">Financial Analyst</a><a class=\"dropdowns__row\" href=\"/research/IN/Job=Finance_Manager/Salary\">Finance Manager</a><a class=\"dropdowns__row\" href=\"/research/IN/Job=Data_Analyst/Salary\">Data Analyst</a><a class=\"dropdowns__row\" href=\"/research/IN/Job=Civil_Engineer/Salary\">Civil Engineer</a><a class=\"dropdowns__row\" href=\"/research/IN/Job=Account_Executive/Salary\">Account Executive</a><a class=\"dropdowns__row\" href=\"/research/IN/Job=Senior_Accountant/Salary\">Senior Accountant</a><a class=\"dropdowns__row\" href=\"/research/IN/Job=Mechanical_Engineer/Salary\">Mechanical Engineer</a><a class=\"dropdowns__row\" href=\"/research/IN/Job=Data_Entry_Operator/Salary\">Data Entry Operator</a><a class=\"dropdowns__row\" href=\"/research/IN/Job=Assistant_Finance_Manager/Salary\">Assistant Finance Manager</a><div class=\"dropdowns__dropdown--survey-start\"><a href=\"/wizards/choose.aspx?tk=dropdown-ps-rc-job\">Don't see what you are looking for?<br><strong>Get A Free Custom Salary Report »</strong></a></div></div></div></div></div></section><section class=\"page-section\"><div class=\"featured-content\"><h2 class=\"featured-content__header\">Featured Content</h2><div class=\"carousel \"><div class=\"carousel__content\" style=\"width:810px\"><div class=\"carousel__arrow carousel__arrow--left carousel__arrow--invisible\"><span>‹</span></div><div class=\"carousel__items-window\"><div class=\"carousel__items\"><a class=\"blog-card \" href=\"https://www.payscale.com/data/remote-work?tk=carousel-ps-rc-job\"><div class=\"blog-card__content\"><div class=\"blog-card__image\" style=\"background-image:url(https://www.payscale.com/wp-content/uploads/2020/08/Remote-Work-Feature-Image.png)\"></div><div class=\"blog-card__divider\"></div><div class=\"blog-card__detail\"><div class=\"blog-card__title\">Remote Work</div><div class=\"blog-card__description\">New research shows how to set pay for remote employees</div></div></div></a><a class=\"blog-card \" href=\"https://www.payscale.com/data/gender-pay-gap?tk=carousel-ps-rc-job\"><div class=\"blog-card__content\"><div class=\"blog-card__image\" style=\"background-image:url(https://www.payscale.com/wp-content/uploads/2020/03/GPG2020-Promo01.jpg)\"></div><div class=\"blog-card__divider\"></div><div class=\"blog-card__detail\"><div class=\"blog-card__title\">Gender Pay Gap</div><div class=\"blog-card__description\">New research shows that each woman experiences the disparity of gender pay gap in different ways, depending on her position, age, race and education.</div></div></div></a><a class=\"blog-card \" href=\"https://www.payscale.com/cbpr?tk=carousel-ps-rc-job\"><div class=\"blog-card__content\"><div class=\"blog-card__image\" style=\"background-image:url(https://www.payscale.com/wp-content/uploads/2020/03/CBPR2020-Social-490x330-1.jpg)\"></div><div class=\"blog-card__divider\"></div><div class=\"blog-card__detail\"><div class=\"blog-card__title\">Compensation Best Practices Report</div><div class=\"blog-card__description\">From compensation planning to variable pay to pay equity analysis, we surveyed 4,900+ organizations on how they manage compensation.</div></div></div></a><a class=\"blog-card \" href=\"https://www.payscale.com/data/why-people-quit-their-jobs?tk=carousel-ps-rc-job\"><div class=\"blog-card__content\"><div class=\"blog-card__image\" style=\"background-image:url(https://www.payscale.com/wp-content/uploads/2019/05/QuitYou_HP.jpg)\"></div><div class=\"blog-card__divider\"></div><div class=\"blog-card__detail\"><div class=\"blog-card__title\">Why people quit their jobs</div><div class=\"blog-card__description\">Why do people leave their jobs? We take a deep dive into what's impacting employee retention and what employees are looking for in their new role.</div></div></div></a><a class=\"blog-card \" href=\"https://www.payscale.com/data/how-to-ask-for-a-raise?tk=carousel-ps-rc-job\"><div class=\"blog-card__content\"><div class=\"blog-card__image\" style=\"background-image:url(https://www.payscale.com/wp-content/uploads/2018/10/raise-anatomy.jpg)\"></div><div class=\"blog-card__divider\"></div><div class=\"blog-card__detail\"><div class=\"blog-card__title\">How to ask for a raise</div><div class=\"blog-card__description\">New research on who's asking for raises and who's getting them as well as advice on how to ensure you're getting the salary you deserve.</div></div></div></a><a class=\"blog-card \" href=\"https://www.payscale.com/hr/variable-pay-playbook?tk=carousel-ps-rc-job\"><div class=\"blog-card__content\"><div class=\"blog-card__image\" style=\"background-image:url(https://www.payscale.com/wp-content/uploads/2018/10/Variable-Pay-Playbook.png)\"></div><div class=\"blog-card__divider\"></div><div class=\"blog-card__detail\"><div class=\"blog-card__title\">Variable Pay Playbook</div><div class=\"blog-card__description\">Before you decide whether variable pay is right for your org, get a deeper understanding of the variable pay options and the cultural impact of pay choices.</div></div></div></a></div></div><div class=\"carousel__arrow carousel__arrow--right \"><span>›</span></div></div><div class=\"carousel__indicator__window\"><div class=\"carousel__indicator__dots\" style=\"transform: translate(0px);\"><div class=\"carousel__indicator__dot__container\"><div class=\"carousel__indicator__dot selected\"></div></div><div class=\"carousel__indicator__dot__container\"><div class=\"carousel__indicator__dot \"></div></div><div class=\"carousel__indicator__dot__container\"><div class=\"carousel__indicator__dot small\"></div></div><div class=\"carousel__indicator__dot__container\"><div class=\"carousel__indicator__dot xsmall\"></div></div><div class=\"carousel__indicator__dot__container\"><div class=\"carousel__indicator__dot xsmall\"></div></div><div class=\"carousel__indicator__dot__container\"><div class=\"carousel__indicator__dot xsmall\"></div></div></div></div></div><div class=\"newsletter-form\"><form class=\"\" action=\"https://payscale.us3.list-manage.com/subscribe/post?u=0350e593d83b212345ced805c&amp;amp;id=cc02db4bff\" method=\"post\" id=\"mc-embedded-subscribe-form\" name=\"mc-embedded-subscribe-form\" target=\"_blank\" novalidate=\"\" __bizdiag=\"59632160\" __biza=\"W___\"><label for=\"mce-EMAIL\"><div class=\"newsletter-form__title\">Subscribe to our newsletter</div></label><div class=\"newsletter-form__container\"><div class=\"newsletter-form__email-container\"><input type=\"email\" value=\"\" name=\"EMAIL\" class=\"newsletter-form__email-input pxl-form-control\" id=\"mce-EMAIL\" placeholder=\"Email address\" required=\"\"><div style=\"position:absolute;left:-5000px\" aria-hidden=\"true\"><input type=\"text\" name=\"b_0350e593d83b212345ced805c_cc02db4bff\" tabindex=\"-1\" value=\"\"></div><div class=\"newsletter-form__email-error\" style=\"visibility:hidden\">Enter a valid email address</div></div><input type=\"submit\" value=\"Subscribe\" name=\"subscribe\" class=\"newsletter-form__submit\"></div></form></div></div></section><section class=\"page-section\"><h2 class=\"page-subtitle\">What is the Pay by Experience Level for <!-- -->Accountants<!-- -->?</h2><div class=\"pay-by-experience\"><div class=\"pay-by-experience__charts\"><div class=\"pay-by-experience__experience-chart-container\"><div style=\"width:100%;height:100%\"><div style=\"position:relative;width:inherit;height:inherit\"><div class=\"lazyload-wrapper\"><div class=\"lazyload-placeholder\" style=\"height: 300px;\"></div></div></div></div></div><div class=\"pay-by-experience__delta-container\"><div class=\"delta-table pay-by-experience__delta-chart\"><div class=\"entry\"><div class=\"name\"><a href=\"/research/IN/Job=Accountant/Salary/71120446/Entry-Level\">Entry Level</a></div><div class=\"value\"></div><div class=\"arrow\"><span class=\"negative\">▼</span>20%</div></div><div class=\"entry\"><div class=\"name\"><a href=\"/research/IN/Job=Accountant/Salary/71120446/Early-Career\">Early Career</a></div><div class=\"value\"></div><div class=\"arrow\"><span class=\"negative\">▼</span>7%</div></div><div class=\"entry\"><div class=\"name\"><a href=\"/research/IN/Job=Accountant/Salary/26cf0477/Mid-Career\">Mid Career</a></div><div class=\"value\"></div><div class=\"arrow\"><span class=\"positive\">▲</span>19%</div></div><div class=\"entry\"><div class=\"name\"><a href=\"/research/IN/Job=Accountant/Salary/76750e49/Experienced\">Late Career</a></div><div class=\"value\"></div><div class=\"arrow\"><span class=\"positive\">▲</span>42%</div></div><div class=\"entry\"><div class=\"name\"><a href=\"/research/IN/Job=Accountant/Salary/f7365459/Late-Career\">Experienced</a></div><div class=\"value\"></div><div class=\"arrow\"><span class=\"positive\">▲</span>58%</div></div></div></div></div><div class=\"expandable pay-by-experience__blurb\"><p class=\"expandable__paragraph\">An entry-level Accountant with less than 1 year experience can expect to earn an average total compensation (includes tips, bonus, and overtime pay) of ₹198,740 based on 654 salaries. An early career Accountant with 1-4 years of experience earns an average total compensation of ₹231,272 based on 3,453 salaries. A mid-career<span class=\"expandable__more\"><span>  <!-- -->…Read more</span></span></p></div></div></section><div class=\"page-section description\"><h2 class=\"page-subtitle\">What Do <!-- -->Accountants<!-- --> Do?</h2><div class=\"job-tasks-container\"><div class=\"expandable \"><p class=\"expandable__paragraph\">Accountants perform financial calculations for companies in a wide variety of fields. Some common duties include creating sales and cash flow reports, administering payroll, keeping balance sheets, carrying out billing activities, managing budgets and keeping inventory. The accountant may also be responsible for filing taxes for the company, as well as reviewing past reports to generate income forecasts.</p><p class=\"expandable__paragraph\">Occasionally, internal audits must be carried out to make sure that the various areas of<span class=\"expandable__more\"><span>  <!-- -->…Read more</span></span></p></div></div></div><section class=\"page-section\"><div class=\"how-it-works \"><div class=\"how-it-works__header\">Find your market worth –&nbsp;how it works:</div><div class=\"how-it-works__graphics\"><div class=\"how-it-works__graphics-group\"><img src=\"//cdn-payscale.com/content/how-it-works-job-details-icon@2x.png\" alt=\"Tell us about your job and pay factors like skills and education\"><span>Tell us about your job and pay factors like skills and education</span></div><div class=\"how-it-works__graphics-group\"><img src=\"//cdn-payscale.com/content/how-it-works-market-worth-icon@2x.png\" alt=\"Find your market worth with a report tailored to you\"><span>Find your market worth with a report tailored to you</span></div><div class=\"how-it-works__graphics-group\"><img src=\"//cdn-payscale.com/content/how-it-works-negotiate-icon@2x.png\" alt=\"Negotiate your pay with confidence\"><span>Negotiate your pay with confidence</span></div></div><form class=\"how-it-works__form survey-start__form\" method=\"post\" action=\"/wizards/choose.aspx?tk=how-it-works-ps-rc-job\" __bizdiag=\"614252771\" __biza=\"W___\"><input type=\"hidden\" value=\"Mehdipatnam\" name=\"city\"><input type=\"hidden\" value=\"Telangana\" name=\"state\"><input type=\"hidden\" value=\"India\" name=\"confCountry\"><input type=\"hidden\" value=\"Mehdipatnam, Telangana\" name=\"location\"><input type=\"hidden\" value=\"\" name=\"job\"><input type=\"hidden\" value=\"\" name=\"yearsexp\"><div id=\"\" class=\"pxl-form-group how-it-works__form-job\"><label class=\" pxl-control-label\" for=\"\">Enter your job title:</label><input type=\"text\" class=\" pxl-form-control\" value=\"\" placeholder=\"\" tabindex=\"0\"></div><div class=\"how-it-works__form-btn\"><input type=\"submit\" class=\"how-it-works__btn-input\" value=\"Get your salary report »\"></div><div class=\"how-it-works__form-logo\"><img src=\"//cdn-payscale.com/content/payscale_logo.svg\" alt=\"payscale logo\"></div></form></div></section><div id=\"div-gpt-ad-22104046045-0\" class=\"ad-slot\" data-google-query-id=\"CO79vLzZpe8CFUUTtwAdlLkK2w\"><div id=\"google_ads_iframe_/1063747/PS_RC_Content_Fluid_0__container__\" style=\"border: 0pt none;\"><iframe id=\"google_ads_iframe_/1063747/PS_RC_Content_Fluid_0\" title=\"3rd party ad content\" name=\"google_ads_iframe_/1063747/PS_RC_Content_Fluid_0\" width=\"970\" height=\"90\" scrolling=\"no\" marginwidth=\"0\" marginheight=\"0\" frameborder=\"0\" sandbox=\"allow-forms allow-popups allow-popups-to-escape-sandbox allow-same-origin allow-scripts allow-top-navigation-by-user-activation\" srcdoc=\"\" style=\"border: 0px; vertical-align: bottom;\" data-google-container-id=\"1\" data-load-complete=\"true\"></iframe></div></div><section class=\"page-section\"><div class=\"jobstats\"><div class=\"jobstats__data\"><div class=\"gender\"><h2 class=\"page-subtitle\">Gender Breakdown</h2><div class=\"gender__data\"><div class=\"gender__item\"><div class=\"gender__label\">Male</div><div class=\"gender__value\">78.5<!-- -->%</div></div><div class=\"gender__item\"><div class=\"gender__label\">Female</div><div class=\"gender__value\">21.4<!-- -->%</div></div><div class=\"gender__item\"><div class=\"gender__label\">Prefer to self-define</div><div class=\"gender__value\">0.1<!-- -->%</div></div></div><div class=\"gender__blurb\">This data is based on <!-- -->1,476<!-- --> survey responses. Learn more about<!-- --> <a href=\"/data-packages/gender-pay-gap?tk=genderwidget-ps-rc-job\">the gender pay gap</a>.</div></div><div class=\"healthbenefits\"><h2 class=\"page-subtitle\">Common Health Benefits</h2><div class=\"healthbenefits__data\"><div class=\"healthbenefits__item\"><div class=\"healthbenefits__item-name\">Medical</div><h2 class=\"healthbenefits__item-value\">34<!-- -->%</h2></div><div class=\"healthbenefits__item\"><div class=\"healthbenefits__item-name\">Dental</div><h2 class=\"healthbenefits__item-value\">3<!-- -->%</h2></div><div class=\"healthbenefits__item\"><div class=\"healthbenefits__item-name\">Vision</div><h2 class=\"healthbenefits__item-value\">3<!-- -->%</h2></div><div class=\"healthbenefits__item\"><div class=\"healthbenefits__item-name\">None</div><h2 class=\"healthbenefits__item-value\">66<!-- -->%</h2></div></div></div></div></div></section><div class=\"lazysection-wrapper\"><div style=\"height: 1397px;\"><div class=\"skeleton page-section\"><div class=\"skeleton__content\"><div class=\"skeleton__content--title\"></div><ul class=\"\" style=\"list-style: none;\"><li class=\"skeleton__content--paragraph\"></li><li class=\"skeleton__content--paragraph\"></li><li class=\"skeleton__content--paragraph\"></li><li class=\"skeleton__content--paragraph\"></li></ul></div></div><div class=\"skeleton page-section\"><div class=\"skeleton__content\"><div class=\"skeleton__content--title\"></div><ul class=\"\" style=\"list-style: none;\"><li class=\"skeleton__content--paragraph\"></li><li class=\"skeleton__content--paragraph\"></li><li class=\"skeleton__content--paragraph\"></li><li class=\"skeleton__content--paragraph\"></li></ul></div></div><div class=\"skeleton page-section\"><div class=\"skeleton__content\"><div class=\"skeleton__content--title\"></div><ul class=\"\" style=\"list-style: none;\"><li class=\"skeleton__content--paragraph\"></li><li class=\"skeleton__content--paragraph\"></li><li class=\"skeleton__content--paragraph\"></li><li class=\"skeleton__content--paragraph\"></li></ul></div></div></div></div><div class=\"lazysection-wrapper\"><div style=\"height: 693px;\"><div class=\"skeleton page-section\"><div class=\"skeleton__content\"><div class=\"skeleton__content--title\"></div><ul class=\"\" style=\"list-style: none;\"><li class=\"skeleton__content--paragraph\"></li><li class=\"skeleton__content--paragraph\"></li><li class=\"skeleton__content--paragraph\"></li><li class=\"skeleton__content--paragraph\"></li></ul></div></div><div class=\"skeleton page-section\"><div class=\"skeleton__content\"><div class=\"skeleton__content--title\"></div><ul class=\"\" style=\"list-style: none;\"><li class=\"skeleton__content--paragraph\"></li><li class=\"skeleton__content--paragraph\"></li><li class=\"skeleton__content--paragraph\"></li><li class=\"skeleton__content--paragraph\"></li></ul></div></div><div class=\"skeleton page-section\"><div class=\"skeleton__content\"><div class=\"skeleton__content--title\"></div><ul class=\"\" style=\"list-style: none;\"><li class=\"skeleton__content--paragraph\"></li><li class=\"skeleton__content--paragraph\"></li><li class=\"skeleton__content--paragraph\"></li><li class=\"skeleton__content--paragraph\"></li></ul></div></div></div></div><div class=\"lazysection-wrapper\"><div style=\"height: 555px;\"><div class=\"skeleton page-section\"><div class=\"skeleton__content\"><div class=\"skeleton__content--title\"></div><ul class=\"\" style=\"list-style: none;\"><li class=\"skeleton__content--paragraph\"></li><li class=\"skeleton__content--paragraph\"></li><li class=\"skeleton__content--paragraph\"></li><li class=\"skeleton__content--paragraph\"></li></ul></div></div><div class=\"skeleton page-section\"><div class=\"skeleton__content\"><div class=\"skeleton__content--title\"></div><ul class=\"\" style=\"list-style: none;\"><li class=\"skeleton__content--paragraph\"></li><li class=\"skeleton__content--paragraph\"></li><li class=\"skeleton__content--paragraph\"></li><li class=\"skeleton__content--paragraph\"></li></ul></div></div><div class=\"skeleton page-section\"><div class=\"skeleton__content\"><div class=\"skeleton__content--title\"></div><ul class=\"\" style=\"list-style: none;\"><li class=\"skeleton__content--paragraph\"></li><li class=\"skeleton__content--paragraph\"></li><li class=\"skeleton__content--paragraph\"></li><li class=\"skeleton__content--paragraph\"></li></ul></div></div></div></div></div></div><div id=\"footer-survey-start\" class=\"footer-survey-start \"><div class=\"footer-survey-start__header\">WHAT AM I WORTH?</div><div class=\"footer-survey-start__subheader\">What your skills are worth in the job market is constantly changing.</div><form class=\"footer-survey-start__form survey-start__form\" method=\"post\" action=\"/wizards/choose.aspx?tk=footer-ps-rc-job\" __bizdiag=\"614252771\" __biza=\"W___\"><input type=\"hidden\" value=\"Mehdipatnam\" name=\"city\"><input type=\"hidden\" value=\"Telangana\" name=\"state\"><input type=\"hidden\" value=\"India\" name=\"confCountry\"><input type=\"hidden\" value=\"Mehdipatnam, Telangana\" name=\"location\"><input type=\"hidden\" value=\"\" name=\"job\"><input type=\"hidden\" value=\"\" name=\"yearsexp\"><div id=\"\" class=\"pxl-form-group footer-survey-start__form-group\"><label class=\" pxl-control-label\" for=\"\">Job Title:</label><input type=\"text\" class=\" pxl-form-control\" value=\"\" placeholder=\"\" tabindex=\"0\"></div><div id=\"\" class=\"pxl-form-group footer-survey-start__form-group footer-survey-start__location\"><label class=\" pxl-control-label\" for=\"\">Location:</label><div class=\"survey-start__location\"><div class=\"survey-start__citystate\"><div role=\"combobox\" aria-haspopup=\"listbox\" aria-owns=\"react-autowhatever-footer-survey-startCityState\" aria-expanded=\"false\" class=\"react-autosuggest__container\"><input type=\"text\" value=\"Mehdipatnam, Telangana\" autocomplete=\"off\" aria-autocomplete=\"list\" aria-controls=\"react-autowhatever-footer-survey-startCityState\" class=\"pxl-form-control survey-start__citystate-ac\" tabindex=\"0\" id=\"footer-survey-startCityState\"><div id=\"react-autowhatever-footer-survey-startCityState\" role=\"listbox\" class=\"react-autosuggest__suggestions-container\"></div></div></div><div class=\"survey-start__country\"><div class=\"survey-start__country-label\">India<!-- --> <a class=\"survey-start__country-link\">(change)</a></div><div role=\"combobox\" aria-haspopup=\"listbox\" aria-owns=\"react-autowhatever-footer-survey-startCountry\" aria-expanded=\"false\" class=\"react-autosuggest__container\"><input type=\"text\" value=\"India\" autocomplete=\"off\" aria-autocomplete=\"list\" aria-controls=\"react-autowhatever-footer-survey-startCountry\" class=\"pxl-form-control survey-start__country-ac survey-start--hidden\" id=\"footer-survey-startCountry\"><div id=\"react-autowhatever-footer-survey-startCountry\" role=\"listbox\" class=\"react-autosuggest__suggestions-container\"></div></div></div></div></div><div class=\"footer-survey-start__btn\"><input type=\"submit\" class=\"footer-survey-start__btn-input\" value=\"Get your salary report »\"></div></form></div><footer id=\"pxl-footer\" class=\"pxl-footer\"><div class=\"pxl-footer__content\"><div class=\"pxl-footer__links\"><div class=\"pxl-footer__toplinks\"><a href=\"/about\" class=\"pxl-footer__toplink-item\">About Us</a><a href=\"/about/press-releases\" class=\"pxl-footer__toplink-item\">Press Center</a><a href=\"/about/jobs\" class=\"pxl-footer__toplink-item\">Work With Us</a><a href=\"/about/methodology\" class=\"pxl-footer__toplink-item\">Methodology</a><a href=\"/about/contact\" class=\"pxl-footer__toplink-item\">Contact Us</a></div><div class=\"pxl-footer__sublinks\"><a href=\"/research/IN/Country=India/Salary\" class=\"pxl-footer__sublinks-title\">Research Careers:</a><a class=\"pxl-footer__sublink-item\" href=\"/research/IN/Job\">Jobs</a><a class=\"pxl-footer__sublink-item\" href=\"/research/IN/Employer\">Employers</a><a class=\"pxl-footer__sublink-item\" href=\"/research/IN/Degree\">Degrees</a></div></div><div class=\"pxl-footer__logos\"><img src=\"//cdn-payscale.com/content/Payscale_Icon_White.svg\" alt=\"PayScale\"></div><div class=\"pxl-footer__social\"><a class=\"pxl-footer__social-link\" href=\"https://www.facebook.com/PayScale/\" rel=\"nofollow\"><img src=\"//cdn-payscale.com/content/Facebook_Icon.svg\" height=\"15px\" alt=\"Facebook\"></a><a class=\"pxl-footer__social-link\" href=\"https://www.linkedin.com/company/payscale-inc-\" rel=\"nofollow\"><img src=\"//cdn-payscale.com/content/LinkedIn_Icon.svg\" height=\"15px\" alt=\"LinkedIn\"></a><a class=\"pxl-footer__social-link\" href=\"https://twitter.com/payscale\" rel=\"nofollow\"><img src=\"//cdn-payscale.com/content/Twitter_Icon.svg\" height=\"15px\" alt=\"Twitter\"></a><a class=\"pxl-footer__social-link\" href=\"https://www.instagram.com/payscale/\" rel=\"nofollow\"><img src=\"//cdn-payscale.com/content/Instagram_Icon.svg\" height=\"15px\" alt=\"Instagram\"></a></div></div><div class=\"pxl-footer__legal\"><div class=\"pxl-footer__copyright\">© <span class=\"pxl-footer__copyright-year\">2021</span> PayScale, Inc. All rights reserved.</div><ul class=\"pxl-footer__privacy\"><li class=\"pxl-footer__privacy-item\"><a class=\"pxl-footer__privacy-link\" href=\"/data-protection\">Data Protection</a></li><li class=\"pxl-footer__privacy-item\"><a class=\"pxl-footer__privacy-link\" href=\"/privacy\">Privacy Policy</a></li><li class=\"pxl-footer__privacy-item\"><a class=\"pxl-footer__privacy-link\" href=\"/license\">Terms of Use</a></li></ul></div></footer></div></div><script crossorigin=\"true\" src=\"https://unpkg.com/react@16.14.0/umd/react.production.min.js\"></script><img src=\"//www.payscale.com/track_hit.aspx?url=https%3A%2F%2Fwww.payscale.com%2Fresearch%2FIN%2FJob%3DAccountant%2FSalary&amp;referer=https%3A%2F%2Fwww.payscale.com%2Fresearch%2FIN%2FJob%2FAccounting-and-Finance\" width=\"1\" height=\"1\" alt=\"\" style=\"position: absolute;\"> \n<script type=\"text/javascript\" id=\"\">!function(b,e,f,g,a,c,d){b.fbq||(a=b.fbq=function(){a.callMethod?a.callMethod.apply(a,arguments):a.queue.push(arguments)},b._fbq||(b._fbq=a),a.push=a,a.loaded=!0,a.version=\"2.0\",a.queue=[],c=e.createElement(f),c.async=!0,c.src=g,d=e.getElementsByTagName(f)[0],d.parentNode.insertBefore(c,d))}(window,document,\"script\",\"https://connect.facebook.net/en_US/fbevents.js\");fbq(\"init\",\"1201912829856435\");fbq(\"track\",\"PageView\");</script>\n<noscript>\n<img height=\"1\" width=\"1\" src=\"https://www.facebook.com/tr?id=1201912829856435&amp;ev=PageView\n&amp;noscript=1\">\n</noscript>\n<script type=\"text/javascript\" id=\"\">var footerLinks=document.getElementsByClassName(\"pxl-footer__privacy\")[0];footerLinks&&(footerLinks.innerHTML+=\"\\x3cli class\\x3d'pxl-footer__privacy-item'\\x3e\\x3ca class\\x3d'pxl-footer__privacy-link' onclick\\x3d'OneTrust.ToggleInfoDisplay()'\\x3eCookie Preferences\\x3c/a\\x3e\\x3c/li\\x3e\");</script><script crossorigin=\"true\" src=\"https://unpkg.com/react-dom@16.14.0/umd/react-dom.production.min.js\"></script><script id=\"__NEXT_DATA__\" type=\"application/json\">{\"props\":{\"pageProps\":{\"reviewCount\":358,\"careerPathData\":{\"value\":{\"jobTitle\":\"Accountant\",\"url\":\"/research/IN/Job=Accountant/Salary\",\"compensation\":248962.51166742906,\"isHourly\":false,\"currency\":\"INR\"},\"childCount\":3,\"children\":[{\"index\":0,\"value\":{\"jobTitle\":\"Senior Accountant\",\"url\":\"/research/IN/Job=Senior_Accountant/Salary\",\"percent\":0.22,\"compensation\":401485.573886885,\"isHourly\":false,\"currency\":\"INR\"},\"childCount\":3,\"children\":[{\"index\":0,\"value\":{\"jobTitle\":\"Accountant\",\"url\":\"/research/IN/Job=Accountant/Salary\",\"percent\":0.19,\"compensation\":248962.51166742906,\"isHourly\":false,\"currency\":\"INR\"}},{\"index\":1,\"value\":{\"jobTitle\":\"Finance Manager\",\"url\":\"/research/IN/Job=Finance_Manager/Salary\",\"percent\":0.1,\"compensation\":995575.4969846683,\"isHourly\":false,\"currency\":\"INR\"}},{\"index\":2,\"value\":{\"jobTitle\":\"Accounting Manager\",\"url\":\"/research/IN/Job=Accounting_Manager/Salary\",\"percent\":0.07,\"compensation\":724312.1922305191,\"isHourly\":false,\"currency\":\"INR\"}}]},{\"index\":1,\"value\":{\"jobTitle\":\"Account Executive\",\"url\":\"/research/IN/Job=Account_Executive/Salary\",\"percent\":0.07,\"compensation\":264830.1309198816,\"isHourly\":false,\"currency\":\"INR\"},\"childCount\":3,\"children\":[{\"index\":0,\"value\":{\"jobTitle\":\"Accountant\",\"url\":\"/research/IN/Job=Accountant/Salary\",\"percent\":0.1,\"compensation\":248962.51166742906,\"isHourly\":false,\"currency\":\"INR\"}},{\"index\":1,\"value\":{\"jobTitle\":\"Senior Account Executive\",\"url\":\"/research/IN/Job=Senior_Account_Executive/Salary\",\"percent\":0.08,\"compensation\":403026.27286896686,\"isHourly\":false,\"currency\":\"INR\"}},{\"index\":2,\"value\":{\"jobTitle\":\"Account Manager\",\"url\":\"/research/IN/Job=Account_Manager/Salary\",\"percent\":0.06,\"compensation\":593987.7742145744,\"isHourly\":false,\"currency\":\"INR\"}}]},{\"index\":2,\"value\":{\"jobTitle\":\"Finance Manager\",\"url\":\"/research/IN/Job=Finance_Manager/Salary\",\"percent\":0.06,\"compensation\":995575.4969846683,\"isHourly\":false,\"currency\":\"INR\"},\"childCount\":3,\"children\":[{\"index\":0,\"value\":{\"jobTitle\":\"Senior Finance Manager\",\"url\":\"/research/IN/Job=Senior_Finance_Manager/Salary\",\"percent\":0.18,\"compensation\":1741929.3658245625,\"isHourly\":false,\"currency\":\"INR\"}},{\"index\":1,\"value\":{\"jobTitle\":\"Financial Controller\",\"url\":\"/research/IN/Job=Financial_Controller/Salary\",\"percent\":0.17,\"compensation\":1822557.6219829062,\"isHourly\":false,\"currency\":\"INR\"}},{\"index\":2,\"value\":{\"jobTitle\":\"Finance Director\",\"url\":\"/research/IN/Job=Finance_Director/Salary\",\"percent\":0.07,\"compensation\":3657014.1906842864,\"isHourly\":false,\"currency\":\"INR\"}}]}]},\"siteUrl\":\"https://www.payscale.com\",\"pageData\":{\"country\":\"IN\",\"category\":\"Job\",\"dimensions\":{\"job\":\"Accountant\"},\"multiDimensional\":false,\"reportType\":\"Salary\",\"refs\":{\"url\":\"/research/IN/Job=Accountant/Salary\",\"defaultUrl\":\"/research/IN/Job=Accountant/Salary\",\"parentUrl\":null},\"currencyCode\":\"INR\",\"lastUpdated\":\"2021-02-23T23:33:13.90966Z\",\"narratives\":{\"description\":\"Accountants perform financial calculations for companies in a wide variety of fields. Some common duties include creating sales and cash flow reports, administering payroll, keeping balance sheets, carrying out billing activities, managing budgets and keeping inventory. The accountant may also be responsible for filing taxes for the company, as well as reviewing past reports to generate income forecasts.\\r\\nOccasionally, internal audits must be carried out to make sure that the various areas of the company are performing as expected; the accountant must also make sure that staff members are adhering to company policies and relevant laws. The accountant should be able to create accurate, detailed reports to illustrate data; sometimes, these reports have to be presented to management. The accountant may oversee the financial transactions of one department or multiple departments within their organization. \\r\\nA bachelor's degree in accounting is required for this position, as is status as a certified public accountant (CPA). Previous accounting experience is generally required or preferred as well. Knowledge of accounting software such as Quickbooks and Microsoft Excel is needed. Additionally, many of the accountant's tasks are performed independently so it is essential to be self-motivated; however, collaboration is necessary, and the accountant must be able to work as part of a team. \\r\\n\",\"summaryHourly\":\"The average pay for an Accountant is ₹150 per hour.\",\"summarySalary\":\"The average pay for an Accountant is ₹249,659 per year.\"},\"tasks\":null,\"compensation\":{\"bonus\":{\"10\":4917.77,\"25\":9951.54,\"50\":18657.9,\"75\":31037.74,\"90\":59567.62,\"profileCount\":2258},\"commission\":{\"10\":2553.21,\"25\":9744.4,\"50\":20481.7,\"75\":57067.81,\"90\":99329.31,\"profileCount\":40},\"hourlyRate\":{\"10\":47.85,\"25\":81.62,\"50\":149.5,\"75\":490.68,\"90\":1012.37,\"profileCount\":159},\"profitSharing\":{\"10\":40.84,\"25\":3051.89,\"50\":15392.01,\"75\":50114.46,\"90\":197672.24,\"profileCount\":528},\"salary\":{\"10\":123469.27,\"25\":182393.41,\"50\":249659.22,\"75\":354995.71,\"90\":506338.08,\"profileCount\":5172},\"total\":{\"10\":128947.05,\"25\":188425.71,\"50\":265158.91,\"75\":384549.98,\"90\":584411.15,\"profileCount\":5331}},\"byDimension\":{\"Average EAC Overall\":{\"lastUpdated\":\"2021-02-23T23:33:13.90966Z\",\"profileCount\":5331,\"description\":\"Effective Annual Compensation\",\"rows\":[{\"name\":\"Main\",\"displayName\":null,\"url\":null,\"profileCount\":5331,\"range\":{\"10\":128947.05,\"25\":188425.71,\"50\":265158.91,\"75\":384549.98,\"90\":584411.15},\"isEstimated\":false}],\"parentUrl\":null},\"Average Hourly Rate Overall\":{\"lastUpdated\":\"2021-02-23T23:33:13.90966Z\",\"profileCount\":159,\"description\":\"Hourly Rate\",\"rows\":[{\"name\":\"Main\",\"displayName\":null,\"url\":null,\"profileCount\":159,\"range\":{\"10\":47.85,\"25\":81.62,\"50\":149.5,\"75\":490.68,\"90\":1012.37},\"isEstimated\":false}],\"parentUrl\":null},\"Average Salary Overall\":{\"lastUpdated\":\"2021-02-23T23:33:13.90966Z\",\"profileCount\":5172,\"description\":\"Salary\",\"rows\":[{\"name\":\"Main\",\"displayName\":null,\"url\":null,\"profileCount\":5172,\"range\":{\"10\":123469.27,\"25\":182393.41,\"50\":249659.22,\"75\":354995.71,\"90\":506338.08},\"isEstimated\":false}],\"parentUrl\":null},\"Gender Breakdown\":{\"lastUpdated\":\"2021-02-23T23:33:13.90966Z\",\"profileCount\":1476,\"description\":\"Gender Breakdown\",\"rows\":[{\"name\":\"Male\",\"displayName\":null,\"url\":null,\"profileCount\":1158,\"range\":null,\"isEstimated\":false},{\"name\":\"Female\",\"displayName\":null,\"url\":null,\"profileCount\":316,\"range\":null,\"isEstimated\":false},{\"name\":\"Prefer to self-define\",\"displayName\":null,\"url\":null,\"profileCount\":2,\"range\":null,\"isEstimated\":true}],\"parentUrl\":null},\"Health Insurance Overall\":{\"lastUpdated\":\"2021-02-23T23:33:13.90966Z\",\"profileCount\":2399,\"description\":\"Health Benefits\",\"rows\":[{\"name\":\"None\",\"displayName\":null,\"url\":null,\"profileCount\":1572,\"range\":null,\"isEstimated\":false},{\"name\":\"Medical / Health\",\"displayName\":null,\"url\":null,\"profileCount\":817,\"range\":null,\"isEstimated\":false},{\"name\":\"Vision\",\"displayName\":null,\"url\":null,\"profileCount\":80,\"range\":null,\"isEstimated\":false},{\"name\":\"Dental\",\"displayName\":null,\"url\":null,\"profileCount\":74,\"range\":null,\"isEstimated\":false}],\"parentUrl\":null},\"Job by Employer\":{\"lastUpdated\":null,\"profileCount\":null,\"description\":\"Job by Employer\",\"rows\":[{\"name\":\"Genpact\",\"displayName\":null,\"url\":\"/research/IN/Job=Accountant/Salary/b6f5ce68/Genpact\",\"profileCount\":14,\"range\":{\"50\":269629.63},\"isEstimated\":null},{\"name\":\"Infosys BPO\",\"displayName\":null,\"url\":\"/research/IN/Job=Accountant/Salary/d9543e73/Infosys-BPO\",\"profileCount\":14,\"range\":{\"50\":330000},\"isEstimated\":null},{\"name\":\"Accountants, Inc.\",\"displayName\":null,\"url\":\"/research/IN/Job=Accountant/Salary/d2735610/Accountants-Inc.\",\"profileCount\":13,\"range\":{\"50\":220000},\"isEstimated\":null},{\"name\":\"Abc Corp.\",\"displayName\":null,\"url\":\"/research/IN/Job=Accountant/Salary/3d28b805/Abc-Corp.\",\"profileCount\":8,\"range\":{\"50\":280000},\"isEstimated\":null},{\"name\":\"Any Company\",\"displayName\":null,\"url\":\"/research/IN/Job=Accountant/Salary/5c209f29/Any-Company\",\"profileCount\":7,\"range\":{\"50\":380000},\"isEstimated\":null},{\"name\":\"Larsen \\u0026 Toubro Limited\",\"displayName\":null,\"url\":\"/research/IN/Job=Accountant/Salary/8b34aa19/Larsen-%26-Toubro-Limited\",\"profileCount\":6,\"range\":{\"50\":580930},\"isEstimated\":null},{\"name\":\"CA, Inc.\",\"displayName\":null,\"url\":\"/research/IN/Job=Accountant/Salary/2e1a871d/CA-Inc.\",\"profileCount\":6,\"range\":{\"50\":122083.78},\"isEstimated\":null},{\"name\":\"Capgemini\",\"displayName\":null,\"url\":\"/research/IN/Job=Accountant/Salary/d3e83f95/Capgemini\",\"profileCount\":6,\"range\":{\"50\":305000},\"isEstimated\":null},{\"name\":\"Sobha Developers Ltd\",\"displayName\":null,\"url\":\"/research/IN/Job=Accountant/Salary/38566bc7/Sobha-Developers-Ltd\",\"profileCount\":6,\"range\":{\"50\":300000},\"isEstimated\":null},{\"name\":\"Tata Motors\",\"displayName\":null,\"url\":\"/research/IN/Job=Accountant/Salary/e746cf4b/Tata-Motors\",\"profileCount\":5,\"range\":{\"50\":250000},\"isEstimated\":null},{\"name\":\"Accenture\",\"displayName\":null,\"url\":\"/research/IN/Job=Accountant/Salary/cf62ac58/Accenture\",\"profileCount\":5,\"range\":{\"50\":518000},\"isEstimated\":null},{\"name\":\"MUTHOOT\",\"displayName\":null,\"url\":\"/research/IN/Job=Accountant/Salary/ac9328a6/MUTHOOT\",\"profileCount\":5,\"range\":{\"50\":250000},\"isEstimated\":null},{\"name\":\"ArcelorMittal\",\"displayName\":null,\"url\":\"/research/IN/Job=Accountant/Salary/c75879f9/ArcelorMittal\",\"profileCount\":5,\"range\":{\"50\":522500},\"isEstimated\":null},{\"name\":\"ABC Pvt Ltd\",\"displayName\":null,\"url\":\"/research/IN/Job=Accountant/Salary/8d56cb03/ABC-Pvt-Ltd\",\"profileCount\":2,\"range\":{\"50\":401600},\"isEstimated\":null}],\"parentUrl\":null},\"Job by Experience\":{\"lastUpdated\":null,\"profileCount\":null,\"description\":\"Job by Experience\",\"rows\":[{\"name\":\"Less than 1 year\",\"displayName\":null,\"url\":\"/research/IN/Job=Accountant/Salary/71120446/Entry-Level\",\"profileCount\":654,\"range\":{\"50\":198739.82},\"isEstimated\":null},{\"name\":\"1-4 years\",\"displayName\":null,\"url\":\"/research/IN/Job=Accountant/Salary/71120446/Early-Career\",\"profileCount\":3453,\"range\":{\"50\":231271.96},\"isEstimated\":null},{\"name\":\"5-9 years\",\"displayName\":null,\"url\":\"/research/IN/Job=Accountant/Salary/26cf0477/Mid-Career\",\"profileCount\":1577,\"range\":{\"50\":297059.55},\"isEstimated\":null},{\"name\":\"10-19 years\",\"displayName\":null,\"url\":\"/research/IN/Job=Accountant/Salary/76750e49/Experienced\",\"profileCount\":865,\"range\":{\"50\":353813.98},\"isEstimated\":null},{\"name\":\"20 years or more\",\"displayName\":null,\"url\":\"/research/IN/Job=Accountant/Salary/f7365459/Late-Career\",\"profileCount\":254,\"range\":{\"50\":394115.14},\"isEstimated\":null}],\"parentUrl\":null},\"Job by Location\":{\"lastUpdated\":null,\"profileCount\":null,\"description\":\"Job by Location\",\"rows\":[{\"name\":\"Mumbai, Maharashtra\",\"displayName\":null,\"url\":\"/research/IN/Job=Accountant/Salary/24a57e4d/Mumbai\",\"profileCount\":737,\"range\":{\"50\":253930.97},\"isEstimated\":null},{\"name\":\"New Delhi, Delhi\",\"displayName\":null,\"url\":\"/research/IN/Job=Accountant/Salary/8ffcc4f2/New-Delhi\",\"profileCount\":476,\"range\":{\"50\":257281.34},\"isEstimated\":null},{\"name\":\"Bangalore, Karnataka\",\"displayName\":null,\"url\":\"/research/IN/Job=Accountant/Salary/17594d0f/Bangalore\",\"profileCount\":394,\"range\":{\"50\":294526.42},\"isEstimated\":null},{\"name\":\"Pune, Maharashtra\",\"displayName\":null,\"url\":\"/research/IN/Job=Accountant/Salary/d2b4b9ae/Pune\",\"profileCount\":339,\"range\":{\"50\":247439.04},\"isEstimated\":null},{\"name\":\"Chennai, Tamil Nadu\",\"displayName\":null,\"url\":\"/research/IN/Job=Accountant/Salary/66d63eb1/Chennai\",\"profileCount\":335,\"range\":{\"50\":279459.52},\"isEstimated\":null},{\"name\":\"Kolkata, West Bengal\",\"displayName\":null,\"url\":\"/research/IN/Job=Accountant/Salary/e511fdf2/Kolkata\",\"profileCount\":329,\"range\":{\"50\":222893.4},\"isEstimated\":null},{\"name\":\"Delhi, Delhi\",\"displayName\":null,\"url\":\"/research/IN/Job=Accountant/Salary/9a8093f3/Delhi\",\"profileCount\":307,\"range\":{\"50\":249543.85},\"isEstimated\":null},{\"name\":\"Hyderabad, Andhra Pradesh\",\"displayName\":null,\"url\":\"/research/IN/Job=Accountant/Salary/20cb79cf/Hyderabad\",\"profileCount\":257,\"range\":{\"50\":278592.03},\"isEstimated\":null},{\"name\":\"Ahmedabad, Gujarat\",\"displayName\":null,\"url\":\"/research/IN/Job=Accountant/Salary/2be3e760/Ahmedabad\",\"profileCount\":251,\"range\":{\"50\":264504.13},\"isEstimated\":null},{\"name\":\"Jaipur, Rajasthan\",\"displayName\":null,\"url\":\"/research/IN/Job=Accountant/Salary/53d1a23e/Jaipur\",\"profileCount\":165,\"range\":{\"50\":248975.11},\"isEstimated\":null},{\"name\":\"Lucknow, Uttar Pradesh\",\"displayName\":null,\"url\":\"/research/IN/Job=Accountant/Salary/6cad4a9d/Lucknow\",\"profileCount\":104,\"range\":{\"50\":237030.28},\"isEstimated\":null},{\"name\":\"Surat, Gujarat\",\"displayName\":null,\"url\":\"/research/IN/Job=Accountant/Salary/679778e1/Surat\",\"profileCount\":97,\"range\":{\"50\":243517.1},\"isEstimated\":null},{\"name\":\"Gurgaon, Haryana\",\"displayName\":null,\"url\":\"/research/IN/Job=Accountant/Salary/5320315d/Gurgaon\",\"profileCount\":92,\"range\":{\"50\":296287.85},\"isEstimated\":null},{\"name\":\"Coimbatore, Tamil Nadu\",\"displayName\":null,\"url\":\"/research/IN/Job=Accountant/Salary/8f6a43dc/Coimbatore\",\"profileCount\":88,\"range\":{\"50\":230147.91},\"isEstimated\":null},{\"name\":\"Guwahati, Assam\",\"displayName\":null,\"url\":\"/research/IN/Job=Accountant/Salary/1d262652/Guwahati\",\"profileCount\":79,\"range\":{\"50\":245998.57},\"isEstimated\":null},{\"name\":\"Patna, Bihar\",\"displayName\":null,\"url\":\"/research/IN/Job=Accountant/Salary/a408c5be/Patna\",\"profileCount\":72,\"range\":{\"50\":251071},\"isEstimated\":null},{\"name\":\"Noida, Uttar Pradesh\",\"displayName\":null,\"url\":\"/research/IN/Job=Accountant/Salary/1fffff81/Noida\",\"profileCount\":68,\"range\":{\"50\":274330.67},\"isEstimated\":null},{\"name\":\"Chandigarh, Chandigarh\",\"displayName\":null,\"url\":\"/research/IN/Job=Accountant/Salary/72bf2b0c/Chandigarh\",\"profileCount\":60,\"range\":{\"50\":265801.5},\"isEstimated\":null},{\"name\":\"Thane, Maharashtra\",\"displayName\":null,\"url\":\"/research/IN/Job=Accountant/Salary/414b4a07/Thane\",\"profileCount\":57,\"range\":{\"50\":260861.13},\"isEstimated\":null},{\"name\":\"Ludhiana, Punjab\",\"displayName\":null,\"url\":\"/research/IN/Job=Accountant/Salary/284f4981/Ludhiana\",\"profileCount\":56,\"range\":{\"50\":275000},\"isEstimated\":null},{\"name\":\"Nagpur, Maharashtra\",\"displayName\":null,\"url\":\"/research/IN/Job=Accountant/Salary/eaa1debc/Nagpur\",\"profileCount\":55,\"range\":{\"50\":249102.16},\"isEstimated\":null},{\"name\":\"Indore, Madhya Pradesh\",\"displayName\":null,\"url\":\"/research/IN/Job=Accountant/Salary/cecdc1d4/Indore\",\"profileCount\":53,\"range\":{\"50\":293959.13},\"isEstimated\":null},{\"name\":\"Bhopal, Madhya Pradesh\",\"displayName\":null,\"url\":\"/research/IN/Job=Accountant/Salary/8df87c4b/Bhopal\",\"profileCount\":50,\"range\":{\"50\":237408.04},\"isEstimated\":null},{\"name\":\"Kochi, Kerala\",\"displayName\":null,\"url\":\"/research/IN/Job=Accountant/Salary/234e07a1/Kochi\",\"profileCount\":48,\"range\":{\"50\":274080.2},\"isEstimated\":null},{\"name\":\"Ranchi, Jharkhand\",\"displayName\":null,\"url\":\"/research/IN/Job=Accountant/Salary/a358aced/Ranchi\",\"profileCount\":41,\"range\":{\"50\":293498.64},\"isEstimated\":null},{\"name\":\"Thrissur, Kerala\",\"displayName\":null,\"url\":\"/research/IN/Job=Accountant/Salary/2470dd90/Thrissur\",\"profileCount\":39,\"range\":{\"50\":180859.76},\"isEstimated\":null},{\"name\":\"Ernakulam, Kerala\",\"displayName\":null,\"url\":\"/research/IN/Job=Accountant/Salary/1fcb9e9a/Ernakulam\",\"profileCount\":39,\"range\":{\"50\":225000},\"isEstimated\":null},{\"name\":\"Rajkot, Gujarat\",\"displayName\":null,\"url\":\"/research/IN/Job=Accountant/Salary/50a1553b/Rajkot\",\"profileCount\":38,\"range\":{\"50\":240796.76},\"isEstimated\":null},{\"name\":\"Visakhapatnam, Andhra Pradesh\",\"displayName\":null,\"url\":\"/research/IN/Job=Accountant/Salary/d1fff17e/Visakhapatnam\",\"profileCount\":37,\"range\":{\"50\":300000},\"isEstimated\":null},{\"name\":\"Nashik, Maharashtra\",\"displayName\":null,\"url\":\"/research/IN/Job=Accountant/Salary/7e6e946a/Nashik\",\"profileCount\":34,\"range\":{\"50\":250142.09},\"isEstimated\":null},{\"name\":\"Jodhpur, Rajasthan\",\"displayName\":null,\"url\":\"/research/IN/Job=Accountant/Salary/5fcc8612/Jodhpur\",\"profileCount\":34,\"range\":{\"50\":254341.2},\"isEstimated\":null},{\"name\":\"Faridabad, Haryana\",\"displayName\":null,\"url\":\"/research/IN/Job=Accountant/Salary/cc61d30e/Faridabad\",\"profileCount\":32,\"range\":{\"50\":253968.68},\"isEstimated\":null},{\"name\":\"Vijayawada, Andhra Pradesh\",\"displayName\":null,\"url\":\"/research/IN/Job=Accountant/Salary/f66732a8/Vijayawada\",\"profileCount\":31,\"range\":{\"50\":200000},\"isEstimated\":null},{\"name\":\"Kanpur, Uttar Pradesh\",\"displayName\":null,\"url\":\"/research/IN/Job=Accountant/Salary/a0e0a536/Kanpur\",\"profileCount\":30,\"range\":{\"50\":176874.33},\"isEstimated\":null},{\"name\":\"Aurangabad, Maharashtra\",\"displayName\":null,\"url\":\"/research/IN/Job=Accountant/Salary/e4426d57/Aurangabad\",\"profileCount\":29,\"range\":{\"50\":191729.37},\"isEstimated\":null},{\"name\":\"Varanasi, Uttar Pradesh\",\"displayName\":null,\"url\":\"/research/IN/Job=Accountant/Salary/eaebdb2a/Varanasi\",\"profileCount\":29,\"range\":{\"50\":199600.26},\"isEstimated\":null},{\"name\":\"Mangalore, Karnataka\",\"displayName\":null,\"url\":\"/research/IN/Job=Accountant/Salary/24bcc901/Mangalore\",\"profileCount\":29,\"range\":{\"50\":235832.44},\"isEstimated\":null},{\"name\":\"Ghaziabad, Uttar Pradesh\",\"displayName\":null,\"url\":\"/research/IN/Job=Accountant/Salary/60cb3d40/Ghaziabad\",\"profileCount\":28,\"range\":{\"50\":250535.87},\"isEstimated\":null},{\"name\":\"Baroda, Gujarat\",\"displayName\":null,\"url\":\"/research/IN/Job=Accountant/Salary/74faa00e/Baroda\",\"profileCount\":28,\"range\":{\"50\":248870.8},\"isEstimated\":null},{\"name\":\"Mysore, Karnataka\",\"displayName\":null,\"url\":\"/research/IN/Job=Accountant/Salary/bdba3f8c/Mysore\",\"profileCount\":26,\"range\":{\"50\":236791.97},\"isEstimated\":null},{\"name\":\"Noida, Delhi\",\"displayName\":null,\"url\":\"/research/IN/Job=Accountant/Salary/70dddbd4/Noida\",\"profileCount\":26,\"range\":{\"50\":308403.01},\"isEstimated\":null},{\"name\":\"Siliguri, West Bengal\",\"displayName\":null,\"url\":\"/research/IN/Job=Accountant/Salary/521e5b59/Siliguri\",\"profileCount\":25,\"range\":{\"50\":197325.78},\"isEstimated\":null},{\"name\":\"Trivandrum, Kerala\",\"displayName\":null,\"url\":\"/research/IN/Job=Accountant/Salary/dbc48309/Trivandrum\",\"profileCount\":25,\"range\":{\"50\":213986.02},\"isEstimated\":null},{\"name\":\"Bhubaneswar, Orissa\",\"displayName\":null,\"url\":\"/research/IN/Job=Accountant/Salary/8ed87e6f/Bhubaneswar\",\"profileCount\":25,\"range\":{\"50\":294790.55},\"isEstimated\":null},{\"name\":\"Gurgaon, Delhi\",\"displayName\":null,\"url\":\"/research/IN/Job=Accountant/Salary/274b57ca/Gurgaon\",\"profileCount\":24,\"range\":{\"50\":300000},\"isEstimated\":null},{\"name\":\"Kozhikode, Kerala\",\"displayName\":null,\"url\":\"/research/IN/Job=Accountant/Salary/eb772509/Kozhikode\",\"profileCount\":23,\"range\":{\"50\":144696.39},\"isEstimated\":null},{\"name\":\"Udaipur, Rajasthan\",\"displayName\":null,\"url\":\"/research/IN/Job=Accountant/Salary/230923d1/Udaipur\",\"profileCount\":23,\"range\":{\"50\":212107.39},\"isEstimated\":null},{\"name\":\"Raipur, Chhattisgarh\",\"displayName\":null,\"url\":\"/research/IN/Job=Accountant/Salary/92af5f50/Raipur\",\"profileCount\":21,\"range\":{\"50\":245154.45},\"isEstimated\":null},{\"name\":\"Gandhidham, Gujarat\",\"displayName\":null,\"url\":\"/research/IN/Job=Accountant/Salary/107e97fb/Gandhidham\",\"profileCount\":21,\"range\":{\"50\":251428.58},\"isEstimated\":null},{\"name\":\"Kolhapur, Maharashtra\",\"displayName\":null,\"url\":\"/research/IN/Job=Accountant/Salary/c237a898/Kolhapur\",\"profileCount\":21,\"range\":{\"50\":232258.06},\"isEstimated\":null},{\"name\":\"Jalandhar, Punjab\",\"displayName\":null,\"url\":\"/research/IN/Job=Accountant/Salary/6e01ac86/Jalandhar\",\"profileCount\":21,\"range\":{\"50\":300000},\"isEstimated\":null},{\"name\":\"Agra, Uttar Pradesh\",\"displayName\":null,\"url\":\"/research/IN/Job=Accountant/Salary/16e927fe/Agra\",\"profileCount\":19,\"range\":{\"50\":208117.73},\"isEstimated\":null},{\"name\":\"Madurai, Tamil Nadu\",\"displayName\":null,\"url\":\"/research/IN/Job=Accountant/Salary/fe0cc629/Madurai\",\"profileCount\":18,\"range\":{\"50\":144094.34},\"isEstimated\":null},{\"name\":\"Kannur, Kerala\",\"displayName\":null,\"url\":\"/research/IN/Job=Accountant/Salary/2276686f/Kannur\",\"profileCount\":17,\"range\":{\"50\":240000},\"isEstimated\":null},{\"name\":\"Amritsar, Punjab\",\"displayName\":null,\"url\":\"/research/IN/Job=Accountant/Salary/28a8d2d9/Amritsar\",\"profileCount\":17,\"range\":{\"50\":249600},\"isEstimated\":null},{\"name\":\"Hubli, Karnataka\",\"displayName\":null,\"url\":\"/research/IN/Job=Accountant/Salary/fcedf3bd/Hubli\",\"profileCount\":17,\"range\":{\"50\":236000},\"isEstimated\":null},{\"name\":\"Jamshedpur, Jharkhand\",\"displayName\":null,\"url\":\"/research/IN/Job=Accountant/Salary/eeaa58dd/Jamshedpur\",\"profileCount\":15,\"range\":{\"50\":210000},\"isEstimated\":null},{\"name\":\"Mohali, Punjab\",\"displayName\":null,\"url\":\"/research/IN/Job=Accountant/Salary/bbbac43e/Mohali\",\"profileCount\":15,\"range\":{\"50\":200000},\"isEstimated\":null},{\"name\":\"Meerut, Uttar Pradesh\",\"displayName\":null,\"url\":\"/research/IN/Job=Accountant/Salary/ab4a2c34/Meerut\",\"profileCount\":14,\"range\":{\"50\":226800.01},\"isEstimated\":null},{\"name\":\"Vadodara (Baroda), Gujarat\",\"displayName\":null,\"url\":\"/research/IN/Job=Accountant/Salary/1554b902/Vadodara-Baroda\",\"profileCount\":14,\"range\":{\"50\":250000},\"isEstimated\":null},{\"name\":\"Erode, Tamil Nadu\",\"displayName\":null,\"url\":\"/research/IN/Job=Accountant/Salary/808f90c6/Erode\",\"profileCount\":14,\"range\":{\"50\":216000.01},\"isEstimated\":null},{\"name\":\"Karnal, Haryana\",\"displayName\":null,\"url\":\"/research/IN/Job=Accountant/Salary/26b69ed4/Karnal\",\"profileCount\":14,\"range\":{\"50\":240000},\"isEstimated\":null},{\"name\":\"Panipat, Haryana\",\"displayName\":null,\"url\":\"/research/IN/Job=Accountant/Salary/0ddebce7/Panipat\",\"profileCount\":14,\"range\":{\"50\":203472.96},\"isEstimated\":null},{\"name\":\"Srinagar, Jammu and Kashmir\",\"displayName\":null,\"url\":\"/research/IN/Job=Accountant/Salary/6a5e39f2/Srinagar\",\"profileCount\":13,\"range\":{\"50\":222300},\"isEstimated\":null},{\"name\":\"Palakkad, Kerala\",\"displayName\":null,\"url\":\"/research/IN/Job=Accountant/Salary/7afd5295/Palakkad\",\"profileCount\":13,\"range\":{\"50\":180000},\"isEstimated\":null},{\"name\":\"Salem, Tamil Nadu\",\"displayName\":null,\"url\":\"/research/IN/Job=Accountant/Salary/f6b254f6/Salem\",\"profileCount\":13,\"range\":{\"50\":300000},\"isEstimated\":null},{\"name\":\"Guntur, Andhra Pradesh\",\"displayName\":null,\"url\":\"/research/IN/Job=Accountant/Salary/e7b0a76b/Guntur\",\"profileCount\":13,\"range\":{\"50\":325000},\"isEstimated\":null},{\"name\":\"Vapi, Gujarat\",\"displayName\":null,\"url\":\"/research/IN/Job=Accountant/Salary/02eabac9/Vapi\",\"profileCount\":13,\"range\":{\"50\":450000},\"isEstimated\":null},{\"name\":\"Kottayam, Kerala\",\"displayName\":null,\"url\":\"/research/IN/Job=Accountant/Salary/d326f100/Kottayam\",\"profileCount\":12,\"range\":{\"50\":210600.01},\"isEstimated\":null},{\"name\":\"Dehradun, Uttaranchal\",\"displayName\":null,\"url\":\"/research/IN/Job=Accountant/Salary/9d9f811c/Dehradun\",\"profileCount\":12,\"range\":{\"50\":215000},\"isEstimated\":null},{\"name\":\"Moradabad, Uttar Pradesh\",\"displayName\":null,\"url\":\"/research/IN/Job=Accountant/Salary/041d78f4/Moradabad\",\"profileCount\":12,\"range\":{\"50\":275000},\"isEstimated\":null},{\"name\":\"Shimla, Himachal Pradesh\",\"displayName\":null,\"url\":\"/research/IN/Job=Accountant/Salary/5e270266/Shimla\",\"profileCount\":11,\"range\":{\"50\":292774.47},\"isEstimated\":null},{\"name\":\"Jabalpur, Madhya Pradesh\",\"displayName\":null,\"url\":\"/research/IN/Job=Accountant/Salary/7c7186e3/Jabalpur\",\"profileCount\":11,\"range\":{\"50\":180000},\"isEstimated\":null},{\"name\":\"Kollam, Kerala\",\"displayName\":null,\"url\":\"/research/IN/Job=Accountant/Salary/b5220843/Kollam\",\"profileCount\":10,\"range\":{\"50\":190000},\"isEstimated\":null},{\"name\":\"Shillong, Meghalaya\",\"displayName\":null,\"url\":\"/research/IN/Job=Accountant/Salary/9b17e1e0/Shillong\",\"profileCount\":10,\"range\":{\"50\":252000},\"isEstimated\":null},{\"name\":\"Ajmer, Rajasthan\",\"displayName\":null,\"url\":\"/research/IN/Job=Accountant/Salary/e7568b5a/Ajmer\",\"profileCount\":10,\"range\":{\"50\":225227.08},\"isEstimated\":null},{\"name\":\"Bellary, Karnataka\",\"displayName\":null,\"url\":\"/research/IN/Job=Accountant/Salary/c9cd601d/Bellary\",\"profileCount\":10,\"range\":{\"50\":122083.78},\"isEstimated\":null},{\"name\":\"Kota, Rajasthan\",\"displayName\":null,\"url\":\"/research/IN/Job=Accountant/Salary/7091a05a/Kota\",\"profileCount\":10,\"range\":{\"50\":198000},\"isEstimated\":null},{\"name\":\"Jammu, Jammu and Kashmir\",\"displayName\":null,\"url\":\"/research/IN/Job=Accountant/Salary/c74f6354/Jammu\",\"profileCount\":10,\"range\":{\"50\":240000},\"isEstimated\":null},{\"name\":\"Navi Mumbai (New Bombay), Maharashtra\",\"displayName\":null,\"url\":\"/research/IN/Job=Accountant/Salary/fdc70802/Navi-Mumbai-New-Bombay\",\"profileCount\":10,\"range\":{\"50\":260359.53},\"isEstimated\":null},{\"name\":\"Cochin (Kochi), Kerala\",\"displayName\":null,\"url\":\"/research/IN/Job=Accountant/Salary/86d40783/Cochin-Kochi\",\"profileCount\":9,\"range\":{\"50\":235832.44},\"isEstimated\":null},{\"name\":\"Patiala, Punjab\",\"displayName\":null,\"url\":\"/research/IN/Job=Accountant/Salary/653af0cd/Patiala\",\"profileCount\":9,\"range\":{\"50\":150000},\"isEstimated\":null},{\"name\":\"Ambala, Haryana\",\"displayName\":null,\"url\":\"/research/IN/Job=Accountant/Salary/4ec3339d/Ambala\",\"profileCount\":9,\"range\":{\"50\":235832.44},\"isEstimated\":null},{\"name\":\"Bareilly, Uttar Pradesh\",\"displayName\":null,\"url\":\"/research/IN/Job=Accountant/Salary/c7ff1ea2/Bareilly\",\"profileCount\":9,\"range\":{\"50\":213191.8},\"isEstimated\":null},{\"name\":\"Dibrugarh, Assam\",\"displayName\":null,\"url\":\"/research/IN/Job=Accountant/Salary/b6aa95ed/Dibrugarh\",\"profileCount\":9,\"range\":{\"50\":225000},\"isEstimated\":null},{\"name\":\"Gandhinagar, Gujarat\",\"displayName\":null,\"url\":\"/research/IN/Job=Accountant/Salary/e089cc91/Gandhinagar\",\"profileCount\":8,\"range\":{\"50\":229565.22},\"isEstimated\":null},{\"name\":\"Rourkela, Orissa\",\"displayName\":null,\"url\":\"/research/IN/Job=Accountant/Salary/fc2938b5/Rourkela\",\"profileCount\":8,\"range\":{\"50\":240000},\"isEstimated\":null},{\"name\":\"Thiruvananthapuram, Kerala\",\"displayName\":null,\"url\":\"/research/IN/Job=Accountant/Salary/f1c45bb9/Thiruvananthapuram\",\"profileCount\":8,\"range\":{\"50\":240000},\"isEstimated\":null},{\"name\":\"Kakinada, Andhra Pradesh\",\"displayName\":null,\"url\":\"/research/IN/Job=Accountant/Salary/62630b00/Kakinada\",\"profileCount\":8,\"range\":{\"50\":100000},\"isEstimated\":null},{\"name\":\"Panaji, Goa\",\"displayName\":null,\"url\":\"/research/IN/Job=Accountant/Salary/201cb346/Panaji\",\"profileCount\":8,\"range\":{\"50\":162000},\"isEstimated\":null},{\"name\":\"Alwar, Rajasthan\",\"displayName\":null,\"url\":\"/research/IN/Job=Accountant/Salary/048b5789/Alwar\",\"profileCount\":8,\"range\":{\"50\":275000},\"isEstimated\":null},{\"name\":\"Baddi, Himachal Pradesh\",\"displayName\":null,\"url\":\"/research/IN/Job=Accountant/Salary/18b54f15/Baddi\",\"profileCount\":7,\"range\":{\"50\":266100},\"isEstimated\":null},{\"name\":\"Jalgaon, Maharashtra\",\"displayName\":null,\"url\":\"/research/IN/Job=Accountant/Salary/628c8256/Jalgaon\",\"profileCount\":7,\"range\":{\"50\":244897.96},\"isEstimated\":null},{\"name\":\"Secunderabad, Andhra Pradesh\",\"displayName\":null,\"url\":\"/research/IN/Job=Accountant/Salary/00319757/Secunderabad\",\"profileCount\":7,\"range\":{\"50\":230000},\"isEstimated\":null},{\"name\":\"Aligarh, Uttar Pradesh\",\"displayName\":null,\"url\":\"/research/IN/Job=Accountant/Salary/d5d8e1c0/Aligarh\",\"profileCount\":7,\"range\":{\"50\":262382.92},\"isEstimated\":null},{\"name\":\"Verna, Goa\",\"displayName\":null,\"url\":\"/research/IN/Job=Accountant/Salary/fee2f005/Verna\",\"profileCount\":7,\"range\":{\"50\":153676.5},\"isEstimated\":null},{\"name\":\"Durgapur, West Bengal\",\"displayName\":null,\"url\":\"/research/IN/Job=Accountant/Salary/86721a83/Durgapur\",\"profileCount\":7,\"range\":{\"50\":170000},\"isEstimated\":null},{\"name\":\"Anand, Gujarat\",\"displayName\":null,\"url\":\"/research/IN/Job=Accountant/Salary/c6a8127b/Anand\",\"profileCount\":7,\"range\":{\"50\":204000},\"isEstimated\":null},{\"name\":\"Asansol, West Bengal\",\"displayName\":null,\"url\":\"/research/IN/Job=Accountant/Salary/013e908b/Asansol\",\"profileCount\":7,\"range\":{\"50\":247000},\"isEstimated\":null},{\"name\":\"Bilaspur, Chhattisgarh\",\"displayName\":null,\"url\":\"/research/IN/Job=Accountant/Salary/b2380c54/Bilaspur\",\"profileCount\":6,\"range\":{\"50\":317400},\"isEstimated\":null},{\"name\":\"Dhanbad, Jharkhand\",\"displayName\":null,\"url\":\"/research/IN/Job=Accountant/Salary/aecd8659/Dhanbad\",\"profileCount\":6,\"range\":{\"50\":220000},\"isEstimated\":null},{\"name\":\"Greater Noida, Uttar Pradesh\",\"displayName\":null,\"url\":\"/research/IN/Job=Accountant/Salary/60bbc31b/Greater-Noida\",\"profileCount\":6,\"range\":{\"50\":500000},\"isEstimated\":null},{\"name\":\"Junagadh, Gujarat\",\"displayName\":null,\"url\":\"/research/IN/Job=Accountant/Salary/bf4c1e5a/Junagadh\",\"profileCount\":6,\"range\":{\"50\":144000},\"isEstimated\":null},{\"name\":\"Jhansi, Uttar Pradesh\",\"displayName\":null,\"url\":\"/research/IN/Job=Accountant/Salary/140b301d/Jhansi\",\"profileCount\":6,\"range\":{\"50\":540000},\"isEstimated\":null},{\"name\":\"Satara, Maharashtra\",\"displayName\":null,\"url\":\"/research/IN/Job=Accountant/Salary/bd717e37/Satara\",\"profileCount\":6,\"range\":{\"50\":235000},\"isEstimated\":null},{\"name\":\"Rohtak, Haryana\",\"displayName\":null,\"url\":\"/research/IN/Job=Accountant/Salary/47e970d5/Rohtak\",\"profileCount\":6,\"range\":{\"50\":300000},\"isEstimated\":null},{\"name\":\"Gwalior, Madhya Pradesh\",\"displayName\":null,\"url\":\"/research/IN/Job=Accountant/Salary/821faf8b/Gwalior\",\"profileCount\":6,\"range\":{\"50\":281000.01},\"isEstimated\":null},{\"name\":\"Nellore, Andhra Pradesh\",\"displayName\":null,\"url\":\"/research/IN/Job=Accountant/Salary/8e10c05e/Nellore\",\"profileCount\":6,\"range\":{\"50\":358000.01},\"isEstimated\":null},{\"name\":\"Alappuzha, Kerala\",\"displayName\":null,\"url\":\"/research/IN/Job=Accountant/Salary/9232a57f/Alappuzha\",\"profileCount\":6,\"range\":{\"50\":270000},\"isEstimated\":null},{\"name\":\"Bhubaneshwar, Orissa\",\"displayName\":null,\"url\":\"/research/IN/Job=Accountant/Salary/bf181f04/Bhubaneshwar\",\"profileCount\":6,\"range\":{\"50\":211500},\"isEstimated\":null},{\"name\":\"Agartala, Tripura\",\"displayName\":null,\"url\":\"/research/IN/Job=Accountant/Salary/be773c6c/Agartala\",\"profileCount\":6,\"range\":{\"50\":395000},\"isEstimated\":null},{\"name\":\"Maharashtra, Maharashtra\",\"displayName\":null,\"url\":\"/research/IN/Job=Accountant/Salary/b3eacb73/Maharashtra\",\"profileCount\":6,\"range\":{\"50\":228471.61},\"isEstimated\":null},{\"name\":\"Gujrat, Gujarat\",\"displayName\":null,\"url\":\"/research/IN/Job=Accountant/Salary/a1667880/Gujrat\",\"profileCount\":6,\"range\":{\"50\":300000},\"isEstimated\":null},{\"name\":\"Bhiwandi, Maharashtra\",\"displayName\":null,\"url\":\"/research/IN/Job=Accountant/Salary/41249dc5/Bhiwandi\",\"profileCount\":6,\"range\":{\"50\":300000},\"isEstimated\":null},{\"name\":\"Tirupati, Andhra Pradesh\",\"displayName\":null,\"url\":\"/research/IN/Job=Accountant/Salary/2bbdb9e0/Tirupati\",\"profileCount\":6,\"range\":{\"50\":294790.55},\"isEstimated\":null},{\"name\":\"Allahabad, Uttar Pradesh\",\"displayName\":null,\"url\":\"/research/IN/Job=Accountant/Salary/251eb0f9/Allahabad\",\"profileCount\":6,\"range\":{\"50\":165000},\"isEstimated\":null},{\"name\":\"Bhiwadi, Rajasthan\",\"displayName\":null,\"url\":\"/research/IN/Job=Accountant/Salary/83b2a9cc/Bhiwadi\",\"profileCount\":5,\"range\":{\"50\":350000},\"isEstimated\":null},{\"name\":\"Pondicherry, Pondicherry\",\"displayName\":null,\"url\":\"/research/IN/Job=Accountant/Salary/87bfde95/Pondicherry\",\"profileCount\":5,\"range\":{\"50\":120000},\"isEstimated\":null},{\"name\":\"Amravati, Maharashtra\",\"displayName\":null,\"url\":\"/research/IN/Job=Accountant/Salary/0470c3f0/Amravati\",\"profileCount\":5,\"range\":{\"50\":180000},\"isEstimated\":null},{\"name\":\"Darbhanga, Bihar\",\"displayName\":null,\"url\":\"/research/IN/Job=Accountant/Salary/1a9b9fee/Darbhanga\",\"profileCount\":5,\"range\":{\"50\":228000},\"isEstimated\":null},{\"name\":\"Mathura, Uttar Pradesh\",\"displayName\":null,\"url\":\"/research/IN/Job=Accountant/Salary/ad0838bc/Mathura\",\"profileCount\":5,\"range\":{\"50\":122083.78},\"isEstimated\":null},{\"name\":\"Raigarh, Chhattisgarh\",\"displayName\":null,\"url\":\"/research/IN/Job=Accountant/Hourly_Rate/62648947/Raigarh\",\"profileCount\":5,\"range\":{\"50\":null},\"isEstimated\":null},{\"name\":\"Bhilai, Chhattisgarh\",\"displayName\":null,\"url\":\"/research/IN/Job=Accountant/Salary/96139624/Bhilai\",\"profileCount\":5,\"range\":{\"50\":222000},\"isEstimated\":null},{\"name\":\"Silchar, Assam\",\"displayName\":null,\"url\":\"/research/IN/Job=Accountant/Salary/626e01b9/Silchar\",\"profileCount\":5,\"range\":{\"50\":150000},\"isEstimated\":null},{\"name\":\"Solapur, Maharashtra\",\"displayName\":null,\"url\":\"/research/IN/Job=Accountant/Salary/c9d561cd/Solapur\",\"profileCount\":5,\"range\":{\"50\":200000},\"isEstimated\":null},{\"name\":\"Jamnagar, Gujarat\",\"displayName\":null,\"url\":\"/research/IN/Job=Accountant/Salary/1bcf4fa0/Jamnagar\",\"profileCount\":5,\"range\":{\"50\":460000},\"isEstimated\":null},{\"name\":\"Hosur, Tamil Nadu\",\"displayName\":null,\"url\":\"/research/IN/Job=Accountant/Salary/e124ebda/Hosur\",\"profileCount\":5,\"range\":{\"50\":405000},\"isEstimated\":null},{\"name\":\"Haridwar, Uttaranchal\",\"displayName\":null,\"url\":\"/research/IN/Job=Accountant/Salary/a1ec9465/Haridwar\",\"profileCount\":5,\"range\":{\"50\":250000},\"isEstimated\":null},{\"name\":\"Cuttack, Orissa\",\"displayName\":null,\"url\":\"/research/IN/Job=Accountant/Salary/74b72737/Cuttack\",\"profileCount\":5,\"range\":{\"50\":315000},\"isEstimated\":null},{\"name\":\"Bhimavaram, Andhra Pradesh\",\"displayName\":null,\"url\":\"/research/IN/Job=Accountant/Salary/bbb88a97/Bhimavaram\",\"profileCount\":5,\"range\":{\"50\":270000},\"isEstimated\":null},{\"name\":\"Margao, Goa\",\"displayName\":null,\"url\":\"/research/IN/Job=Accountant/Salary/614cbe0e/Margao\",\"profileCount\":4,\"range\":{\"50\":289800},\"isEstimated\":null},{\"name\":\"Chittoor, Andhra Pradesh\",\"displayName\":null,\"url\":\"/research/IN/Job=Accountant/Salary/2dc1c951/Chittoor\",\"profileCount\":3,\"range\":{\"50\":300000},\"isEstimated\":null},{\"name\":\"Mumbai (Bombay), Maharashtra\",\"displayName\":null,\"url\":\"/research/IN/Job=Accountant/Salary/75860f1b/Mumbai-Bombay\",\"profileCount\":2,\"range\":{\"50\":460000},\"isEstimated\":null}],\"parentUrl\":null},\"Job by Skill\":{\"lastUpdated\":null,\"profileCount\":null,\"description\":\"Job by Skill\",\"rows\":[{\"name\":\"General Ledger Accounting\",\"displayName\":null,\"url\":\"/research/IN/Job=Accountant/Salary/ebd17157/General-Ledger-Accounting\",\"profileCount\":2274,\"range\":{\"50\":248280.94},\"isEstimated\":null},{\"name\":\"Accounts Payable\",\"displayName\":null,\"url\":\"/research/IN/Job=Accountant/Salary/35a6591b/Accounts-Payable\",\"profileCount\":1746,\"range\":{\"50\":262045.01},\"isEstimated\":null},{\"name\":\"Microsoft Excel\",\"displayName\":null,\"url\":\"/research/IN/Job=Accountant/Salary/dc47b68a/Microsoft-Excel\",\"profileCount\":1669,\"range\":{\"50\":240280.29},\"isEstimated\":null},{\"name\":\"Bookkeeping\",\"displayName\":null,\"url\":\"/research/IN/Job=Accountant/Salary/d440a488/Bookkeeping\",\"profileCount\":1638,\"range\":{\"50\":242575.02},\"isEstimated\":null},{\"name\":\"Financial Reporting\",\"displayName\":null,\"url\":\"/research/IN/Job=Accountant/Salary/60f5899c/Financial-Reporting\",\"profileCount\":1449,\"range\":{\"50\":265920.15},\"isEstimated\":null},{\"name\":\"Accounts Receivable\",\"displayName\":null,\"url\":\"/research/IN/Job=Accountant/Salary/e8eeb682/Accounts-Receivable\",\"profileCount\":891,\"range\":{\"50\":247490.63},\"isEstimated\":null},{\"name\":\"Account Management\",\"displayName\":null,\"url\":\"/research/IN/Job=Accountant/Salary/5daf93b6/Account-Management\",\"profileCount\":731,\"range\":{\"50\":282471.21},\"isEstimated\":null},{\"name\":\"Tax Preparation\",\"displayName\":null,\"url\":\"/research/IN/Job=Accountant/Salary/7c2820eb/Tax-Preparation\",\"profileCount\":490,\"range\":{\"50\":248835.1},\"isEstimated\":null},{\"name\":\"Sales Tax Reports\",\"displayName\":null,\"url\":\"/research/IN/Job=Accountant/Salary/0699604f/Sales-Tax-Reports\",\"profileCount\":328,\"range\":{\"50\":248633.7},\"isEstimated\":null},{\"name\":\"Microsoft Office\",\"displayName\":null,\"url\":\"/research/IN/Job=Accountant/Salary/9a4745f8/Microsoft-Office\",\"profileCount\":321,\"range\":{\"50\":247912.27},\"isEstimated\":null},{\"name\":\"Accounting\",\"displayName\":null,\"url\":\"/research/IN/Job=Accountant/Salary/20b362bd/Accounting\",\"profileCount\":291,\"range\":{\"50\":239633.95},\"isEstimated\":null},{\"name\":\"Enterprise Resource Planning (ERP)\",\"displayName\":null,\"url\":\"/research/IN/Job=Accountant/Salary/88624f3c/Enterprise-Resource-Planning-ERP\",\"profileCount\":254,\"range\":{\"50\":247612.37},\"isEstimated\":null},{\"name\":\"Financial Analysis\",\"displayName\":null,\"url\":\"/research/IN/Job=Accountant/Salary/f81d0b9a/Financial-Analysis\",\"profileCount\":239,\"range\":{\"50\":294804.13},\"isEstimated\":null},{\"name\":\"Tax Compliance\",\"displayName\":null,\"url\":\"/research/IN/Job=Accountant/Salary/4e46a65f/Tax-Compliance\",\"profileCount\":224,\"range\":{\"50\":291967.73},\"isEstimated\":null},{\"name\":\"SAP Financial Accounting and Controlling (SAP FICO)\",\"displayName\":null,\"url\":\"/research/IN/Job=Accountant/Salary/66322e3f/SAP-Financial-Accounting-and-Controlling-SAP-FICO\",\"profileCount\":99,\"range\":{\"50\":294790.55},\"isEstimated\":null},{\"name\":\"Auditing\",\"displayName\":null,\"url\":\"/research/IN/Job=Accountant/Salary/78eb8c7b/Auditing\",\"profileCount\":66,\"range\":{\"50\":223272.17},\"isEstimated\":null},{\"name\":\"Reconciliation\",\"displayName\":null,\"url\":\"/research/IN/Job=Accountant/Salary/cc40cb50/Reconciliation\",\"profileCount\":47,\"range\":{\"50\":244472.13},\"isEstimated\":null},{\"name\":\"Payroll Administration\",\"displayName\":null,\"url\":\"/research/IN/Job=Accountant/Salary/6fb24d9c/Payroll-Administration\",\"profileCount\":28,\"range\":{\"50\":250683.36},\"isEstimated\":null},{\"name\":\"Data Analysis\",\"displayName\":null,\"url\":\"/research/IN/Job=Accountant/Salary/e4587645/Data-Analysis\",\"profileCount\":19,\"range\":{\"50\":239447.37},\"isEstimated\":null},{\"name\":\"Microsoft Word\",\"displayName\":null,\"url\":\"/research/IN/Job=Accountant/Salary/27e865a9/Microsoft-Word\",\"profileCount\":14,\"range\":{\"50\":178000},\"isEstimated\":null},{\"name\":\"Banking\",\"displayName\":null,\"url\":\"/research/IN/Job=Accountant/Salary/a5e74380/Banking\",\"profileCount\":9,\"range\":{\"50\":204599.99},\"isEstimated\":null},{\"name\":\"Financial Applications\",\"displayName\":null,\"url\":\"/research/IN/Job=Accountant/Salary/cb234c7e/Financial-Applications\",\"profileCount\":8,\"range\":{\"50\":250252.3},\"isEstimated\":null},{\"name\":\"Tax Consulting\",\"displayName\":null,\"url\":\"/research/IN/Job=Accountant/Salary/0b7aaf92/Tax-Consulting\",\"profileCount\":8,\"range\":{\"50\":193600},\"isEstimated\":null},{\"name\":\"Data Entry\",\"displayName\":null,\"url\":\"/research/IN/Job=Accountant/Salary/feef7e79/Data-Entry\",\"profileCount\":7,\"range\":{\"50\":392000},\"isEstimated\":null},{\"name\":\"Cash Handling\",\"displayName\":null,\"url\":\"/research/IN/Job=Accountant/Salary/0a786367/Cash-Handling\",\"profileCount\":6,\"range\":{\"50\":380000},\"isEstimated\":null},{\"name\":\"Account Sales\",\"displayName\":null,\"url\":\"/research/IN/Job=Accountant/Salary/9b390b5e/Account-Sales\",\"profileCount\":5,\"range\":{\"50\":150000},\"isEstimated\":null},{\"name\":\"Billing\",\"displayName\":null,\"url\":\"/research/IN/Job=Accountant/Salary/22845590/Billing\",\"profileCount\":5,\"range\":{\"50\":270000},\"isEstimated\":null},{\"name\":\"Budget Management\",\"displayName\":null,\"url\":\"/research/IN/Job=Accountant/Salary/22812c71/Budget-Management\",\"profileCount\":5,\"range\":{\"50\":900000},\"isEstimated\":null},{\"name\":\"Database Management \\u0026 Reporting\",\"displayName\":null,\"url\":\"/research/IN/Job=Accountant/Salary/6aa78946/Database-Management-%26-Reporting\",\"profileCount\":5,\"range\":{\"50\":408500},\"isEstimated\":null},{\"name\":\"Internal Audit\",\"displayName\":null,\"url\":\"/research/IN/Job=Accountant/Salary/5c162dd9/Internal-Audit\",\"profileCount\":5,\"range\":{\"50\":240000},\"isEstimated\":null},{\"name\":\"Typing\",\"displayName\":null,\"url\":\"/research/IN/Job=Accountant/Salary/05ec3fdc/Typing\",\"profileCount\":2,\"range\":{\"50\":452000},\"isEstimated\":null}],\"parentUrl\":null}},\"ratings\":null,\"related\":[{\"name\":\"Chartered Accountant\",\"url\":\"/research/IN/Job=Chartered_Accountant/Salary\",\"profileCount\":513,\"range\":{\"10\":412945.15,\"25\":594549.72,\"50\":794525.4,\"75\":1205432.81,\"90\":2020462},\"average\":null},{\"name\":\"Financial Analyst\",\"url\":\"/research/IN/Job=Financial_Analyst/Salary\",\"profileCount\":505,\"range\":{\"10\":219376.14,\"25\":295047.01,\"50\":411369.42,\"75\":630295.34,\"90\":991724.91},\"average\":null},{\"name\":\"Finance Manager\",\"url\":\"/research/IN/Job=Finance_Manager/Salary\",\"profileCount\":502,\"range\":{\"10\":353874.14,\"25\":596932.22,\"50\":991797.55,\"75\":1486714.85,\"90\":2015596.16},\"average\":null},{\"name\":\"Data Analyst\",\"url\":\"/research/IN/Job=Data_Analyst/Salary\",\"profileCount\":436,\"range\":{\"10\":201011.18,\"25\":298846.79,\"50\":429754.27,\"75\":630494.46,\"90\":965540.78},\"average\":null},{\"name\":\"Civil Engineer\",\"url\":\"/research/IN/Job=Civil_Engineer/Salary\",\"profileCount\":430,\"range\":{\"10\":153812.13,\"25\":222043.27,\"50\":308345.78,\"75\":492509.23,\"90\":774024.53},\"average\":null},{\"name\":\"Account Executive\",\"url\":\"/research/IN/Job=Account_Executive/Salary\",\"profileCount\":426,\"range\":{\"10\":155082.38,\"25\":200508.23,\"50\":270575.58,\"75\":364537.64,\"90\":498203.22},\"average\":null},{\"name\":\"Senior Accountant\",\"url\":\"/research/IN/Job=Senior_Accountant/Salary\",\"profileCount\":371,\"range\":{\"10\":241857.37,\"25\":303594.45,\"50\":407650.53,\"75\":560833.67,\"90\":771643.64},\"average\":null},{\"name\":\"Mechanical Engineer\",\"url\":\"/research/IN/Job=Mechanical_Engineer/Salary\",\"profileCount\":352,\"range\":{\"10\":151649.42,\"25\":225654.75,\"50\":349704.25,\"75\":581578.82,\"90\":982307.02},\"average\":null},{\"name\":\"Data Entry Operator\",\"url\":\"/research/IN/Job=Data_Entry_Operator/Salary\",\"profileCount\":292,\"range\":{\"10\":98484.25,\"25\":123734.54,\"50\":181514.3,\"75\":246232.08,\"90\":348378.56},\"average\":null},{\"name\":\"Assistant Finance Manager\",\"url\":\"/research/IN/Job=Assistant_Finance_Manager/Salary\",\"profileCount\":288,\"range\":{\"10\":351037.29,\"25\":533738.96,\"50\":724793.25,\"75\":953170.45,\"90\":1239392.26},\"average\":null},{\"name\":\"Account Manager\",\"url\":\"/research/IN/Job=Account_Manager/Salary\",\"profileCount\":288,\"range\":{\"10\":278642.17,\"25\":402064.69,\"50\":585641.99,\"75\":867024.89,\"90\":1356959.57},\"average\":null},{\"name\":\"Operations Manager\",\"url\":\"/research/IN/Job=Operations_Manager/Salary\",\"profileCount\":281,\"range\":{\"10\":296329.44,\"25\":474851.3,\"50\":766239.08,\"75\":1191460.22,\"90\":1725486.96},\"average\":null},{\"name\":\"Senior Financial Analyst\",\"url\":\"/research/IN/Job=Senior_Financial_Analyst/Salary\",\"profileCount\":263,\"range\":{\"10\":305429.22,\"25\":408359.42,\"50\":599645.68,\"75\":926210.32,\"90\":1363707},\"average\":null},{\"name\":\"Senior Account Executive\",\"url\":\"/research/IN/Job=Senior_Account_Executive/Salary\",\"profileCount\":235,\"range\":{\"10\":246132.73,\"25\":307211.66,\"50\":405700.14,\"75\":511132.53,\"90\":645729.12},\"average\":null},{\"name\":\"Computer Operator\",\"url\":\"/research/IN/Job=Computer_Operator/Salary\",\"profileCount\":227,\"range\":{\"10\":99038.96,\"25\":125443.37,\"50\":184706.41,\"75\":255229.88,\"90\":403963.26},\"average\":null},{\"name\":\"Senior Business Analyst\",\"url\":\"/research/IN/Job=Senior_Business_Analyst/Salary\",\"profileCount\":224,\"range\":{\"10\":449591.5,\"25\":691805.58,\"50\":987990.04,\"75\":1403827.81,\"90\":1770526.54},\"average\":null},{\"name\":\"Financial Controller\",\"url\":\"/research/IN/Job=Financial_Controller/Salary\",\"profileCount\":220,\"range\":{\"10\":595958.74,\"25\":1017971.96,\"50\":1825944.56,\"75\":2914866.07,\"90\":4021141.11},\"average\":null},{\"name\":\"Business Analyst, Finance/Banking\",\"url\":\"/research/IN/Job=Business_Analyst%2C_Finance%2FBanking/Salary\",\"profileCount\":209,\"range\":{\"10\":277474.65,\"25\":427410.2,\"50\":658298.77,\"75\":1002376.81,\"90\":1522967.91},\"average\":null},{\"name\":\"Business Analyst, IT\",\"url\":\"/research/IN/Job=Business_Analyst%2C_IT/Salary\",\"profileCount\":209,\"range\":{\"10\":272358.88,\"25\":406011.15,\"50\":604827.8,\"75\":890268.76,\"90\":1226652.77},\"average\":null},{\"name\":\"Project Manager, (Unspecified Type / General)\",\"url\":\"/research/IN/Job=Project_Manager%2C_(Unspecified_Type_%2F_General)/Salary\",\"profileCount\":204,\"range\":{\"10\":452729.87,\"25\":784815.26,\"50\":1245928.7,\"75\":1749315.05,\"90\":2279899.8},\"average\":null}],\"benefits\":null,\"comparisons\":null,\"locations\":null,\"occupationalDetails\":{\"@context\":\"http://schema.googleapis.com/\",\"@id\":\"/research/IN/Job=Accountant/Salary\",\"occupationalCategory\":\"13-2010.00 - Accountants and Auditors\",\"skills\":\"General Ledger Accounting, Accounts Payable, Microsoft Excel, Bookkeeping, Financial Reporting, Accounts Receivable, Account Management, Tax Preparation, Sales Tax Reports, Microsoft Office, Accounting, Enterprise Resource Planning (ERP), Financial Analysis, Tax Compliance, SAP Financial Accounting and Controlling (SAP FICO), Auditing, Reconciliation, Payroll Administration, Data Analysis, Microsoft Word, Banking, Financial Applications, Tax Consulting, Data Entry, Cash Handling, Account Sales, Billing, Budget Management, Database Management \\u0026 Reporting, Internal Audit, Typing\",\"educationRequirements\":null,\"experienceRequirements\":null,\"qualifications\":null,\"responsibilities\":null,\"mainEntityOfPage\":{\"lastReviewed\":\"2021-02-23T23:33:13.90966Z\",\"description\":null,\"name\":null,\"url\":null,\"@type\":\"WebPage\"},\"occupationLocation\":[{\"description\":null,\"name\":\"India\",\"url\":null,\"@type\":\"Country\"}],\"estimatedSalary\":[{\"currency\":\"INR\",\"unitText\":\"YEAR\",\"percentile10\":\"128947.05\",\"percentile25\":\"188425.71\",\"median\":\"265158.91\",\"percentile75\":\"384549.98\",\"percentile90\":\"584411.15\",\"description\":null,\"name\":\"total\",\"url\":null,\"@type\":\"MonetaryAmountDistribution\"},{\"currency\":\"INR\",\"unitText\":\"YEAR\",\"percentile10\":\"123469.27\",\"percentile25\":\"182393.41\",\"median\":\"249659.22\",\"percentile75\":\"354995.71\",\"percentile90\":\"506338.08\",\"description\":null,\"name\":\"base\",\"url\":null,\"@type\":\"MonetaryAmountDistribution\"},{\"currency\":\"INR\",\"unitText\":\"YEAR\",\"percentile10\":\"4917.77\",\"percentile25\":\"9951.54\",\"median\":\"18657.9\",\"percentile75\":\"31037.74\",\"percentile90\":\"59567.62\",\"description\":null,\"name\":\"bonus\",\"url\":null,\"@type\":\"MonetaryAmountDistribution\"}],\"sampleSize\":6805,\"yearsExperienceMin\":null,\"yearsExperienceMax\":null,\"hiringOrganization\":null,\"description\":\"Accountants perform financial calculations for companies in a wide variety of fields. Some common duties include creating sales and cash flow reports, administering payroll, keeping balance sheets, carrying out billing activities, managing budgets and keeping inventory. The accountant may also be responsible for filing taxes for the company, as well as reviewing past reports to generate income forecasts.\\r\\nOccasionally, internal audits must be carried out to make sure that the various areas of the company are performing as expected; the accountant must also make sure that staff members are adhering to company policies and relevant laws. The accountant should be able to create accurate, detailed reports to illustrate data; sometimes, these reports have to be presented to management. The accountant may oversee the financial transactions of one department or multiple departments within their organization. \\r\\nA bachelor's degree in accounting is required for this position, as is status as a certified public accountant (CPA). Previous accounting experience is generally required or preferred as well. Knowledge of accounting software such as Quickbooks and Microsoft Excel is needed. Additionally, many of the accountant's tasks are performed independently so it is essential to be self-motivated; however, collaboration is necessary, and the accountant must be able to work as part of a team. \\r\\n\",\"name\":\"Accountant\",\"url\":\"/research/IN/Job=Accountant/Salary\",\"@type\":\"OccupationAggregation\"}}},\"__N_SSP\":true},\"page\":\"/job/pay\",\"query\":{},\"buildId\":\"app\",\"assetPrefix\":\"https://cdn-payscale.com/content/research-center-web/prod/2021.0208.2046.252-master\",\"isFallback\":false,\"gssp\":true,\"customServer\":true}</script><script nomodule=\"\" src=\"https://cdn-payscale.com/content/research-center-web/prod/2021.0208.2046.252-master/_next/static/chunks/polyfills-b74fd9d4ba0d6abe21cb.js\"></script><script src=\"https://cdn-payscale.com/content/research-center-web/prod/2021.0208.2046.252-master/_next/static/chunks/main-6cec33473fbdb69509e8.js\" async=\"\"></script><script src=\"https://cdn-payscale.com/content/research-center-web/prod/2021.0208.2046.252-master/_next/static/chunks/webpack-c060148b42bba71f8a7a.js\" async=\"\"></script><script src=\"https://cdn-payscale.com/content/research-center-web/prod/2021.0208.2046.252-master/_next/static/chunks/commons.0c513591aac5aea1df4c.js\" async=\"\"></script><script src=\"https://cdn-payscale.com/content/research-center-web/prod/2021.0208.2046.252-master/_next/static/chunks/f6078781a05fe1bcb0902d23dbbb2662c8d200b3.233d4f01bf6ec4c37fc3.js\" async=\"\"></script><script src=\"https://cdn-payscale.com/content/research-center-web/prod/2021.0208.2046.252-master/_next/static/chunks/pages/_app-89f1c4721ae4f2fe3fde.js\" async=\"\"></script><script src=\"https://cdn-payscale.com/content/research-center-web/prod/2021.0208.2046.252-master/_next/static/chunks/framework.75101f9487a7a475ce88.js\" async=\"\"></script><script src=\"https://cdn-payscale.com/content/research-center-web/prod/2021.0208.2046.252-master/_next/static/chunks/cc69ab74a9ba72eea94736558729e81c2bd655f3.26c13cf5aedb6ea95a1c.js\" async=\"\"></script><script src=\"https://cdn-payscale.com/content/research-center-web/prod/2021.0208.2046.252-master/_next/static/chunks/4d4feda2456de235db09b29c583479af129999e0.1fb7365a3f7fbeba8abc.js\" async=\"\"></script><script src=\"https://cdn-payscale.com/content/research-center-web/prod/2021.0208.2046.252-master/_next/static/chunks/fa26498fafef60f501c64b06cec839d007da4e70.343e0494ce0488088446.js\" async=\"\"></script><script src=\"https://cdn-payscale.com/content/research-center-web/prod/2021.0208.2046.252-master/_next/static/chunks/4d1d62af250902ce0f83bc284e189364548c5c21.39a093079767798521c5.js\" async=\"\"></script><script src=\"https://cdn-payscale.com/content/research-center-web/prod/2021.0208.2046.252-master/_next/static/chunks/4699d7e6a61b952e51311f390f955e9a91c7a24d.ea92ec11cbdb296acd78.js\" async=\"\"></script><script src=\"https://cdn-payscale.com/content/research-center-web/prod/2021.0208.2046.252-master/_next/static/chunks/1d702dc2a46913e334a0d518d1f316ce60acc9ce.3808c6f06c8377373706.js\" async=\"\"></script><script src=\"https://cdn-payscale.com/content/research-center-web/prod/2021.0208.2046.252-master/_next/static/chunks/f651bd44fd1cbb875495552cc10ebe204e4809e8.36ed96af6144c8de99e8.js\" async=\"\"></script><script src=\"https://cdn-payscale.com/content/research-center-web/prod/2021.0208.2046.252-master/_next/static/chunks/f457c96651799a12cf52d5e467d410eb70c755ff.db47a5eb374ee61b5d4a.js\" async=\"\"></script><script src=\"https://cdn-payscale.com/content/research-center-web/prod/2021.0208.2046.252-master/_next/static/chunks/a129799a303bc049890e96af24a6d6ea5a880873.3bc74ab583f25c8441ec.js\" async=\"\"></script><script src=\"https://cdn-payscale.com/content/research-center-web/prod/2021.0208.2046.252-master/_next/static/chunks/styles.c2666581feb32a659f31.js\" async=\"\"></script><script src=\"https://cdn-payscale.com/content/research-center-web/prod/2021.0208.2046.252-master/_next/static/chunks/pages/job/pay-04b93fe8d5a6d6a2d6a9.js\" async=\"\"></script><script src=\"https://cdn-payscale.com/content/research-center-web/prod/2021.0208.2046.252-master/_next/static/app/_buildManifest.js\" async=\"\"></script><script src=\"https://cdn-payscale.com/content/research-center-web/prod/2021.0208.2046.252-master/_next/static/app/_ssgManifest.js\" async=\"\"></script><div style=\"display: none; visibility: hidden;\">\n<script src=\"https://cdn.cookielaw.org/scripttemplates/otSDKStub.js\" type=\"text/javascript\" charset=\"UTF-8\" data-domain-script=\"832d08fb-f25f-4eb2-86ed-d72dd85e17cd\"></script>\n<script type=\"text/javascript\">function OptanonWrapper(){};</script>\n</div><div style=\"display: none; visibility: hidden;\">\n<script src=\"https://dev.visualwebsiteoptimizer.com/lib/268416.js\"></script>\n</div><script type=\"text/javascript\" id=\"\">(function(c,d,f,g,e){c[e]=c[e]||[];var h=function(){var b={ti:\"4024402\"};b.q=c[e];c[e]=new UET(b);c[e].push(\"pageLoad\")};var a=d.createElement(f);a.src=g;a.async=1;a.onload=a.onreadystatechange=function(){var b=this.readyState;b&&\"loaded\"!==b&&\"complete\"!==b||(h(),a.onload=a.onreadystatechange=null)};d=d.getElementsByTagName(f)[0];d.parentNode.insertBefore(a,d)})(window,document,\"script\",\"//bat.bing.com/bat.js\",\"uetq\");</script><noscript><img src=\"//bat.bing.com/action/0?ti=4024402&amp;Ver=2\" height=\"0\" width=\"0\" style=\"display:none; visibility: hidden;\"></noscript> <script type=\"text/javascript\" id=\"\">(function(c,d,f,g,e){c[e]=c[e]||[];var h=function(){var b={ti:\"4029085\"};b.q=c[e];c[e]=new UET(b);c[e].push(\"pageLoad\")};var a=d.createElement(f);a.src=g;a.async=1;a.onload=a.onreadystatechange=function(){var b=this.readyState;b&&\"loaded\"!==b&&\"complete\"!==b||(h(),a.onload=a.onreadystatechange=null)};d=d.getElementsByTagName(f)[0];d.parentNode.insertBefore(a,d)})(window,document,\"script\",\"//bat.bing.com/bat.js\",\"uetq\");</script><noscript><img src=\"//bat.bing.com/action/0?ti=4029085&amp;Ver=2\" height=\"0\" width=\"0\" style=\"display:none; visibility: hidden;\"></noscript><script type=\"text/javascript\" id=\"\" src=\"//munchkin.marketo.net/munchkin.js\"></script><script type=\"text/javascript\" id=\"\">if(\"object\"===typeof localStorage){var localStorageRcVal=localStorage.getItem(\"rcVisits\"),localStorageRcVisits=[encodeURIComponent(window.location.pathname)];localStorageRcVal?-1===localStorageRcVal.indexOf(localStorageRcVisits)&&(localStorageRcVisits=localStorageRcVisits.concat(localStorageRcVal.split(\",\")).slice(0,10),localStorage.setItem(\"rcVisits\",localStorageRcVisits.join(\",\"))):localStorage.setItem(\"rcVisits\",localStorageRcVisits)};</script>\n<script type=\"text/javascript\" id=\"\">!function(){var a=window.driftt=window.drift=window.driftt||[];if(!a.init){if(a.invoked)return void(window.console&&console.error&&console.error(\"Drift snippet included twice.\"));a.invoked=!0;a.methods=\"identify config track reset debug show ping page hide off on\".split(\" \");a.factory=function(c){return function(){var b=Array.prototype.slice.call(arguments);return b.unshift(c),a.push(b),a}};a.methods.forEach(function(c){a[c]=a.factory(c)});a.load=function(c){var b=3E5,d=Math.ceil(new Date/b)*b;b=\ndocument.createElement(\"script\");b.type=\"text/javascript\";b.async=!0;b.crossorigin=\"anonymous\";b.src=\"https://js.driftt.com/include/\"+d+\"/\"+c+\".js\";c=document.getElementsByTagName(\"script\")[0];c.parentNode.insertBefore(b,c)}}}();drift.SNIPPET_VERSION=\"0.3.1\";drift.load(\"gmup7bc7xmk3\");</script>\n<script type=\"text/javascript\" id=\"\" src=\"//cdn.bizible.com/scripts/bizible.js\"></script><script type=\"text/javascript\" id=\"\">window._6si=window._6si||[];window._6si.push([\"enableEventTracking\",!0]);window._6si.push([\"setToken\",\"ee948a18a3c80b8c30cd25f1d0565f3c\"]);window._6si.push([\"setEndpoint\",\"b.6sc.co\"]);(function(){var a=document.createElement(\"script\");a.type=\"text/javascript\";a.async=!0;a.src=\"//j.6sc.co/6si.min.js\";var b=document.getElementsByTagName(\"script\")[0];b.parentNode.insertBefore(a,b)})();</script><div style=\"width:0px; height:0px; display:none; visibility:hidden;\" id=\"batBeacon134486682309\"><img style=\"width:0px; height:0px; display:none; visibility:hidden;\" id=\"batBeacon212966988868\" width=\"0\" height=\"0\" alt=\"\" src=\"https://bat.bing.com/action/0?ti=4029085&amp;Ver=2&amp;mid=956af6d6-6d61-42b9-8760-4e036d7c9990&amp;sid=fe5eaf30819911eb98b0c11b71032c7e&amp;vid=fe5ebd80819911eba1497d4ea32e6c1d&amp;vids=0&amp;pi=0&amp;lg=en-US&amp;sw=1920&amp;sh=1080&amp;sc=24&amp;tl=Accountant%20Salary%20in%20India%20%7C%20PayScale&amp;p=https%3A%2F%2Fwww.payscale.com%2Fresearch%2FIN%2FJob%3DAccountant%2FSalary&amp;r=https%3A%2F%2Fwww.payscale.com%2Fresearch%2FIN%2FJob%2FAccounting-and-Finance&amp;lt=702&amp;evt=pageLoad&amp;msclkid=N&amp;sv=1&amp;rn=281040\"></div><div style=\"width:0px; height:0px; display:none; visibility:hidden;\" id=\"batBeacon424116386983\"><img style=\"width:0px; height:0px; display:none; visibility:hidden;\" id=\"batBeacon345600420190\" width=\"0\" height=\"0\" alt=\"\" src=\"https://bat.bing.com/action/0?ti=4024402&amp;Ver=2&amp;mid=a1a18261-f4f7-44b4-802d-4fe89e75e2c1&amp;sid=fe5eaf30819911eb98b0c11b71032c7e&amp;vid=fe5ebd80819911eba1497d4ea32e6c1d&amp;vids=0&amp;pi=0&amp;lg=en-US&amp;sw=1920&amp;sh=1080&amp;sc=24&amp;tl=Accountant%20Salary%20in%20India%20%7C%20PayScale&amp;p=https%3A%2F%2Fwww.payscale.com%2Fresearch%2FIN%2FJob%3DAccountant%2FSalary&amp;r=https%3A%2F%2Fwww.payscale.com%2Fresearch%2FIN%2FJob%2FAccounting-and-Finance&amp;lt=702&amp;evt=pageLoad&amp;msclkid=N&amp;sv=1&amp;rn=528348\"></div>\n\n<script type=\"text/javascript\" id=\"\">(function(){var b=0<document.getElementsByClassName(\"ADPLogo\").length;if(!b&&\"undefined\"!==typeof Munchkin){b=\"417-NLK-080\";var e=window.location.pathname.toLowerCase(),d=[\"/hr\",\"/biz/\",\"/compensation\",\"/payscale-index\",\"/createaccount.aspx\"],a;if(!(a=\"marketrate.payscale.com\"===window.location.hostname.toLowerCase()||\"/\"===window.location.pathname)){a=!1;for(var c=0;c<d.length&&!a;c++)a=e.substr(0,d[c].length)===d[c]}a?Munchkin.init(b):Munchkin.init(b,{cookieAnon:!1})}})();</script><img alt=\"\" src=\"https://cdn.bizible.com/m/ipv?_biz_r=https%3A%2F%2Fwww.payscale.com%2Fresearch%2FIN%2FJob%2FAccounting-and-Finance&amp;_biz_h=802059049&amp;_biz_u=c5d881485d3d485da585260f36966122&amp;_biz_s=82cf2e&amp;_biz_l=https%3A%2F%2Fwww.payscale.com%2Fresearch%2FIN%2FJob%3DAccountant%2FSalary&amp;_biz_t=1615378399043&amp;_biz_i=Accountant%20Salary%20in%20India%20%7C%20PayScale&amp;_biz_n=1&amp;rnd=42196&amp;cdn_o=a&amp;_biz_z=1615378399045\" style=\"display: none;\"><div id=\"onetrust-consent-sdk\"><div class=\"onetrust-pc-dark-filter ot-hide ot-fade-in\"></div><div id=\"onetrust-pc-sdk\" class=\"otPcTab ot-hide ot-fade-in\" role=\"dialog\" aria-modal=\"true\" aria-labelledby=\"ot-pc-title\" lang=\"en-US\"><!-- pc header --><div class=\"ot-pc-header\" role=\"presentation\"><!-- Header logo --><div class=\"ot-pc-logo\" role=\"img\" aria-label=\"Company Logo\" style=\"background-image: url(&quot;https://cdn.cookielaw.org/logos/8b9f65a8-036c-46b0-b3ee-9bebb016b192/c02c8860-29d2-4deb-b5c7-341ba3e8f39c/7b664feb-495c-446a-b1aa-7cf77f3e2f8b/payscale-logo-black.png&quot;);\n                    background-position: left\"></div><div class=\"ot-title-cntr\"><h2 id=\"ot-pc-title\">Cookie Preferences</h2><div class=\"ot-close-cntr\"><button id=\"close-pc-btn-handler\" class=\"ot-close-icon\" aria-label=\"Close\"></button></div></div></div><!-- content --><!-- Groups / Sub groups with cookies --><div id=\"ot-pc-content\" class=\"ot-pc-scrollbar ot-sdk-row\"><div class=\"ot-sdk-container ot-grps-cntr ot-sdk-column\"><div class=\"ot-sdk-four ot-sdk-columns ot-tab-list\" aria-label=\"Cookie Categories\"><ul class=\"ot-cat-grp\" role=\"tablist\"><li class=\"ot-abt-tab\" role=\"presentation\"><!-- About Privacy container --><div class=\"ot-active-menu category-menu-switch-handler\" role=\"tab\" tabindex=\"0\" aria-selected=\"true\" aria-controls=\"ot-tab-desc\"><h3 id=\"ot-pvcy-txt\">Your Privacy</h3></div></li><li class=\"ot-cat-item\" role=\"presentation\" data-optanongroupid=\"C0005\"><div class=\"category-menu-switch-handler\" role=\"tab\" tabindex=\"-1\" aria-selected=\"false\" aria-controls=\"ot-desc-id-C0005\"><h3 id=\"ot-header-id-C0005\">Social Media Cookies</h3></div></li><li class=\"ot-cat-item\" role=\"presentation\" data-optanongroupid=\"C0004\"><div class=\"category-menu-switch-handler\" role=\"tab\" tabindex=\"-1\" aria-selected=\"false\" aria-controls=\"ot-desc-id-C0004\"><h3 id=\"ot-header-id-C0004\">Targeting Cookies</h3></div></li><li class=\"ot-cat-item\" role=\"presentation\" data-optanongroupid=\"C0003\"><div class=\"category-menu-switch-handler\" role=\"tab\" tabindex=\"-1\" aria-selected=\"false\" aria-controls=\"ot-desc-id-C0003\"><h3 id=\"ot-header-id-C0003\">Functional Cookies</h3></div></li><li class=\"ot-cat-item\" role=\"presentation\" data-optanongroupid=\"C0002\"><div class=\"category-menu-switch-handler\" role=\"tab\" tabindex=\"-1\" aria-selected=\"false\" aria-controls=\"ot-desc-id-C0002\"><h3 id=\"ot-header-id-C0002\">Performance Cookies</h3></div></li><li class=\"ot-cat-item ot-always-active-group\" role=\"presentation\" data-optanongroupid=\"C0001\"><div class=\"category-menu-switch-handler\" role=\"tab\" tabindex=\"-1\" aria-selected=\"false\" aria-controls=\"ot-desc-id-C0001\"><h3 id=\"ot-header-id-C0001\">Strictly Necessary Cookies</h3></div></li></ul></div><div class=\"ot-tab-desc ot-sdk-eight ot-sdk-columns\"><div class=\"ot-desc-cntr\" id=\"ot-tab-desc\" tabindex=\"0\" role=\"tabpanel\" aria-labelledby=\"ot-pvcy-hdr\"><h3 id=\"ot-pvcy-hdr\">Your Privacy</h3><p id=\"ot-pc-desc\" class=\"ot-grp-desc\">When you visit any website, it may store or retrieve information on your browser, mostly in the form of cookies. This information might be about you, your preferences or your device and is mostly used to make the site work as you expect it to. The information does not usually directly identify you, but it can give you a more personalized web experience. Because we respect your right to privacy, you can choose not to allow some types of cookies. Click on the different category headings to find out more and change our default settings. However, blocking some types of cookies may impact your experience of the site and the services we are able to offer.\n                        <a href=\"https://www.payscale.com/cookie-policy\" class=\"privacy-notice-link\" target=\"_blank\" aria-label=\"More information, Opens in a new window\">More information</a></p></div><div class=\"ot-desc-cntr ot-hide\" role=\"tabpanel\" tabindex=\"0\" id=\"ot-desc-id-C0005\"><div class=\"ot-grp-hdr1\"><h3 class=\"ot-cat-header\">Social Media Cookies</h3><div class=\"ot-tgl\"><input type=\"checkbox\" name=\"ot-group-id-C0005\" id=\"ot-group-id-C0005\" aria-checked=\"true\" role=\"switch\" class=\"category-switch-handler\" data-optanongroupid=\"C0005\" checked=\"\" aria-labelledby=\"ot-header-id-C0005\"> <label class=\"ot-switch\" for=\"ot-group-id-C0005\"><span class=\"ot-switch-nob\"></span> <span class=\"ot-label-txt\">Social Media Cookies</span></label> </div><div class=\"ot-tgl-cntr\"></div></div><p class=\"ot-grp-desc ot-category-desc\">These cookies are set by a range of social media services that we have added to the site to enable you to share our content with your friends and networks. They are capable of tracking your browser across other sites and building up a profile of your interests. This may impact the content and messages you see on other websites you visit. If you do not allow these cookies you may not be able to use or see these sharing tools.</p></div><div class=\"ot-desc-cntr ot-hide\" role=\"tabpanel\" tabindex=\"0\" id=\"ot-desc-id-C0004\"><div class=\"ot-grp-hdr1\"><h3 class=\"ot-cat-header\">Targeting Cookies</h3><div class=\"ot-tgl\"><input type=\"checkbox\" name=\"ot-group-id-C0004\" id=\"ot-group-id-C0004\" aria-checked=\"true\" role=\"switch\" class=\"category-switch-handler\" data-optanongroupid=\"C0004\" checked=\"\" aria-labelledby=\"ot-header-id-C0004\"> <label class=\"ot-switch\" for=\"ot-group-id-C0004\"><span class=\"ot-switch-nob\"></span> <span class=\"ot-label-txt\">Targeting Cookies</span></label> </div><div class=\"ot-tgl-cntr\"></div></div><p class=\"ot-grp-desc ot-category-desc\">These cookies may be set through our site by our advertising partners. They may be used by those companies to build a profile of your interests and show you relevant adverts on other sites. They do not store directly personal information, but are based on uniquely identifying your browser and internet device. If you do not allow these cookies, you will experience less targeted advertising.</p></div><div class=\"ot-desc-cntr ot-hide\" role=\"tabpanel\" tabindex=\"0\" id=\"ot-desc-id-C0003\"><div class=\"ot-grp-hdr1\"><h3 class=\"ot-cat-header\">Functional Cookies</h3><div class=\"ot-tgl\"><input type=\"checkbox\" name=\"ot-group-id-C0003\" id=\"ot-group-id-C0003\" aria-checked=\"true\" role=\"switch\" class=\"category-switch-handler\" data-optanongroupid=\"C0003\" checked=\"\" aria-labelledby=\"ot-header-id-C0003\"> <label class=\"ot-switch\" for=\"ot-group-id-C0003\"><span class=\"ot-switch-nob\"></span> <span class=\"ot-label-txt\">Functional Cookies</span></label> </div><div class=\"ot-tgl-cntr\"></div></div><p class=\"ot-grp-desc ot-category-desc\">These cookies enable the website to provide enhanced functionality and personalisation. They may be set by us or by third party providers whose services we have added to our pages. If you do not allow these cookies then some or all of these services may not function properly.</p></div><div class=\"ot-desc-cntr ot-hide\" role=\"tabpanel\" tabindex=\"0\" id=\"ot-desc-id-C0002\"><div class=\"ot-grp-hdr1\"><h3 class=\"ot-cat-header\">Performance Cookies</h3><div class=\"ot-tgl\"><input type=\"checkbox\" name=\"ot-group-id-C0002\" id=\"ot-group-id-C0002\" aria-checked=\"true\" role=\"switch\" class=\"category-switch-handler\" data-optanongroupid=\"C0002\" checked=\"\" aria-labelledby=\"ot-header-id-C0002\"> <label class=\"ot-switch\" for=\"ot-group-id-C0002\"><span class=\"ot-switch-nob\"></span> <span class=\"ot-label-txt\">Performance Cookies</span></label> </div><div class=\"ot-tgl-cntr\"></div></div><p class=\"ot-grp-desc ot-category-desc\">These cookies allow us to count visits and traffic sources so we can measure and improve the performance of our site. They help us to know which pages are the most and least popular and see how visitors move around the site. All information these cookies collect is aggregated and therefore anonymous. If you do not allow these cookies we will not know when you have visited our site, and will not be able to monitor its performance.</p></div><div class=\"ot-desc-cntr ot-hide ot-always-active-group\" role=\"tabpanel\" tabindex=\"0\" id=\"ot-desc-id-C0001\"><div class=\"ot-grp-hdr1\"><h3 class=\"ot-cat-header\">Strictly Necessary Cookies</h3><div class=\"ot-tgl-cntr\"><div class=\"ot-always-active\">Always Active</div></div></div><p class=\"ot-grp-desc ot-category-desc\">These cookies are necessary for the website to function and cannot be switched off in our systems. They are usually only set in response to actions made by you which amount to a request for services, such as setting your privacy preferences, logging in or filling in forms. You can set your browser to block or alert you about these cookies, but some parts of the site will not then work. These cookies do not store any personally identifiable information.</p></div></div></div></div><!-- Vendors / Hosts --><section id=\"ot-pc-lst\" class=\"ot-hide ot-enbl-chr\"><div class=\"ot-lst-cntr ot-pc-scrollbar\"><div id=\"ot-pc-hdr\"><h3 id=\"ot-lst-title\"><button class=\"ot-link-btn back-btn-handler\" aria-label=\"Back\"><svg id=\"ot-back-arw\" xmlns=\"http://www.w3.org/2000/svg\" xmlns:xlink=\"http://www.w3.org/1999/xlink\" x=\"0px\" y=\"0px\" viewBox=\"0 0 444.531 444.531\" xml:space=\"preserve\"><title>Back Button</title><g><path fill=\"#656565\" d=\"M213.13,222.409L351.88,83.653c7.05-7.043,10.567-15.657,10.567-25.841c0-10.183-3.518-18.793-10.567-25.835\n                  l-21.409-21.416C323.432,3.521,314.817,0,304.637,0s-18.791,3.521-25.841,10.561L92.649,196.425\n                  c-7.044,7.043-10.566,15.656-10.566,25.841s3.521,18.791,10.566,25.837l186.146,185.864c7.05,7.043,15.66,10.564,25.841,10.564\n                  s18.795-3.521,25.834-10.564l21.409-21.412c7.05-7.039,10.567-15.604,10.567-25.697c0-10.085-3.518-18.746-10.567-25.978\n                  L213.13,222.409z\"></path></g></svg></button> <span>Back</span></h3><div class=\"ot-lst-subhdr\"><div id=\"ot-search-cntr\"><label for=\"vendor-search-handler\" class=\"ot-scrn-rdr\">Vendor Search</label> <input id=\"vendor-search-handler\" aria-label=\"Vendor Search\" type=\"text\" placeholder=\"Search...\" name=\"vendor-search-handler\"> <svg xmlns=\"http://www.w3.org/2000/svg\" xmlns:xlink=\"http://www.w3.org/1999/xlink\" x=\"0px\" y=\"0px\" viewBox=\"0 -30 110 110\"><path fill=\"#2e3644\" d=\"M55.146,51.887L41.588,37.786c3.486-4.144,5.396-9.358,5.396-14.786c0-12.682-10.318-23-23-23s-23,10.318-23,23\n              s10.318,23,23,23c4.761,0,9.298-1.436,13.177-4.162l13.661,14.208c0.571,0.593,1.339,0.92,2.162,0.92\n              c0.779,0,1.518-0.297,2.079-0.837C56.255,54.982,56.293,53.08,55.146,51.887z M23.984,6c9.374,0,17,7.626,17,17s-7.626,17-17,17\n              s-17-7.626-17-17S14.61,6,23.984,6z\"></path></svg></div><div id=\"ot-fltr-cntr\"><button id=\"filter-btn-handler\" aria-label=\"Filter\" aria-haspopup=\"true\"><svg role=\"presentation\" aria-hidden=\"true\" xmlns=\"http://www.w3.org/2000/svg\" xmlns:xlink=\"http://www.w3.org/1999/xlink\" x=\"0px\" y=\"0px\" viewBox=\"0 0 402.577 402.577\" style=\"enable-background:new 0 0 402.577 402.577;\" xml:space=\"preserve\"><title>Filter Button</title><g><path fill=\"#2c3643\" d=\"M400.858,11.427c-3.241-7.421-8.85-11.132-16.854-11.136H18.564c-7.993,0-13.61,3.715-16.846,11.136\n                            c-3.234,7.801-1.903,14.467,3.999,19.985l140.757,140.753v138.755c0,4.955,1.809,9.232,5.424,12.854l73.085,73.083\n                            c3.429,3.614,7.71,5.428,12.851,5.428c2.282,0,4.66-0.479,7.135-1.43c7.426-3.238,11.14-8.851,11.14-16.845V172.166L396.861,31.413\n                            C402.765,25.895,404.093,19.231,400.858,11.427z\"></path></g></svg></button></div></div></div><section id=\"ot-lst-cnt\" class=\"ot-pc-scrollbar\"><div class=\"ot-sdk-row\"><div class=\"ot-sdk-column\"><div id=\"ot-sel-blk\"><div class=\"ot-sel-all\"><div class=\"ot-sel-all-hdr\"><span class=\"ot-consent-hdr\">Consent</span> <span class=\"ot-li-hdr\">Leg.Interest</span></div><div class=\"ot-sel-all-chkbox\"><div class=\"ot-chkbox\" id=\"ot-selall-hostcntr\"><input id=\"select-all-hosts-groups-handler\" type=\"checkbox\" aria-checked=\"false\"> <label for=\"select-all-hosts-groups-handler\"><span class=\"ot-label-txt\">checkbox label</span></label> <span class=\"ot-label-status\">label</span></div><div class=\"ot-chkbox\" id=\"ot-selall-vencntr\"><input id=\"select-all-vendor-groups-handler\" type=\"checkbox\" aria-checked=\"false\"> <label for=\"select-all-vendor-groups-handler\"><span class=\"ot-label-txt\">checkbox label</span></label> <span class=\"ot-label-status\">label</span></div><div class=\"ot-chkbox\" id=\"ot-selall-licntr\"><input id=\"select-all-vendor-leg-handler\" type=\"checkbox\" aria-checked=\"false\"> <label for=\"select-all-vendor-leg-handler\"><span class=\"ot-label-txt\">checkbox label</span></label> <span class=\"ot-label-status\">label</span></div></div></div></div></div></div></section></div><div id=\"ot-anchor\"></div><section id=\"ot-fltr-modal\"><div id=\"ot-fltr-cnt\"><button id=\"clear-filters-handler\">Clear</button><div class=\"ot-fltr-scrlcnt ot-pc-scrollbar\"><div class=\"ot-fltr-opts\"><div class=\"ot-fltr-opt\"><div class=\"ot-chkbox\"><input id=\"chkbox-id\" type=\"checkbox\" aria-checked=\"false\" class=\"category-filter-handler\"> <label for=\"chkbox-id\"><span class=\"ot-label-txt\">checkbox label</span></label> <span class=\"ot-label-status\">label</span></div></div></div><div class=\"ot-fltr-btns\"><button id=\"filter-apply-handler\">Apply</button> <button id=\"filter-cancel-handler\">Cancel</button></div></div></div></section></section><!-- Footer buttons and logo --><div class=\"ot-pc-footer\"><div class=\"ot-btn-container\"><button class=\"save-preference-btn-handler onetrust-close-btn-handler\">Confirm My Choices</button><div class=\"ot-btn-subcntr\"> <button id=\"accept-recommended-btn-handler\">Allow All</button></div></div><div class=\"ot-pc-footer-logo\"><a href=\"https://onetrust.com/poweredbyonetrust\" target=\"_blank\" rel=\"noopener noreferrer\" style=\"background-image: url(&quot;https://cdn.cookielaw.org/logos/static/poweredBy_ot_logo.svg&quot;)\" aria-label=\"Powered by OneTrust Opens in a new window\"></a></div></div><!-- Cookie subgroup container --><!-- Vendor list link --><!-- Cookie lost link --><!-- Toggle HTML element --><!-- Checkbox HTML --><!-- Arrow SVG element --><!-- Accordion basic element --><span class=\"ot-scrn-rdr\" aria-atomic=\"true\" aria-live=\"polite\"></span><iframe class=\"ot-text-resize\" title=\"onetrust-text-resize\" style=\"position:absolute;top:-50000px;width:100em;\" aria-hidden=\"true\"></iframe></div></div><div id=\"drift-frame-chat\" class=\"drift-conductor-item drift-frame-chat drift-frame-chat-align-right with-transition\" style=\"right: 12px; width: 400px !important; max-width: none; position: fixed; z-index: 2147483647; max-height: 0px; bottom: 88px; visibility: hidden; min-width: 320px; height: 100% !important;\"><iframe class=\"drift-frame-chat\" scrolling=\"no\" title=\"Drift Widget Chat Window\" allow=\"autoplay; encrypted-media; fullscreen;\" frameborder=\"0\" src=\"https://js.driftt.com/core/chat\" style=\"background: transparent; width: 400px !important; max-width: 100%; height: 100%; border: none; min-width: 100%; min-height: 100%;\"></iframe></div><div id=\"drift-frame-controller\" class=\"drift-conductor-item drift-frame-controller drift-frame-controller-align-right\" style=\"right: 24px; width: 76px !important; max-width: none; position: fixed; z-index: 2147483647; height: 0px !important; bottom: 24px;\"><iframe class=\"drift-frame-controller\" scrolling=\"no\" title=\"Drift Widget Icon\" allow=\"autoplay; encrypted-media; fullscreen;\" frameborder=\"0\" src=\"https://js.driftt.com/core?embedId=gmup7bc7xmk3&amp;forceShow=false&amp;skipCampaigns=false&amp;sessionId=37436689-9e14-4914-afc0-be4a390d592c&amp;sessionStarted=1615378400&amp;campaignRefreshToken=677202ec-8df1-4dc8-9a08-9e5f8dd03d6e&amp;pageLoadStartTime=1615378397596\" style=\"background: transparent; width: 100%; max-width: 100%; height: 100%; border: none; min-width: 100%; min-height: 100%;\"></iframe></div><iframe src=\"https://tpc.googlesyndication.com/sodar/sodar2/221/runner.html\" width=\"0\" height=\"0\" style=\"display: none;\"></iframe></body></html>"""

In [4]:
sel = Selector(text = h)

salary_dict = dict()

# Extract the job title
job_title = 'demo'

# Extract the average salary
avg_salary = sel.css('span.paycharts__value::text').extract_first()

# Extract base salary, bonus, profit sharing, commission and total pay 
salary_component_title_list = sel.css('div.tablerow__title::text').extract()
salary_component_value_list = sel.css('div.tablerow__value::text').extract()

# Extract pay by experience percent level
pay_percent_component_title_list = sel.css('div.delta-table.pay-by-experience__delta-chart > div.entry > div.name > a::text').extract()
pay_percent_sign_list = sel.css('div.delta-table.pay-by-experience__delta-chart > div.entry > div.arrow').xpath('./span/@class').extract()
pay_percent_value_list_without_sign = sel.css('div.delta-table.pay-by-experience__delta-chart > div.entry').xpath('./div[@class="arrow"]/text()').extract()
pay_percent_component_value_list = list()
for sign, value in zip(pay_percent_sign_list, pay_percent_value_list_without_sign):
    if sign == 'negative':
        pay_percent_component_value_list.append('-' + value)
    else:
        pay_percent_component_value_list.append(value)

# Extract pay by experience years level
pay_by_experience_years_text = sel.xpath('//p[@class="expandable__paragraph"]/text()').extract_first()
pay_by_experience_years_component_title_list = list()
pay_by_experience_years_component_value_list = list()

temp_numbers_list = list()
for text in pay_by_experience_years_text.split():
    if any(char.isdigit() for char in text):
        temp_numbers_list.append(text)

for i in range(0, len(temp_numbers_list) - 2, 3):
    pay_by_experience_years_component_title_list.append(temp_numbers_list[i])
    pay_by_experience_years_component_value_list.append(temp_numbers_list[i + 1])

# Extract Gender Breakdown Percentage
gender_component_title_list = sel.css('div.gender__label::text').extract()
temp_gender_component_value_list = sel.css('div.gender__value::text').extract()
gender_component_value_list = list()
for i in range(0, len(temp_gender_component_value_list), 2):
    gender_component_value_list.append(temp_gender_component_value_list[i])

# Extract Health benefits item name
health_benefits_component_title_list = sel.css('div.healthbenefits__item-name::text').extract()
temp_health_benefits_component_value_list = sel.css('h2.healthbenefits__item-value::text').extract()
health_benefits_component_value_list = list()
for i in range(0, len(temp_health_benefits_component_value_list), 2):
    health_benefits_component_value_list.append(temp_health_benefits_component_value_list[i])

# Popular Skills
popular_skills_component_title_list = sel.css('a.popular-skill__name::text').extract()
popular_skills_component_value_list = sel.css('div.popular-skill__pay::text').extract()

# Extract Skills that affect salary
affect_skills_component_title_list = sel.css('div.pay-diff-by-dimension div.name > a::text').extract()
temp_affect_skills_component_value_list = sel.css('div.pay-diff-by-dimension div.arrow::text').extract()
temp_sign = sel.css('div.pay-diff-by-dimension div.arrow').xpath('./span/@class').extract()
affect_skills_component_value_list = list()
for sign, value in zip(temp_sign, temp_affect_skills_component_value_list):
    if sign == 'negative':
        affect_skills_component_value_list.append('-' + value)
    else:
        affect_skills_component_value_list.append(value)

# Extract Career Paths
career_path_component_title_list = sel.css('div.tree-chart__leaf_container').xpath('./*[contains(@class,"tree-chart__leaf")]/text()').extract()

# Extract pay difference by location
location_component_title_list = sel.css('div.carousel.related-carousel.pay-diff-related-carousel div.related-content-card__title::text').extract()
temp_location_component_value_list = sel.css('div.carousel.related-carousel.pay-diff-related-carousel div.related-content-card__subtitle::text').extract()
location_component_value_list = list()
for value in temp_location_component_value_list:
    if value[0] == '▼':
        location_component_value_list.append('-' + value[2:])
    else:
        location_component_value_list.append(value[2:])

# Extract popular employers
employers_component_title_list = sel.css('div.carousel.related-carousel.popular-employers-related-carousel div.related-content-card__title::text').extract()
employers_component_value_list = sel.css('div.carousel.related-carousel.popular-employers-related-carousel div.related-content-card__subtitle::text').extract()

# Extract Related Job Salaries
related_job_component_title_list = sel.css('span.relatedentity__card-title::text').extract()
related_job_component_value_list = sel.css('p.relatedentity__card-detail::text').extract()

# Updating all the values in the dictionary
salary_dict['job_title'] = job_title
salary_dict['avg_salary'] = avg_salary
salary_dict['salary_component'] = salary_component_title_list
for label, value in zip(salary_component_title_list, salary_component_value_list):
    salary_dict[label] = value
salary_dict['pay_by_experience_percent'] = pay_percent_component_title_list
for label, value in zip(pay_percent_component_title_list, pay_percent_component_value_list):
    salary_dict[label] = value
salary_dict['pay_by_experience_years'] =  pay_by_experience_years_component_title_list
for label, value in zip( pay_by_experience_years_component_title_list,  pay_by_experience_years_component_value_list):
    salary_dict[label] = value
salary_dict['gender_component'] =  gender_component_title_list
for label, value in zip( gender_component_title_list,  gender_component_value_list):
    salary_dict[label] = value
salary_dict['health_benefits_componentt'] =  health_benefits_component_title_list
for label, value in zip( health_benefits_component_title_list,  health_benefits_component_value_list):
    salary_dict[label] = value
salary_dict['popular_skills_component'] =  popular_skills_component_title_list
for label, value in zip( popular_skills_component_title_list,  popular_skills_component_value_list):
    salary_dict['Popular ' + label] = value
salary_dict['affect_skills_component'] =  affect_skills_component_title_list
for label, value in zip( affect_skills_component_title_list,  affect_skills_component_value_list):
    salary_dict['Affect ' + label] = value
salary_dict['career_path_component'] = career_path_component_title_list
salary_dict['location_component'] =  location_component_title_list[:10]
for label, value in zip( location_component_title_list[:10], location_component_value_list[:10]):
    salary_dict[label] = value
salary_dict['employers_component'] =  employers_component_title_list[:10]
for label, value in zip(employers_component_title_list[:10],  employers_component_value_list[:10]):
    salary_dict[label] = value
salary_dict['related_job_component'] =  related_job_component_title_list
for label, value in zip(related_job_component_title_list[:10], related_job_component_value_list[:10]):
    salary_dict[label] = value

# # Storing the values in json file
df = pd.DataFrame.from_dict(salary_dict, orient='index')
df.to_json('' + job_title + '.json')
print(salary_dict)

{'job_title': 'demo', 'avg_salary': '₹249,659', 'salary_component': ['Base Salary', 'Bonus', 'Profit Sharing', 'Commission', 'Total Pay'], 'Base Salary': '₹123k - ₹506k', 'Bonus': '₹5k - ₹60k', 'Profit Sharing': '₹41 - ₹198k', 'Commission': '₹3k - ₹99k', 'Total Pay': '₹129k - ₹584k', 'pay_by_experience_percent': ['Entry Level', 'Early Career', 'Mid Career', 'Late Career', 'Experienced'], 'Entry Level': '-20%', 'Early Career': '-7%', 'Mid Career': '19%', 'Late Career': '42%', 'Experienced': '58%', 'pay_by_experience_years': ['1', '1-4'], '1': '₹198,740', '1-4': '₹231,272', 'gender_component': ['Male', 'Female', 'Prefer to self-define'], 'Male': '78.5', 'Female': '21.4', 'Prefer to self-define': '0.1', 'health_benefits_componentt': ['Medical', 'Dental', 'Vision', 'None'], 'Medical': '34', 'Dental': '3', 'Vision': '3', 'None': '66', 'popular_skills_component': [], 'affect_skills_component': [], 'career_path_component': [], 'location_component': [], 'employers_component': [], 'related_job_

In [ ]:
len(html)

## Retail (Not included)
Optician%2C_Retail_Store_Manager , Branch_Manager%2C_Rental_Services , Divisional_Merchandising_Manager , Information_%26_Ticketing_Manager%2C_Arts_(but_not_Cinema) , Sales_Consultant%2C_Mobile_Telecommunications